In [0]:
delete from hive_metastore.default.widetable_4g_site_gold2_tmp
;


insert into hive_metastore.default.widetable_4g_site_gold2_tmp
with bh_conf as (
  select SUB_MARKET_ID,REGION_SILVER_TB, Busy_Hr,bhcode from hive_metastore.default.vz_conf_market A
  inner join (
    select DAILY,REGION,Busy_Hr, sum(bhcode) bhcode from (            
        select DAILY,REGION,LTE_Busy_Hour 	Busy_Hr,1 bhcode from snap.enb_iris.4g_busy_hour_kpi_gold where Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")
        union all
        select DAILY,REGION,VoLTE_Busy_Hour Busy_Hr,2 bhcode from snap.enb_iris.4g_busy_hour_kpi_gold where Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")        
    ) x group by DAILY,REGION,Busy_Hr
  ) B on A.REGION_SILVER_TB=B.REGION
)
SELECT 
convert_timezone('UTC','America/Chicago',current_timestamp())  Created_DTM,
decode(bhcode,null,0,bhcode) Busy_Hr,
/*X.NE_ID,*/
case when left(X.NE_ID,5)=='UADPF' and length(Y.SUB_MARKET_ID) = 3 then  replace(concat(left(X.NE_ID,9),right(X.NE_ID,3)),'UADPF','eNB') when left(X.NE_ID,5)=='UADPF' and length(Y.SUB_MARKET_ID) = 2 then replace(concat(left(X.NE_ID,8),right(X.NE_ID,3)),'UADPF','eNB')  else X.NE_ID  end NE_ID,            
X.SYSTEM_ID2 SYSTEM_ID,
X.LOCATION,D_DTM,Timezone,replace(SITE_ID,'N/A',null) SITE_ID,
/*case when length(ne_ID)=5 then substring(ne_id, 1, 2)
     when length(ne_ID)=6 then substring(ne_id, 1, 3)
     when length(ne_ID)=9 then substring(ne_id, 5, 2)
     when length(ne_ID)=10 then substring(ne_id,5, 3)
     when length(ne_ID)=16 then substring(ne_id, 7, 2)
     when length(ne_ID)=17 then substring(ne_id, 7, 3) end SITE_ID,*/ 
Y.MARKET_NAME,
Y.SUB_MARKET_ID,
Y.SUB_MARKET_NAME,
Y.BAND_NAME,     
max(RRC_connection_failure_rate_NUM) RRC_connection_failure_rate_NUM,
max(RRC_connection_failure_rate_DENOM) RRC_connection_failure_rate_DENOM,
max(Total_ERAB_Drop_Rate_NUM) Total_ERAB_Drop_Rate_NUM,
max(Total_ERAB_Drop_Rate_DENOM) Total_ERAB_Drop_Rate_DENOM,
max(VoLTE_Drop_Rate_NUM) VoLTE_Drop_Rate_NUM,
max(VoLTE_Drop_Rate_DENOM) VoLTE_Drop_Rate_DENOM,
max(dl_user_throughput_num) dl_user_throughput_num,
max(dl_user_throughput_denom) dl_user_throughput_denom,
max(ul_user_throughput_num) ul_user_throughput_num,
max(ul_user_throughput_denom) ul_user_throughput_denom,
max(volte_dl_pdcp_packet_loss_num) volte_dl_pdcp_packet_loss_num,
max(volte_dl_pdcp_packet_loss_denom) volte_dl_pdcp_packet_loss_denom,
max(avg_rrc_connected_users_num) avg_rrc_connected_users_num,
max(avg_rrc_connected_users_denom) avg_rrc_connected_users_denom,
max(dl_rlc_data_volume_num) dl_rlc_data_volume,
max(ul_rlc_data_volume_num) ul_rlc_data_volume,
max(RRC_Connection_Attempts_NUM) RRC_Connection_Attempts_NUM,
max(endc_add_attempt_num) endc_add_attempt_num,
max(SEA_ENDC_SgNBAdd_Fail_num) SEA_ENDC_SgNBAdd_Fail_num,
max(SEA_ENDC_SgNBAdd_Fail_denom) SEA_ENDC_SgNBAdd_Fail_denom,
max(SEA_ENDC_SCG_Failure_NUM) SEA_ENDC_SCG_Failure_NUM,
max(SEA_ENDC_SCG_Failure_DENOM) SEA_ENDC_SCG_Failure_DENOM,
max(ERAB_Setup_Failure_Rate_NUM) ERAB_Setup_Failure_Rate_NUM,
max(ERAB_Setup_Failure_Rate_DENOM) ERAB_Setup_Failure_Rate_DENOM,
max(Context_Setup_Failure_NUM) Context_Setup_Failure_NUM,
max(Context_Setup_Failure_DENOM) Context_Setup_Failure_DENOM,
max(RRC_connection_Drop_Rate_NUM) RRC_connection_Drop_Rate_NUM,
max(RRC_connection_Drop_Rate_DENOM) RRC_connection_Drop_Rate_DENOM,
max(Context_Drop_Rate_NUM) Context_Drop_Rate_NUM,
max(Context_Drop_Rate_DENOM) Context_Drop_Rate_DENOM,
max(RRC_Reestablishment_Rate_NUM) RRC_Reestablishment_Rate_NUM,
max(RRC_Reestablishment_Rate_DENUM) RRC_Reestablishment_Rate_DENUM,
max(RRC_Reest_Failure_Rate_NUM) RRC_Reest_Failure_Rate_NUM,
max(RRC_Reest_Failure_Rate_DENOM) RRC_Reest_Failure_Rate_DENOM,
max(Intra_eNB_HOFR_NUM) Intra_eNB_HOFR_NUM,
max(Intra_eNB_HOFR_DENOM) Intra_eNB_HOFR_DENOM,
max(X2_Total_HOFR_NUM) X2_Total_HOFR_NUM,
max(X2_Total_HOFR_DENOM) X2_Total_HOFR_DENOM,
max(S1_HOFR_NUM) S1_HOFR_NUM,
max(S1_HOFR_DENOM) S1_HOFR_DENOM,
max(Total_DL_RLC_ReTx_NUM) Total_DL_RLC_ReTx_NUM,
max(Total_DL_RLC_ReTx_DENOM) Total_DL_RLC_ReTx_DENOM,
max(TTI_Bundling_Activation_FR_NUM) TTI_Bundling_Activation_FR_NUM,
max(TTI_Bundling_Activation_FR_DENOM) TTI_Bundling_Activation_FR_DENOM,
max(DL_Residual_BLER_NUM) DL_Residual_BLER_NUM,
max(DL_Residual_BLER_DENOM) DL_Residual_BLER_DENOM,
max(UL_Residual_BLER_NUM) UL_Residual_BLER_NUM,
max(UL_Residual_BLER_DENOM) UL_Residual_BLER_DENOM,
max(VoLTE_DL_Residual_BLER_NUM) VoLTE_DL_Residual_BLER_NUM,
max(VoLTE_DL_Residual_BLER_DENOM) VoLTE_DL_Residual_BLER_DENOM,
max(VoLTE_UL_Residual_BLER_NUM) VoLTE_UL_Residual_BLER_NUM,
max(VoLTE_UL_Residual_BLER_DENOM) VoLTE_UL_Residual_BLER_DENOM,
max(PDCCH_DL_Blocking_NUM) PDCCH_DL_Blocking_NUM,
max(PDCCH_DL_Blocking_DENOM) PDCCH_DL_Blocking_DENOM,
max(PDCCH_UL_Blocking_NUM) PDCCH_UL_Blocking_NUM,
max(PDCCH_UL_Blocking_DENOM) PDCCH_UL_Blocking_DENOM,
max(Intra_eNB_Total_HO_Attempts) Intra_eNB_Total_HO_Attempts,
max(X2_Total_HO_Attempts) X2_Total_HO_Attempts,
max(S1_HO_Attempt) S1_HO_Attempt,
max(DL_PRB_Utilization_NUM) DL_PRB_Utilization_NUM,
max(DL_PRB_Utilization_DENOM) DL_PRB_Utilization_DENOM,
max(UL_PRB_Utilization_NUM) UL_PRB_Utilization_NUM,
max(UL_PRB_Utilization_DENOM) UL_PRB_Utilization_DENOM,
max(Avg_TTI_Bundling_Users_NUM) Avg_TTI_Bundling_Users_NUM,
max(Avg_TTI_Bundling_Users_DENOM) Avg_TTI_Bundling_Users_DENOM,
max(Avg_Volte_Users_NUM) Avg_Volte_Users_NUM,
max(Avg_Volte_Users_DENOM) Avg_Volte_Users_DENOM,
max(Data_Drop_Rate_NUM) Data_Drop_Rate_NUM,
max(Data_Drop_Rate_DENOM) Data_Drop_Rate_DENOM,
max(max_rrc_connected_users) max_rrc_connected_users,
max(RRC_Connection_HoldTime_s_NUM) RRC_Connection_HoldTime_s_NUM,
max(RRC_Connection_HoldTime_s_DENOM) RRC_Connection_HoldTime_s_DENOM,
max(DL_MCS_NUM) DL_MCS_NUM,
max(DL_MCS_DENOM) DL_MCS_DENOM,
max(MAC_DL_Cell_Throughput_NUM) MAC_DL_Cell_Throughput_NUM,
max(MAC_DL_Cell_Throughput_DENOM) MAC_DL_Cell_Throughput_DENOM,
max(Inter_Freq_HOFR_NUM) Inter_Freq_HOFR_NUM,
max(Inter_Freq_HOFR_DENOM) Inter_Freq_HOFR_DENOM,
max(PUCCH_SINR_NUM) PUCCH_SINR_NUM,
max(PUCCH_SINR_DENOM) PUCCH_SINR_DENOM,
max(PUSCH_SINR_NUM) PUSCH_SINR_NUM,
max(PUSCH_SINR_DENOM) PUSCH_SINR_DENOM,
max(UL_MCS_NUM) UL_MCS_NUM,
max(UL_MCS_DENOM) UL_MCS_DENOM,
max(MAC_UL_Cell_Throughput_NUM) MAC_UL_Cell_Throughput_NUM,
max(MAC_UL_Cell_Throughput_DENOM) MAC_UL_Cell_Throughput_DENOM,
max(Avg_CQI_NUM) Avg_CQI_NUM,
max(Avg_CQI_DENOM) Avg_CQI_DENOM,
max(Avg_CA_Active_Length_per_activation_attempt_NUM) Avg_CA_Active_Length_per_activation_attempt_NUM,
max(Avg_CA_Active_Length_per_activation_attempt_DENOM) Avg_CA_Active_Length_per_activation_attempt_DENOM,
max(CA_activation_rate_NUM) CA_activation_rate_NUM,
max(CA_activation_rate_DENOM) CA_activation_rate_DENOM,
max(DL_Init_BLER_NUM) DL_Init_BLER_NUM,
max(DL_Init_BLER_DENOM) DL_Init_BLER_DENOM,
max(DL_User_Perceived_Throughput_CA_NUM) DL_User_Perceived_Throughput_CA_NUM,
max(DL_User_Perceived_Throughput_CA_DENOM) DL_User_Perceived_Throughput_CA_DENOM,
max(UL_Init_BLER_NUM) UL_Init_BLER_NUM,
max(UL_Init_BLER_DENOM) UL_Init_BLER_DENOM,
max(DL_MAC_Layer_Data_Volume_pCell_NUM) DL_MAC_Layer_Data_Volume_pCell,
max(DL_MAC_Layer_Data_Volume_sCell_NUM) DL_MAC_Layer_Data_Volume_sCell,
max(UL_MAC_Volume_MB_NUM) UL_MAC_Volume_MB,
max(Average_Timing_Advance_NUM) Average_Timing_Advance_NUM,
max(Average_Timing_Advance_DENOM) Average_Timing_Advance_DENOM,
max(SEA_CAConfigurationRate_NUM) SEA_CAConfigurationRate_NUM,
max(SEA_CAConfigurationRate_DENOM) SEA_CAConfigurationRate_DENOM,
max(SEA_CAConfigurationFailure_pct_NUM) SEA_CAConfigurationFailure_pct_NUM,
max(SEA_CAConfigurationFailure_pct_DENOM) SEA_CAConfigurationFailure_pct_DENOM,
max(NB_IOT_RRC_Conn_Setup_Failure_Rate_NUM) NB_IOT_RRC_Conn_Setup_Failure_Rate_NUM,
max(NB_IOT_RRC_Conn_Setup_Failure_Rate_DENOM) NB_IOT_RRC_Conn_Setup_Failure_Rate_DENOM,
max(NB_IOT_Avg_RRC_Connections_NUM) NB_IOT_Avg_RRC_Connections_NUM,
max(NB_IOT_Avg_RRC_Connections_DENOM) NB_IOT_Avg_RRC_Connections_DENOM,
max(NB_IOT_DL_MAC_PDU_Tput_Kbps_NUM) NB_IOT_DL_MAC_PDU_Tput_Kbps_NUM,
max(NB_IOT_DL_MAC_PDU_Tput_Kbps_DENOM) NB_IOT_DL_MAC_PDU_Tput_Kbps_DENOM,
max(NB_IOT_UL_MAC_PDU_Tput_Kbps_NUM) NB_IOT_UL_MAC_PDU_Tput_Kbps_NUM,
max(NB_IOT_UL_MAC_PDU_Tput_Kbps_DENOM) NB_IOT_UL_MAC_PDU_Tput_Kbps_DENOM,
max(NB_IOT_RRC_Paging_Request_Atts) NB_IOT_RRC_Paging_Request_Atts,
max(NBIoT_Total_ERAB_Drop_Rate_pct_NUM) NBIoT_Total_ERAB_Drop_Rate_pct_NUM,
max(NBIoT_Total_ERAB_Drop_Rate_pct_DENOM) NBIoT_Total_ERAB_Drop_Rate_pct_DENOM,
max(NBIoT_DL_Initial_BLER_pct_NUM) NBIoT_DL_Initial_BLER_pct_NUM,
max(NBIoT_DL_Initial_BLER_pct_DENOM) NBIoT_DL_Initial_BLER_pct_DENOM,
max(NBIoT_DL_Residual_BLER_pct_NUM) NBIoT_DL_Residual_BLER_pct_NUM,
max(NBIoT_DL_Residual_BLER_pct_DENOM) NBIoT_DL_Residual_BLER_pct_DENOM,
max(NBIoT_UL_Initial_BLER_pct_NUM) NBIoT_UL_Initial_BLER_pct_NUM,
max(NBIoT_UL_Initial_BLER_pct_DENOM) NBIoT_UL_Initial_BLER_pct_DENOM,
max(NBIoT_UL_Residual_BLER_pct_NUM) NBIoT_UL_Residual_BLER_pct_NUM,
max(NBIoT_UL_Residual_BLER_pct_DENOM) NBIoT_UL_Residual_BLER_pct_DENOM,
max(NBIoT_MaxRRCconnUsers_CeLv0) NBIoT_MaxRRCconnUsers_CeLv0,
max(NBIoT_MaxRRCconnUsers_CeLv1) NBIoT_MaxRRCconnUsers_CeLv1,
max(NBIoT_MaxRRCconnUsers_CeLv2) NBIoT_MaxRRCconnUsers_CeLv2,
max(NBIoT_Air_RLC_Volume) NBIoT_Air_RLC_Volume,
max(eMTC_Downlink_PDCP_Volume_NUM) eMTC_Downlink_PDCP_Volume,
max(eMTC_Downlink_PDCP_Throughput_NUM) eMTC_Downlink_PDCP_Throughput_NUM,
max(eMTC_Downlink_PDCP_Throughput_DENOM) eMTC_Downlink_PDCP_Throughput_DENOM,
max(eMTC_Avg_RRC_Connected_Users_NUM) eMTC_Avg_RRC_Connected_Users_NUM,
max(eMTC_Avg_RRC_Connected_Users_DENOM) eMTC_Avg_RRC_Connected_Users_DENOM,
max(eMTC_Max_RRC_Connected_Users) eMTC_Max_RRC_Connected_Users,
max(eMTC_Paging_Attempt) eMTC_Paging_Attempt,
max(eMTC_Total_ERAB_Setup_Failure_Rate_NUM) eMTC_Total_ERAB_Setup_Failure_Rate_NUM,
max(eMTC_Total_ERAB_Setup_Failure_Rate_DENOM) eMTC_Total_ERAB_Setup_Failure_Rate_DENOM,
max(eMTC_RRC_Connection_Drop_Rate_NUM) eMTC_RRC_Connection_Drop_Rate_NUM,
max(eMTC_RRC_Connection_Drop_Rate_DENOM) eMTC_RRC_Connection_Drop_Rate_DENOM,
max(eMTC_Total_ERAB_Drop_Rate_NUM) eMTC_Total_ERAB_Drop_Rate_NUM,
max(eMTC_Total_ERAB_Drop_Rate_DENOM) eMTC_Total_ERAB_Drop_Rate_DENOM,
max(eMTC_Context_Setup_Failure_NUM) eMTC_Context_Setup_Failure_NUM,
max(eMTC_Context_Setup_Failure_DENOM) eMTC_Context_Setup_Failure_DENOM,
max(eMTC_Context_Drop_Rate_NUM) eMTC_Context_Drop_Rate_NUM,
max(eMTC_Context_Drop_Rate_DENOM) eMTC_Context_Drop_Rate_DENOM,
max(eMTC_Intra_Freq_HO_Fail_Rate_NUM) eMTC_Intra_Freq_HO_Fail_Rate_NUM,
max(eMTC_Intra_Freq_HO_Fail_Rate_DENOM) eMTC_Intra_Freq_HO_Fail_Rate_DENOM,
max(eMTC_DL_Initial_BLER_pct_NUM) eMTC_DL_Initial_BLER_pct_NUM,
max(eMTC_DL_Initial_BLER_pct_DENOM) eMTC_DL_Initial_BLER_pct_DENOM,
max(eMTC_DL_Residual_BLER_pct_NUM) eMTC_DL_Residual_BLER_pct_NUM,
max(eMTC_DL_Residual_BLER_pct_DENOM) eMTC_DL_Residual_BLER_pct_DENOM,
max(eMTC_UL_Initial_BLER_pct_NUM) eMTC_UL_Initial_BLER_pct_NUM,
max(eMTC_UL_Initial_BLER_pct_DENOM) eMTC_UL_Initial_BLER_pct_DENOM,
max(eMTC_UL_Residual_BLER_pct_NUM) eMTC_UL_Residual_BLER_pct_NUM,
max(eMTC_UL_Residual_BLER_pct_DENOM) eMTC_UL_Residual_BLER_pct_DENOM,
max(eMTC_RRC_Setup_Failure_pct_NUM) eMTC_RRC_Setup_Failure_pct_NUM,
max(eMTC_RRC_Setup_Failure_pct_DENOM) eMTC_RRC_Setup_Failure_pct_DENOM,
max(ENDC_Cap_UE_Avg_RRC_Connected_Users_NUM) ENDC_Cap_UE_Avg_RRC_Connected_Users_NUM,
max(ENDC_Cap_UE_Avg_RRC_Connected_Users_DENOM) ENDC_Cap_UE_Avg_RRC_Connected_Users_DENOM,
max(ENDC_Cap_UE_RRC_Drop_pct_NUM) ENDC_Cap_UE_RRC_Drop_pct_NUM,
max(ENDC_Cap_UE_RRC_Drop_pct_DENOM) ENDC_Cap_UE_RRC_Drop_pct_DENOM, 
max(SEA_ENDC_SgNBAdd_Fail_PCT_NCR_NUM) SEA_ENDC_SgNBAdd_Fail_PCT_NCR_NUM,  -- from add 
max(SEA_ENDC_SgNBAdd_Fail_PCT_NCR_DENOM) SEA_ENDC_SgNBAdd_Fail_PCT_NCR_DENOM,
max(DL_MAC_Layer_Data_Volume_CA) DL_MAC_Layer_Data_Volume_CA,
max(Cell_Availability_PCT_NUM) Cell_Availability_PCT_NUM,
max(Cell_Availability_PCT_DENOM) Cell_Availability_PCT_DENOM,
max(NB_IOT_Max_Num_of_RRC_Conn_UE) NB_IOT_Max_Num_of_RRC_Conn_UE,
-- add
max(DL_MAC_Volume_MB) DL_MAC_Volume_MB,
max(SEA_ENDC_SgNBAddPrep_Fail_PCT_Num) SEA_ENDC_SgNBAddPrep_Fail_PCT_Num,
max(SEA_ENDC_SgNBAddPrep_Fail_PCT_Denom) SEA_ENDC_SgNBAddPrep_Fail_PCT_Denom,
max(SEA_ENDC_CapUE_AvgEnDcUEAddition_Num) SEA_ENDC_CapUE_AvgEnDcUEAddition_Num,
max(SEA_ENDC_CapUE_AvgEnDcUEAddition_Denom) SEA_ENDC_CapUE_AvgEnDcUEAddition_Denom,
max(SEA_ENDC_CapUE_AvgRRCConnectionHoldTime_s_Num) SEA_ENDC_CapUE_AvgRRCConnectionHoldTime_s_Num,
max(SEA_ENDC_CapUE_AvgRRCConnectionHoldTime_s_Denom) SEA_ENDC_CapUE_AvgRRCConnectionHoldTime_s_Denom,
max(SEA_ENDC_NRsyncRecon_Fail_PCT_NCR_Num) SEA_ENDC_NRsyncRecon_Fail_PCT_NCR_Num,
max(SEA_ENDC_NRsyncRecon_Fail_PCT_NCR_Denom) SEA_ENDC_NRsyncRecon_Fail_PCT_NCR_Denom,
max(SEA_ENDC_SgNBAddPrep_Fail_PCT_NCR_Num) SEA_ENDC_SgNBAddPrep_Fail_PCT_NCR_Num,
max(SEA_ENDC_SgNBAddPrep_Fail_PCT_NCR_Denom) SEA_ENDC_SgNBAddPrep_Fail_PCT_NCR_Denom,
max(volte_ul_pdcp_packet_loss_num) volte_ul_pdcp_packet_loss_num,
max(volte_ul_pdcp_packet_loss_denom) volte_ul_pdcp_packet_loss_denom,
-- add qci
max(DL_User_Throughput_QCI8_NUM) DL_User_Throughput_QCI8_NUM,
max(Dl_User_Throughput_QCI8_DENOM) Dl_User_Throughput_QCI8_DENOM,
max(Dl_Rlc_Data_volume_QCI8) Dl_Rlc_Data_volume_QCI8,
max(Ul_Rlc_Data_volume_QCI8) Ul_Rlc_Data_volume_QCI8,
max(ERAB_Setup_Failure_Rate_QCI5_NUM) ERAB_Setup_Failure_Rate_QCI5_NUM,
max(ERAB_Setup_Failure_Rate_QCI5_DENOM) ERAB_Setup_Failure_Rate_QCI5_DENOM,
max(Total_ERAB_Drop_Rate_QCI5_NUM) Total_ERAB_Drop_Rate_QCI5_NUM,
max(Total_ERAB_Drop_Rate_QCI5_DENOM) Total_ERAB_Drop_Rate_QCI5_DENOM,
-- add
max(ASEU) ASEU,
max(CellCallContextCntAvg_Num) CellCallContextCntAvg_Num,
max(CellCallContextCntAvg_Denom) CellCallContextCntAvg_Denom,
max(CellCallContextCntMax) CellCallContextCntMax,
-- add
max(NBIoT_Air_RLC_Volume_UL) NBIoT_Air_RLC_Volume_UL,
max(NB_IOT_Context_Drop_Rate_Num) NB_IOT_Context_Drop_Rate_Num,
max(NB_IOT_Context_Drop_Rate_Denom) NB_IOT_Context_Drop_Rate_Denom,
-- add
max(UL_MAC_Layer_Data_Volume_pCell) UL_MAC_Layer_Data_Volume_pCell,
max(UL_MAC_Layer_Data_Volume_sCell) UL_MAC_Layer_Data_Volume_sCell,
-- add DENG-2770
max(	SEA_ENDC_CapUE_AvgRRCconnectedUsers_Num	)	SEA_ENDC_CapUE_AvgRRCconnectedUsers_Num,
max(	SEA_ENDC_CapUE_AvgRRCconnectedUsers_Denom          	)	SEA_ENDC_CapUE_AvgRRCconnectedUsers_Denom,          
max(	5G_Fraction_of_RRC_Call_Hold_Time_pct_Denom	)	5G_Fraction_of_RRC_Call_Hold_Time_pct_Denom,
max(	SEA_ENDC_CapUE_MaxRRCconnectedUsers	)	SEA_ENDC_CapUE_MaxRRCconnectedUsers,
max(	SEA_ENDC_CapUE_MaxEnDcUEAddition          	)	SEA_ENDC_CapUE_MaxEnDcUEAddition,          
max(	SEA_ENDC_DSS_LTE_DL_PRB_Ratio_pct_Denom	)	SEA_ENDC_DSS_LTE_DL_PRB_Ratio_pct_Denom,
max(	SEA_ENDC_DSS_NR_DL_PRB_Ratio_pct_Denom	)	SEA_ENDC_DSS_NR_DL_PRB_Ratio_pct_Denom,
max(	SEA_ENDC_DSS_LTE_UL_PRB_Ratio_pct_Denom	)	SEA_ENDC_DSS_LTE_UL_PRB_Ratio_pct_Denom,
max(	SEA_ENDC_DSS_NR_UL_PRB_Ratio_pct_Denom	)	SEA_ENDC_DSS_NR_UL_PRB_Ratio_pct_Denom,
max(	SEA_ENDC_DSS_LTE_DL_TTI_Ratio_pct_Denom	)	SEA_ENDC_DSS_LTE_DL_TTI_Ratio_pct_Denom,
max(	SEA_ENDC_DSS_NR_DL_TTI_Ratio_pct_Denom	)	SEA_ENDC_DSS_NR_DL_TTI_Ratio_pct_Denom,
max(	SEA_ENDC_DSS_LTE_UL_TTI_Ratio_pct_Denom	)	SEA_ENDC_DSS_LTE_UL_TTI_Ratio_pct_Denom,
max(	SEA_ENDC_DSS_NR_UL_TTI_Ratio_pct_Denom	)	SEA_ENDC_DSS_NR_UL_TTI_Ratio_pct_Denom,
max(ReadCellUnavailableTimeLock_s_SUM) ReadCellUnavailableTimeLock_s_SUM,
max(ReadCellUnavailableTimeES_s_SUM) ReadCellUnavailableTimeES_s_SUM,
max(CellAvailPmPeriodTime_s_SUM) CellAvailPmPeriodTime_s_SUM,
max(ConnEstabReject_CpCcFail) RRC_Rejection_CpCc,
max(ConnEstabReject_CpCapaCacFail) RRC_Rejection_CapaCac,
max(HPA_RRC_Setup_Failure_Rate_Num) HPA_RRC_Setup_Failure_Rate_Num,
max(HPA_RRC_Setup_Failure_Rate_Denom) HPA_RRC_Setup_Failure_Rate_Denom,
max(HPA_ERAB_Setup_Failure_Rate_Num) HPA_ERAB_Setup_Failure_Rate_Num,
max(HPA_ERAB_Setup_Failure_Rate_Denom) HPA_ERAB_Setup_Failure_Rate_Denom,
max(HPA_ERAB_Drop_Rate_Num) HPA_ERAB_Drop_Rate_Num,
max(HPA_ERAB_Drop_Rate_Denom) HPA_ERAB_Drop_Rate_Denom,
max(HPA_DL_User_Throughput_QCI7_NUM) HPA_DL_User_Throughput_Num,
max(HPA_DL_User_Throughput_QCI7_DENOM) HPA_DL_User_Throughput_Denom,
max(HPA_DL_RLC_Vol) HPA_DL_RLC_Vol,
max(HPA_UL_RLC_Vol) HPA_UL_RLC_Vol
from ( 	 
SELECT  cast(replace(replace(K.NE_ID,'eNB_',''),'UADPF_','') as int) SYSTEM_ID2,* from (
        SELECT 
          COALESCE(M0.NE_ID,T1.NE_ID,T2.NE_ID,T3.NE_ID,T4.NE_ID,T5.NE_ID,T6.NE_ID,T7.NE_ID,T8.NE_ID,T9.NE_ID,T10.NE_ID,T11.NE_ID,T12.NE_ID,T13.NE_ID,T14.NE_ID,T15.NE_ID,T16.NE_ID,T17.NE_ID,T18.NE_ID,T19.NE_ID,T20.NE_ID,T21.NE_ID,T22.NE_ID,T23.NE_ID,T24.NE_ID,T25.NE_ID,T26.NE_ID,T27.NE_ID,T28.NE_ID,T29.NE_ID,T30.NE_ID,T31.NE_ID,T32.NE_ID,T33.NE_ID,T34.NE_ID,T35.NE_ID,T36.NE_ID,T37.NE_ID,T38.NE_ID,T39.NE_ID,T40.NE_ID,T41.NE_ID,T42.NE_ID,T43.NE_ID,T44.NE_ID,T45.NE_ID,T46.NE_ID,T47.NE_ID,T48.NE_ID,T49.NE_ID,T50.NE_ID,T51.NE_ID,T52.NE_ID,T53.NE_ID,T54.NE_ID,T55.NE_ID,T56.NE_ID,T57.NE_ID,T58.NE_ID,T59.NE_ID,T60.NE_ID,T61.NE_ID,T62.NE_ID,T63.NE_ID,T64.NE_ID,T65.NE_ID) NE_ID,          
          COALESCE(M0.LOCATION,T1.LOCATION,T2.LOCATION,T3.LOCATION,T4.LOCATION,T5.LOCATION,T6.LOCATION,T7.LOCATION,T8.LOCATION,T9.LOCATION,T10.LOCATION,T11.LOCATION,T12.LOCATION,T13.LOCATION,T14.LOCATION,T15.LOCATION,T16.LOCATION,T17.LOCATION,T18.LOCATION,T19.LOCATION,T20.LOCATION,T21.LOCATION,T22.LOCATION,T23.LOCATION,T24.LOCATION,T25.LOCATION,T26.LOCATION,T27.LOCATION,T28.LOCATION,T29.LOCATION,T30.LOCATION,T31.LOCATION,T32.LOCATION,T33.LOCATION,T34.LOCATION,T35.LOCATION,T36.LOCATION,T37.LOCATION,T38.LOCATION,T39.LOCATION,T40.LOCATION,T41.LOCATION,T42.LOCATION,T43.LOCATION,T44.LOCATION,T45.LOCATION,T46.LOCATION,T47.LOCATION,T48.LOCATION,T49.LOCATION,T50.LOCATION,T51.LOCATION,T52.LOCATION,T53.LOCATION,T54.LOCATION,T55.LOCATION,T56.LOCATION,T57.LOCATION,T58.LOCATION,T59.LOCATION,T60.LOCATION,T61.LOCATION,T62.LOCATION,T63.LOCATION,T64.LOCATION,T65.LOCATION) LOCATION,
          COALESCE(M0.D_DTM,T1.D_DTM,T2.D_DTM,T3.D_DTM,T4.D_DTM,T5.D_DTM,T6.D_DTM,T7.D_DTM,T8.D_DTM,T9.D_DTM,T10.D_DTM,T11.D_DTM,T12.D_DTM,T13.D_DTM,T14.D_DTM,T15.D_DTM,T16.D_DTM,T17.D_DTM,T18.D_DTM,T19.D_DTM,T20.D_DTM,T21.D_DTM,T22.D_DTM,T23.D_DTM,T24.D_DTM,T25.D_DTM,T26.D_DTM,T27.D_DTM,T28.D_DTM,T29.D_DTM,T30.D_DTM,T31.D_DTM,T32.D_DTM,T33.D_DTM,T34.D_DTM,T35.D_DTM,T36.D_DTM,T37.D_DTM,T38.D_DTM,T39.D_DTM,T40.D_DTM,T41.D_DTM,T42.D_DTM,T43.D_DTM,T44.D_DTM,T45.D_DTM,T46.D_DTM,T47.D_DTM,T48.D_DTM,T49.D_DTM,T50.D_DTM,T51.D_DTM,T52.D_DTM,T53.D_DTM,T54.D_DTM,T55.D_DTM,T56.D_DTM,T57.D_DTM,T58.D_DTM,T59.D_DTM,T60.D_DTM,T61.D_DTM,T62.D_DTM,T63.D_DTM,T64.D_DTM,T65.D_DTM) D_DTM,
          COALESCE(M0.Timezone,T1.Timezone,T2.Timezone,T3.Timezone,T4.Timezone,T5.Timezone,T6.Timezone,T7.Timezone,T8.Timezone,T9.Timezone,T10.Timezone,T11.Timezone,T12.Timezone,T13.Timezone,T14.Timezone,T15.Timezone,T16.Timezone,T17.Timezone,T18.Timezone,T19.Timezone,T20.Timezone,T21.Timezone,T22.Timezone,T23.Timezone,T24.Timezone,T25.Timezone,T26.Timezone,T27.Timezone,T28.Timezone,T29.Timezone,T30.Timezone,T31.Timezone,T32.Timezone,T33.Timezone,T34.Timezone,T35.Timezone,T36.Timezone,T37.Timezone,T38.Timezone,T39.Timezone,T40.Timezone,T41.Timezone,T42.Timezone,T43.Timezone,T44.Timezone,T45.Timezone,T46.Timezone,T47.Timezone,T48.Timezone,T49.Timezone,T50.Timezone,T51.Timezone,T52.Timezone,T53.Timezone,T54.Timezone,T55.Timezone,T56.Timezone,T57.Timezone,T58.Timezone,T59.Timezone,T60.Timezone,T61.Timezone,T62.Timezone,T63.Timezone,T64.Timezone,T65.Timezone) Timezone,
          COALESCE(M0.SITE_ID,T1.SITE_ID,T2.SITE_ID,T3.SITE_ID,T4.SITE_ID,T5.SITE_ID,T6.SITE_ID,T7.SITE_ID,T8.SITE_ID,T9.SITE_ID,T10.SITE_ID,T11.SITE_ID,T12.SITE_ID,T13.SITE_ID,T14.SITE_ID,T15.SITE_ID,T16.SITE_ID,T17.SITE_ID,T18.SITE_ID,T19.SITE_ID,T20.SITE_ID,T21.SITE_ID,T22.SITE_ID,T23.SITE_ID,T24.SITE_ID,T25.SITE_ID,T26.SITE_ID,T27.SITE_ID,T28.SITE_ID,T29.SITE_ID,T30.SITE_ID,T31.SITE_ID,T32.SITE_ID,T33.SITE_ID,T34.SITE_ID,T35.SITE_ID,T36.SITE_ID,T37.SITE_ID,T38.SITE_ID,T39.SITE_ID,T40.SITE_ID,T41.SITE_ID,T42.SITE_ID,T43.SITE_ID,T44.SITE_ID,T45.SITE_ID,T46.SITE_ID,T47.SITE_ID,T48.SITE_ID,T49.SITE_ID,T50.SITE_ID,T51.SITE_ID,T52.SITE_ID,T53.SITE_ID,T54.SITE_ID,T55.SITE_ID,T56.SITE_ID,T57.SITE_ID,T58.SITE_ID,T59.SITE_ID,T60.SITE_ID,T61.SITE_ID,T62.SITE_ID,T63.SITE_ID,T64.SITE_ID,T65.SITE_ID) SITE_ID,
          RrcEstabMsg_RrcConnectionRequest-RrcEstabMsg_RrcConnectionSetupComplete as RRC_connection_failure_rate_NUM,
          RrcEstabMsg_RrcConnectionRequest as RRC_connection_failure_rate_DENOM,
          NVL(RelAbnormalbyEnbQci,0) + NVL(RelAbnormalbyMmeActQci,0) as Total_ERAB_Drop_Rate_NUM,
          NVL((RelNormalbyEnbQci + RelAbnormalbyEnbQci),0) + NVL((RelNormalbyMmeQci + RelAbnormalbyMmeQci),0) as Total_ERAB_Drop_Rate_DENOM,
          NVL(RelAbnormalbyEnbQci_QCI1,0) + NVL(RelAbnormalbyMmeActQci_QCI1,0) as VoLTE_Drop_Rate_NUM,
          NVL((RelNormalbyEnbQci_QCI1 + RelAbnormalbyEnbQci_QCI1),0) + NVL((RelNormalbyMmeQci_QCI1 + RelAbnormalbyMmeQci_QCI1),0) as VoLTE_Drop_Rate_DENOM,
          `dl user throughput_num` dl_user_throughput_num,
          `dl user throughput_denom` dl_user_throughput_denom,
          `ul user throughput_num` ul_user_throughput_num,
          `ul user throughput_denom` ul_user_throughput_denom,
          `volte dl pdcp packet loss num` volte_dl_pdcp_packet_loss_num,
          `volte dl pdcp packet loss denom` volte_dl_pdcp_packet_loss_denom,
          `avg. rrc connected users num` avg_rrc_connected_users_num,
          `avg. rrc connected users denom` avg_rrc_connected_users_denom,
          `dl rlc data volume num` dl_rlc_data_volume_num,          
          `ul rlc data volume num` ul_rlc_data_volume_num,          
          RrcEstabMsg_RrcConnectionRequest RRC_Connection_Attempts_NUM,          
          EnDc_AddAtt endc_add_attempt_num,          
          (EnDc_AddAtt - EnDc_AddSucc) SEA_ENDC_SgNBAdd_Fail_num,
          EnDc_AddAtt SEA_ENDC_SgNBAdd_Fail_denom,
          (EnDc_RelRequest_UeContxtRel+EnDc_RelRequest_InternalResouFail+EnDc_RelRequest_DL_RLF_randomAccessProblem+EnDc_RelRequest_DL_RLF_rlc_MaxNumRetx+EnDc_RelRequest_DL_RLF_scg_reconfigFailure+EnDc_RelRequest_DL_RLF_srb3_IntegrityFailure+
          EnDc_RelRequest_DL_RLF_t310_Expiry+EnDc_RelRequest_UL_RLF_RadioConnection_UeLost+EnDc_RelRequire_BearerNotSupp+EnDc_RelRequire_DcOverallTO+EnDc_RelRequire_UL_RLF_RadioConnection_UeLost+
          EnDc_RelRequest_DL_RLF_synchReconfigFailure_SCG) as SEA_ENDC_SCG_Failure_NUM,
          EnDc_AddSucc as SEA_ENDC_SCG_Failure_DENOM,--1
          (EstabInitAttNbr + EstabAddAttNbr - ErabAddFailNbr_CpCcInteraction) -  (EstabInitSuccNbr + EstabAddSuccNbr ) as ERAB_Setup_Failure_Rate_NUM,
          (EstabInitAttNbr + EstabAddAttNbr -ErabAddFailNbr_CpCcInteraction)  ERAB_Setup_Failure_Rate_DENOM,
          (Tx_S1apInitialContextSetupFailure+S1ContextSetupFailure) Context_Setup_Failure_NUM,
          Rx_S1apInitialContextSetupRequest Context_Setup_Failure_DENOM,
          (RlfDetection_RadioLinkFailure + RlfDetection_ArqMaxReTransmission) RRC_connection_Drop_Rate_NUM,
          (ConnEstabSucc) RRC_connection_Drop_Rate_DENOM,
          (Context_RelAbnormalByEnb+Context_RelAbnormalByMmeAct) Context_Drop_Rate_NUM,
          (Context_RelNormalByEnb + Context_RelNormalByMme +  Context_RelAbnormalByMme + Context_RelAbnormalByEnb) Context_Drop_Rate_DENOM,
           ConnReEstabAtt_3cause RRC_Reestablishment_Rate_NUM,
           RrcRrcConnectionSetupComplete RRC_Reestablishment_Rate_DENUM,
          (ConnReEstabAtt-ConnReEstabSucc ) RRC_Reest_Failure_Rate_NUM,(ConnReEstabAtt) RRC_Reest_Failure_Rate_DENOM, 
          (IntraEnbAtt-IntraEnbPrepFail_CpCapaCacFail-IntraEnbSucc) Intra_eNB_HOFR_NUM,
          (IntraEnbAtt-IntraEnbPrepFail_CpCapaCacFail) Intra_eNB_HOFR_DENOM,
          (InterX2OutAtt - InterX2OutSucc) X2_Total_HOFR_NUM,
          (InterX2OutAtt) X2_Total_HOFR_DENOM,
          (InterS1OutAtt - InterS1OutSucc) S1_HOFR_NUM,
          (InterS1OutAtt) S1_HOFR_DENOM,
          `Total DL RLC ReTx NUM` Total_DL_RLC_ReTx_NUM,
          `Total DL RLC ReTx DENOM` Total_DL_RLC_ReTx_DENOM,
          (TtibActAtt-TtibActSucc) TTI_Bundling_Activation_FR_NUM,
          TtibActAtt TTI_Bundling_Activation_FR_DENOM,
          (DLTransmissionNackedRetrans + DLTransmissionNackedRetrans_256QAM) DL_Residual_BLER_NUM,
          (DLTransmissionRetrans0 + DLTransmissionRetrans0_256QAM) DL_Residual_BLER_DENOM,
          (ULTransmissionNackedRetrans + ULTransmissionNackedRetrans_256QAM) UL_Residual_BLER_NUM,
          (ULTransmissionRetrans0 + ULTransmissionRetrans0_256QAM) UL_Residual_BLER_DENOM,
          ( DLVoLTEHARQFail ) VoLTE_DL_Residual_BLER_NUM, 
          ( DLVoLTEHARQFail + DLVoLTEHARQSucc ) VoLTE_DL_Residual_BLER_DENOM,
          ( ULVoLTEHARQFail )  VoLTE_UL_Residual_BLER_NUM,
          ( ULVoLTEHARQFail + ULVoLTEHARQSucc ) VoLTE_UL_Residual_BLER_DENOM,
          RruCceAllocationFailDL PDCCH_DL_Blocking_NUM,
          RruCceAllocationDL PDCCH_DL_Blocking_DENOM,
          RruCceAllocationFailUL PDCCH_UL_Blocking_NUM,
          RruCceAllocationUL PDCCH_UL_Blocking_DENOM,
          (IntraEnbAtt-IntraEnbPrepFail_CpCapaCacFail) Intra_eNB_Total_HO_Attempts,
          (InterX2OutAtt) X2_Total_HO_Attempts ,
          (InterS1OutAtt) S1_HO_Attempt,
          TotPrbDLTot DL_PRB_Utilization_NUM,
          TotPrbDLCnt DL_PRB_Utilization_DENOM,
          TotPrbULTot UL_PRB_Utilization_NUM,
          TotPrbULCnt UL_PRB_Utilization_DENOM,
          TtibNoTot Avg_TTI_Bundling_Users_NUM,
          TtibNoCnt Avg_TTI_Bundling_Users_DENOM, 
          UsageNbrTot Avg_Volte_Users_NUM,
          UsageNbrCnt Avg_Volte_Users_DENOM,
          NVL(RelAbnormalbyEnbQci_QCI5789,0) + NVL(RelAbnormalbyMmeActQci_QCI5789,0) as Data_Drop_Rate_NUM,
          NVL((RelNormalbyEnbQci_QCI5789 + RelAbnormalbyEnbQci_QCI5789),0) + NVL((RelNormalbyMmeQci_QCI5789 + RelAbnormalbyMmeQci_QCI5789),0) as Data_Drop_Rate_DENOM,      
          `Max. RRC Connected Users` max_rrc_connected_users,
          ReleaseCallHoldingTimeTot	RRC_Connection_HoldTime_s_NUM,
          ReleaseCallHoldingTimeCnt   RRC_Connection_HoldTime_s_DENOM,
          DL_MCS_NUM,
          DL_MCS_DENOM,
          MAC_DL_Cell_Throughput_NUM,
          MAC_DL_Cell_Throughput_DENOM,
          Inter_Freq_HOFR_NUM,
          Inter_Freq_HOFR_DENOM,
          PUCCH_SINR_NUM,
          PUCCH_SINR_DENOM,
          PUSCH_SINR_NUM,
          PUSCH_SINR_DENOM,
          UL_MCS_NUM,
          UL_MCS_DENOM,
          MAC_UL_Cell_Throughput_NUM,
          MAC_UL_Cell_Throughput_DENOM,
          Avg_CQI_NUM,
          Avg_CQI_DENOM,
          Avg_CA_Active_Length_per_activation_attempt_NUM,
          Avg_CA_Active_Length_per_activation_attempt_DENOM,
          Avg_CA_Active_Length_per_activation_attempt_NUM  CA_activation_rate_NUM,
          `Avg. RRC Connected Users NUM` * 2  CA_activation_rate_DENOM,
          DLTransmissionRetrans1 + DLTransmissionRetrans1_256QAM DL_Init_BLER_NUM,
          DLTransmissionRetrans0 + DLTransmissionRetrans0_256QAM DL_Init_BLER_DENOM,
          DL_User_Perceived_Throughput_CA_NUM,
          DL_User_Perceived_Throughput_CA_DENOM,
          ULTransmissionRetrans1 UL_Init_BLER_NUM,
          ULTransmissionRetrans0 UL_Init_BLER_DENOM,
          DL_MAC_Layer_Data_Volume_pCell_NUM,          
          DL_MAC_Layer_Data_Volume_sCell_NUM,          
          (nvl(AirMacULByte_P,0) + nvl(AirMacULByte_S,0)) UL_MAC_Volume_MB_NUM,          
          Average_Timing_Advance_NUM,
          Average_Timing_Advance_DENOM,
          SEA_CAConfigurationRate_NUM,
          ConnEstabSucc SEA_CAConfigurationRate_DENOM,
          (SCellAddAtt-SEA_CAConfigurationRate_NUM) SEA_CAConfigurationFailure_pct_NUM,
          SCellAddAtt  SEA_CAConfigurationFailure_pct_DENOM,
          NB_IOT_RRC_Conn_Setup_Failure_Rate_NUM,
          NB_IOT_RRC_Conn_Setup_Failure_Rate_DENOM,
          NB_IOT_Avg_RRC_Connections_NUM,
          NB_IOT_Avg_RRC_Connections_DENOM,
          NB_IOT_DL_MAC_PDU_Tput_Kbps_NUM,
          NB_IOT_DL_MAC_PDU_Tput_Kbps_DENOM,
          NB_IOT_UL_MAC_PDU_Tput_Kbps_NUM,
          NB_IOT_UL_MAC_PDU_Tput_Kbps_DENOM,
          NB_IOT_RRC_Paging_Request_Atts,
          NBIoT_Total_ERAB_Drop_Rate_pct_NUM,
          NBIoT_Total_ERAB_Drop_Rate_pct_DENOM,
          NBIoT_DL_Initial_BLER_pct_NUM,
          NBIoT_DL_Initial_BLER_pct_DENOM,            
          NBIoT_DL_Residual_BLER_pct_NUM,
          NBIoT_DL_Residual_BLER_pct_DENOM,
          NBIoT_UL_Initial_BLER_pct_NUM,
          NBIoT_UL_Initial_BLER_pct_DENOM,
          NBIoT_UL_Residual_BLER_pct_NUM,
          NBIoT_UL_Residual_BLER_pct_DENOM,
          NBIoT_MaxRRCconnUsers_CeLv0,
          NBIoT_MaxRRCconnUsers_CeLv1,
          NBIoT_MaxRRCconnUsers_CeLv2,
          NBIoT_Air_RLC_Volume,
          eMTC_Downlink_PDCP_Volume_NUM,          
          eMTC_Downlink_PDCP_Throughput_NUM,
          eMTC_Downlink_PDCP_Throughput_DENOM,
          eMTC_Avg_RRC_Connected_Users_NUM,
          eMTC_Avg_RRC_Connected_Users_DENOM,
          eMTC_Max_RRC_Connected_Users,
          eMTC_Paging_Attempt,
          eMTC_Total_ERAB_Setup_Failure_Rate_NUM,
          eMTC_Total_ERAB_Setup_Failure_Rate_DENOM,
          eMTC_RRC_Connection_Drop_Rate_NUM,
          eMTC_RRC_Connection_Drop_Rate_DENOM,
          eMTC_Total_ERAB_Drop_Rate_NUM,
          (eMTC_EstabInitSuccNbr+eMTC_EstabAddSuccNbr) eMTC_Total_ERAB_Drop_Rate_DENOM,
          (eMTC_S1ConnEstabAtt-eMTC_S1ConnEstabSucc)	 eMTC_Context_Setup_Failure_NUM,
          eMTC_S1ConnEstabAtt eMTC_Context_Setup_Failure_DENOM,
          eMTC_Total_ERAB_Drop_Rate_NUM eMTC_Context_Drop_Rate_NUM,
          eMTC_S1ConnEstabSucc  eMTC_Context_Drop_Rate_DENOM,
          eMTC_Intra_Freq_HO_Fail_Rate_NUM,
          eMTC_Intra_Freq_HO_Fail_Rate_DENOM,
          eMTC_DL_Initial_BLER_pct_NUM,
          eMTC_DL_Initial_BLER_pct_DENOM,
          eMTC_DL_Residual_BLER_pct_NUM,
          eMTC_DL_Residual_BLER_pct_DENOM,
          eMTC_UL_Initial_BLER_pct_NUM,
          eMTC_UL_Initial_BLER_pct_DENOM,
          eMTC_UL_Residual_BLER_pct_NUM,
          eMTC_UL_Residual_BLER_pct_DENOM,
          eMTC_RRC_Setup_Failure_pct_NUM,
          eMTC_RRC_Setup_Failure_pct_DENOM,
          ENDC_Cap_UE_Avg_RRC_Connected_Users_NUM,
          ENDC_Cap_UE_Avg_RRC_Connected_Users_DENOM,
          ENDC_Cap_UE_RRC_Drop_pct_NUM,
          ENDC_Cap_UE_RRC_Drop_pct_DENOM,
          SEA_ENDC_SgNBAdd_Fail_PCT_NCR_NUM,
          SEA_ENDC_SgNBAdd_Fail_PCT_NCR_DENOM,
          DL_MAC_Layer_Data_Volume_pCell_NUM+DL_MAC_Layer_Data_Volume_sCell_NUM DL_MAC_Layer_Data_Volume_CA,
          Cell_Availability_PCT_NUM,
          Cell_Availability_PCT_DENOM,
          NBIoT_ConnNoMax NB_IOT_Max_Num_of_RRC_Conn_UE,
          -- add
          MAC_DL_Cell_Throughput_NUM/8 DL_MAC_Volume_MB,
          EnDc_AddAtt-EnDc_AddPrepSucc SEA_ENDC_SgNBAddPrep_Fail_PCT_Num,
          EnDc_AddAtt SEA_ENDC_SgNBAddPrep_Fail_PCT_Denom,
          EnDcAdditionUeTot SEA_ENDC_CapUE_AvgEnDcUEAddition_Num,
          EnDcAdditionUeCnt SEA_ENDC_CapUE_AvgEnDcUEAddition_Denom,
          Release5GCallHoldingTimeTot SEA_ENDC_CapUE_AvgRRCConnectionHoldTime_s_Num,
          Release5GCallHoldingTimeCnt SEA_ENDC_CapUE_AvgRRCConnectionHoldTime_s_Denom,
          EnDc_AddFailSyncReconfigFail_NrNcr SEA_ENDC_NRsyncRecon_Fail_PCT_NCR_Num,
          EnDc_AddRrcReconfigSucc_NrNcr SEA_ENDC_NRsyncRecon_Fail_PCT_NCR_Denom,
          EnDc_AddPrepFail_NrNcr SEA_ENDC_SgNBAddPrep_Fail_PCT_NCR_Num,
          EnDc_AddAtt_NrNcr SEA_ENDC_SgNBAddPrep_Fail_PCT_NCR_Denom,
          `volte ul pdcp packet loss num` volte_ul_pdcp_packet_loss_num,
          `volte ul pdcp packet loss denom` volte_ul_pdcp_packet_loss_denom,
          -- add qci
          DL_User_Throughput_QCI8_NUM,
          Dl_User_Throughput_QCI8_DENOM,
          Dl_Rlc_Data_volume_QCI8,
          Ul_Rlc_Data_volume_QCI8,
          (EstabInitAttNbr_QCI5 + EstabAddAttNbr_QCI5 - ErabAddFailNbr_CpCcInteraction_QCI5) -  (EstabInitSuccNbr_QCI5 + EstabAddSuccNbr_QCI5 ) as ERAB_Setup_Failure_Rate_QCI5_NUM,
          (EstabInitAttNbr_QCI5 + EstabAddAttNbr_QCI5 -ErabAddFailNbr_CpCcInteraction_QCI5)  ERAB_Setup_Failure_Rate_QCI5_DENOM,
          NVL(RelAbnormalbyEnbQci_QCI5,0) + NVL(RelAbnormalbyMmeActQci_QCI5,0) as Total_ERAB_Drop_Rate_QCI5_NUM,
          NVL((RelNormalbyEnbQci_QCI5 + RelAbnormalbyEnbQci_QCI5),0) + NVL((RelNormalbyMmeQci_QCI5 + RelAbnormalbyMmeQci_QCI5),0) as Total_ERAB_Drop_Rate_QCI5_DENOM,
          -- add          
          ASEU,
          CellCallContextCntAvg_Num,
          CellCallContextCntAvg_Denom,
          CellCallContextCntMax,
          -- add  
          NBIoT_Air_RLC_Volume_UL,
          NB_IOT_Context_Drop_Rate_Num,
          NB_IOT_Context_Drop_Rate_Denom,
          -- add
          AirMacULByte_P UL_MAC_Layer_Data_Volume_pCell,
          AirMacULByte_S UL_MAC_Layer_Data_Volume_sCell,
         
        -- add DENG-2770
		EnDcConnNoTot	SEA_ENDC_CapUE_AvgRRCconnectedUsers_Num	,
		EnDcConnNoCnt	SEA_ENDC_CapUE_AvgRRCconnectedUsers_Denom,          
		Release5GCallHoldingTimeTot	5G_Fraction_of_RRC_Call_Hold_Time_pct_Denom,
		EnDcConnNoMax	SEA_ENDC_CapUE_MaxRRCconnectedUsers,
		EnDcAdditionUeMax	SEA_ENDC_CapUE_MaxEnDcUEAddition,          
		TotAvailPrbDLNum	SEA_ENDC_DSS_LTE_DL_PRB_Ratio_pct_Denom,
		TotAvailPrbDLNum	SEA_ENDC_DSS_NR_DL_PRB_Ratio_pct_Denom,
		TotAvailPrbULNum	SEA_ENDC_DSS_LTE_UL_PRB_Ratio_pct_Denom,
		TotAvailPrbULNum	SEA_ENDC_DSS_NR_UL_PRB_Ratio_pct_Denom,
		DLTTINum	SEA_ENDC_DSS_LTE_DL_TTI_Ratio_pct_Denom,
		DLTTINum	SEA_ENDC_DSS_NR_DL_TTI_Ratio_pct_Denom,
		ULTTINum	SEA_ENDC_DSS_LTE_UL_TTI_Ratio_pct_Denom,
		ULTTINum	SEA_ENDC_DSS_NR_UL_TTI_Ratio_pct_Denom,
    ReadCellUnavailableTimeLock_s_SUM,
    ReadCellUnavailableTimeES_s_SUM,
    CellAvailPmPeriodTime_s_SUM,
    ConnEstabReject_CpCcFail,
    ConnEstabReject_CpCapaCacFail,
    HPA_RrcEstabMsg_RrcConnectionRequest-HPA_RrcEstabMsg_RrcConnectionSetupComplete HPA_RRC_Setup_Failure_Rate_Num,
    HPA_RrcEstabMsg_RrcConnectionRequest  HPA_RRC_Setup_Failure_Rate_Denom,
    (EstabInitAttNbr_QCI7 + EstabAddAttNbr_QCI7 - ErabAddFailNbr_CpCcInteraction_QCI7) -  (EstabInitSuccNbr_QCI7 + EstabAddSuccNbr_QCI7 ) as HPA_ERAB_Setup_Failure_Rate_Num,
    (EstabInitAttNbr_QCI7 + EstabAddAttNbr_QCI7 -ErabAddFailNbr_CpCcInteraction_QCI7)  HPA_ERAB_Setup_Failure_Rate_Denom,
    NVL(RelAbnormalbyEnbQci_QCI7,0) + NVL(RelAbnormalbyMmeActQci_QCI7,0) as HPA_ERAB_Drop_Rate_Num,
    NVL((RelNormalbyEnbQci_QCI7 + RelAbnormalbyEnbQci_QCI7),0) + NVL((RelNormalbyMmeQci_QCI7 + RelAbnormalbyMmeQci_QCI7),0) as HPA_ERAB_Drop_Rate_Denom,
    DL_User_Throughput_QCI7_NUM HPA_DL_User_Throughput_QCI7_NUM,
    Dl_User_Throughput_QCI7_DENOM HPA_DL_User_Throughput_QCI7_DENOM,
    Dl_Rlc_Data_volume_QCI7 HPA_DL_RLC_Vol,
    Ul_Rlc_Data_volume_QCI7 HPA_UL_RLC_Vol
FROM (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(RrcEstabMsg_RrcConnectionRequest_count_SUM) RrcEstabMsg_RrcConnectionRequest, 
        sum(RrcEstabMsg_RrcConnectionSetupComplete_count_SUM) RrcEstabMsg_RrcConnectionSetupComplete,     
        sum(case when split(LOCATION,'/')[1]='HI_PRIO_ACCESS' then RrcEstabMsg_RrcConnectionRequest_count_SUM else 0 end) HPA_RrcEstabMsg_RrcConnectionRequest,
        sum(case when split(LOCATION,'/')[1]='HI_PRIO_ACCESS' then RrcEstabMsg_RrcConnectionSetupComplete_count_SUM else 0 end)   HPA_RrcEstabMsg_RrcConnectionSetupComplete      
        -- select * 
      from apachedataingestion.lte.rrcconnectionestablishmentmessage_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone  
    ) M0			        
    full join  (    
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(RelAbnormalbyEnbQci_count_SUM) RelAbnormalbyEnbQci,
        sum(RelNormalbyEnbQci_count_SUM) RelNormalbyEnbQci,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI1') then RelAbnormalbyEnbQci_count_SUM else 0 end) RelAbnormalbyEnbQci_QCI1,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI1') then RelNormalbyEnbQci_count_SUM else 0 end) RelNormalbyEnbQci_QCI1,			                
        SUM(case when split(LOCATION,'/')[1] IN ('QCI5','QCI7','QCI8','QCI9') then RelAbnormalbyEnbQci_count_SUM else 0 end) RelAbnormalbyEnbQci_QCI5789,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI5','QCI7','QCI8','QCI9') then RelNormalbyEnbQci_count_SUM else 0 end) RelNormalbyEnbQci_QCI5789,     
        SUM(case when split(LOCATION,'/')[1] IN ('QCI5') then RelAbnormalbyEnbQci_count_SUM else 0 end) RelAbnormalbyEnbQci_QCI5,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI5') then RelNormalbyEnbQci_count_SUM else 0 end) RelNormalbyEnbQci_QCI5,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI7') then RelAbnormalbyEnbQci_count_SUM else 0 end) RelAbnormalbyEnbQci_QCI7,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI7') then RelNormalbyEnbQci_count_SUM else 0 end) RelNormalbyEnbQci_QCI7		
      from apachedataingestion.lte.erabreleasebyenbpercellperqci_lte_hr_silver  
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")       
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T1 on M0.D_DTM=T1.D_DTM and M0.NE_ID=T1.NE_ID  and M0.LOCATION=T1.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(RelAbnormalbyMmeActQci_count_SUM) RelAbnormalbyMmeActQci,
        sum(RelNormalbyMmeQci_count_SUM) RelNormalbyMmeQci,
        sum(RelAbnormalbyMmeQci_count_SUM) RelAbnormalbyMmeQci,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI1') then RelAbnormalbyMmeActQci_count_SUM else 0 end) RelAbnormalbyMmeActQci_QCI1,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI1') then RelNormalbyMmeQci_count_SUM else 0 end) RelNormalbyMmeQci_QCI1,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI1') then RelAbnormalbyMmeQci_count_SUM else 0 end) RelAbnormalbyMmeQci_QCI1 ,			                
        SUM(case when split(LOCATION,'/')[1] IN ('QCI5','QCI7','QCI8','QCI9') then RelAbnormalbyMmeActQci_count_SUM else 0 end) RelAbnormalbyMmeActQci_QCI5789,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI5','QCI7','QCI8','QCI9') then RelNormalbyMmeQci_count_SUM else 0 end) RelNormalbyMmeQci_QCI5789,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI5','QCI7','QCI8','QCI9') then RelAbnormalbyMmeQci_count_SUM else 0 end) RelAbnormalbyMmeQci_QCI5789,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI5') then RelAbnormalbyMmeActQci_count_SUM else 0 end) RelAbnormalbyMmeActQci_QCI5,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI5') then RelNormalbyMmeQci_count_SUM else 0 end) RelNormalbyMmeQci_QCI5,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI5') then RelAbnormalbyMmeQci_count_SUM else 0 end) RelAbnormalbyMmeQci_QCI5,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI7') then RelAbnormalbyMmeActQci_count_SUM else 0 end) RelAbnormalbyMmeActQci_QCI7,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI7') then RelNormalbyMmeQci_count_SUM else 0 end) RelNormalbyMmeQci_QCI7,
        SUM(case when split(LOCATION,'/')[1] IN ('QCI7') then RelAbnormalbyMmeQci_count_SUM else 0 end) RelAbnormalbyMmeQci_QCI7
      from apachedataingestion.lte.erabreleasebymmepercellperqci_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone     
    ) T2 on M0.D_DTM=T2.D_DTM and M0.NE_ID=T2.NE_ID  and M0.LOCATION=T2.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(case when split(LOCATION,'/')[1] in ('QCI7','QCI8','QCI9','QCI132','QCI133') then IpThruThpVoDLByte_Kbytes_SUM end *8/1000 ) as `DL User Throughput_NUM`,
        sum(case when split(LOCATION,'/')[1] in ('QCI7','QCI8','QCI9','QCI132','QCI133') then IpThruThpDLTime_ms_SUM end /1000) as `DL User Throughput_DENOM`,
        sum(case when split(LOCATION,'/')[1] in ('QCI1','QCI5','QCI6','QCI7','QCI8','QCI9','QCI132','QCI133') then AirRlcDLByte_Kbytes_SUM end )  `DL RLC Data Volume NUM`,
        1000 `DL RLC Data Volume DENOM`,
        sum(case when split(LOCATION,'/')[1] in ('QCI1','QCI5','QCI6','QCI7','QCI8','QCI9','QCI132','QCI133') then AirRlcULByte_Kbytes_SUM end )  `UL RLC Data Volume NUM`,
        1000 `UL RLC Data Volume DENOM`,		 
        sum(AirRlcDLByteRe_Kbytes_SUM) `Total DL RLC ReTx NUM`,
        sum(AirRlcDLByte_Kbytes_SUM) `Total DL RLC ReTx DENOM`,
        sum(case when split(LOCATION,'/')[1] in ('QCI8') then IpThruThpVoDLByte_Kbytes_SUM end *8/1000 ) as DL_User_Throughput_QCI8_NUM,
        sum(case when split(LOCATION,'/')[1] in ('QCI8') then IpThruThpDLTime_ms_SUM end /1000) as Dl_User_Throughput_QCI8_DENOM,
        sum(case when split(LOCATION,'/')[1] in ('QCI8') then AirRlcDLByte_Kbytes_SUM end )  Dl_Rlc_Data_volume_QCI8,
        sum(case when split(LOCATION,'/')[1] in ('QCI8') then AirRlcULByte_Kbytes_SUM end )  Ul_Rlc_Data_volume_QCI8,
        sum(case when split(LOCATION,'/')[1] in ('QCI7') then IpThruThpVoDLByte_Kbytes_SUM end *8/1000 ) as DL_User_Throughput_QCI7_NUM,
        sum(case when split(LOCATION,'/')[1] in ('QCI7') then IpThruThpDLTime_ms_SUM end /1000) as Dl_User_Throughput_QCI7_DENOM,
        sum(case when split(LOCATION,'/')[1] in ('QCI7') then AirRlcDLByte_Kbytes_SUM end )  Dl_Rlc_Data_volume_QCI7,
        sum(case when split(LOCATION,'/')[1] in ('QCI7') then AirRlcULByte_Kbytes_SUM end )  Ul_Rlc_Data_volume_QCI7
      from apachedataingestion.lte.airrlcpacket_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")    		 
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone       
    ) T3 on M0.D_DTM=T3.D_DTM and M0.NE_ID=T3.NE_ID  and M0.LOCATION=T3.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(ULIpThruVol_Kbytes_SUM)*8/1000 `UL User Throughput_NUM`,
        sum(ULIpThruTime_TTI_SUM)/1000 `UL User Throughput_DENOM`,				            
        8*sum(AirMacDLByte_Kbytes_SUM)/1000 MAC_DL_Cell_Throughput_NUM,
        8*sum(AirMacULByte_Kbytes_SUM)/1000 MAC_UL_Cell_Throughput_NUM     
      from apachedataingestion.lte.airmacpacket_lte_hr_silver    
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")    
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone      
    ) T4 on M0.D_DTM=T4.D_DTM and M0.NE_ID=T4.NE_ID  and M0.LOCATION=T4.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(PdcpSduLossDLNum_count_SUM)  `VoLTE DL PDCP Packet Loss NUM`,
        sum(PdcpSduTotalDLNum_count_SUM) `VoLTE DL PDCP Packet Loss DENOM`,
        sum(PdcpSduLossULNum_count_SUM)  `VoLTE UL PDCP Packet Loss NUM`,
        sum(PdcpSduTotalULNum_count_SUM) `VoLTE UL PDCP Packet Loss DENOM`
      from apachedataingestion.lte.packetlossrate_lte_hr_silver      
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")    
      and split(LOCATION,'/')[1]='QCI1'
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone     
    ) T5 on M0.D_DTM=T5.D_DTM and M0.NE_ID=T5.NE_ID  and M0.LOCATION=T5.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(ConnNoTot_count_SUM) `Avg. RRC Connected Users NUM`,
        sum(ConnNoCnt_count_SUM) `Avg. RRC Connected Users DENOM`,
        max(ConnNoMax_count_MAX) `Max. RRC Connected Users` ,
        sum(ReleaseCallHoldingTimeTot_s_SUM) ReleaseCallHoldingTimeTot,
        sum(ReleaseCallHoldingTimeCnt_count_SUM) ReleaseCallHoldingTimeCnt
      from apachedataingestion.lte.rrcconnectionnumber_lte_hr_silver      
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     		 
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T6 on M0.D_DTM=T6.D_DTM and M0.NE_ID=T6.NE_ID  and M0.LOCATION=T6.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(EnDc_AddAtt_count_SUM) EnDc_AddAtt,
        sum(EnDc_AddSucc_count_SUM) EnDc_AddSucc,
        sum(EnDc_AddPrepSucc_count_SUM)  EnDc_AddPrepSucc          
      from apachedataingestion.lte.endcadditioninformation_lte_hr_silver      
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     		 
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T7 on M0.D_DTM=T7.D_DTM and M0.NE_ID=T7.NE_ID  and M0.LOCATION=T7.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(EnDc_RelRequest_UeContxtRel_count_SUM)  EnDc_RelRequest_UeContxtRel,
        sum(EnDc_RelRequest_InternalResouFail_count_SUM) EnDc_RelRequest_InternalResouFail,
        sum(EnDc_RelRequest_DL_RLF_randomAccessProblem_count_SUM) EnDc_RelRequest_DL_RLF_randomAccessProblem,
        sum(EnDc_RelRequest_DL_RLF_rlc_MaxNumRetx_count_MAX) EnDc_RelRequest_DL_RLF_rlc_MaxNumRetx,
        sum(EnDc_RelRequest_DL_RLF_scg_reconfigFailure_count_SUM) EnDc_RelRequest_DL_RLF_scg_reconfigFailure,
        sum(EnDc_RelRequest_DL_RLF_srb3_IntegrityFailure_count_SUM) EnDc_RelRequest_DL_RLF_srb3_IntegrityFailure,
        sum(EnDc_RelRequest_DL_RLF_t310_Expiry_count_SUM) EnDc_RelRequest_DL_RLF_t310_Expiry,
        sum(EnDc_RelRequest_UL_RLF_RadioConnection_UeLost_count_SUM) EnDc_RelRequest_UL_RLF_RadioConnection_UeLost,
        sum(EnDc_RelRequire_BearerNotSupp_count_SUM) EnDc_RelRequire_BearerNotSupp,
        sum(EnDc_RelRequire_DcOverallTO_count_SUM) EnDc_RelRequire_DcOverallTO,
        sum(EnDc_RelRequire_UL_RLF_RadioConnection_UeLost_count_SUM) EnDc_RelRequire_UL_RLF_RadioConnection_UeLost,
        sum(EnDc_RelRequest_DL_RLF_synchReconfigFailure_SCG_count_SUM) EnDc_RelRequest_DL_RLF_synchReconfigFailure_SCG          
      from apachedataingestion.lte.endcreleaseinformation_lte_hr_silver      
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     		 
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T8 on M0.D_DTM=T8.D_DTM and M0.NE_ID=T8.NE_ID  and M0.LOCATION=T8.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(EstabAddAttNbr_count_SUM) EstabAddAttNbr,
        sum(EstabAddSuccNbr_count_SUM) EstabAddSuccNbr,
        sum(ErabAddFailNbr_CpCcInteraction_count_SUM) ErabAddFailNbr_CpCcInteraction,
        sum(case when split(LOCATION,'/')[1] in ('QCI5') then EstabAddAttNbr_count_SUM end )  EstabAddAttNbr_QCI5,
        sum(case when split(LOCATION,'/')[1] in ('QCI5') then EstabAddSuccNbr_count_SUM end )  EstabAddSuccNbr_QCI5,
        sum(case when split(LOCATION,'/')[1] in ('QCI5') then ErabAddFailNbr_CpCcInteraction_count_SUM end )  ErabAddFailNbr_CpCcInteraction_QCI5,
        sum(case when split(LOCATION,'/')[1] in ('QCI7') then EstabAddAttNbr_count_SUM end )  EstabAddAttNbr_QCI7,
        sum(case when split(LOCATION,'/')[1] in ('QCI7') then EstabAddSuccNbr_count_SUM end )  EstabAddSuccNbr_QCI7,
        sum(case when split(LOCATION,'/')[1] in ('QCI7') then ErabAddFailNbr_CpCcInteraction_count_SUM end )  ErabAddFailNbr_CpCcInteraction_QCI7
      -- select *
      from apachedataingestion.lte.erabsetupadd_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")    	
      --and split(LOCATION,'/')[1] in ('QCI1','QCI2','QCI3','QCI4','QCI5','QCI6','QCI7','QCI8','QCI9')
      and split(LOCATION,'/')[1] in ('QCI1','QCI5','QCI7','QCI8','QCI9','QCI132','QCI133')
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone  
    ) T9 on M0.D_DTM=T9.D_DTM and M0.NE_ID=T9.NE_ID  and M0.LOCATION=T9.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(EstabInitAttNbr_count_SUM) EstabInitAttNbr,
        sum(EstabInitSuccNbr_count_SUM) EstabInitSuccNbr,
        sum(case when split(LOCATION,'/')[2] in ('QCI5') then EstabInitAttNbr_count_SUM end )  EstabInitAttNbr_QCI5,
        sum(case when split(LOCATION,'/')[2] in ('QCI5') then EstabInitSuccNbr_count_SUM end )  EstabInitSuccNbr_QCI5,
        sum(case when split(LOCATION,'/')[2] in ('QCI7') then EstabInitAttNbr_count_SUM end )  EstabInitAttNbr_QCI7,
        sum(case when split(LOCATION,'/')[2] in ('QCI7') then EstabInitSuccNbr_count_SUM end )  EstabInitSuccNbr_QCI7
        -- select *
      from apachedataingestion.lte.erabsetup_lte_hr_silver			        			
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")         
      --and split(LOCATION,'/')[2] in ('QCI1','QCI2','QCI3','QCI4','QCI5','QCI6','QCI7','QCI8','QCI9')
      and split(LOCATION,'/')[2] in ('QCI1','QCI5','QCI7','QCI8','QCI9','QCI132','QCI133')
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone  
    ) T10 on M0.D_DTM=T10.D_DTM and M0.NE_ID=T10.NE_ID  and M0.LOCATION=T10.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(Tx_S1apInitialContextSetupFailure_count_SUM) Tx_S1apInitialContextSetupFailure,
        sum(S1ContextSetupFailure_count_SUM) S1ContextSetupFailure,
        sum(Rx_S1apInitialContextSetupRequest_count_SUM) Rx_S1apInitialContextSetupRequest
      from apachedataingestion.lte.s1contextsetup_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T11 on M0.D_DTM=T11.D_DTM and M0.NE_ID=T11.NE_ID  and M0.LOCATION=T11.LOCATION
    full join  (
    SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,    
      sum(ConnEstabSucc_count_SUM) ConnEstabSucc,
      sum(ConnEstabReject_CpCcFail_count_SUM) ConnEstabReject_CpCcFail,
      sum(ConnEstabReject_CpCapaCacFail_count_SUM) ConnEstabReject_CpCapaCacFail
    from apachedataingestion.lte.rrcconnectionestablishment_lte_hr_silver
    WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
    GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T12 on M0.D_DTM=T12.D_DTM and M0.NE_ID=T12.NE_ID  and M0.LOCATION=T12.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(RlfDetection_RadioLinkFailure_count_SUM) RlfDetection_RadioLinkFailure,
        sum(RlfDetection_ArqMaxReTransmission_count_SUM) RlfDetection_ArqMaxReTransmission
      from apachedataingestion.lte.rlfdetectionnumberpercell_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")  
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T13 on M0.D_DTM=T13.D_DTM and M0.NE_ID=T13.NE_ID  and M0.LOCATION=T13.LOCATION  
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(Context_RelAbnormalByEnb_count_SUM) context_relabnormalbyenb,
        sum(Context_RelAbnormalByMmeAct_count_SUM) context_relabnormalbymmeact,
        sum(Context_RelNormalByEnb_count_SUM) context_relnormalbyenb,
        sum(Context_RelNormalByMme_count_SUM) context_relnormalbymme,
        sum(Context_RelAbnormalByMme_count_SUM) context_relabnormalbymme   
      from apachedataingestion.lte.contextreleasebymmeandenbpercell_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")    
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T14 on M0.D_DTM=T14.D_DTM and M0.NE_ID=T14.NE_ID  and M0.LOCATION=T14.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(case when split(LOCATION,'/')[1] in ('HO_FAIL','OTHER','RECONF_FAIL')   then ConnReEstabAtt_count_SUM end) ConnReEstabAtt_3cause,
        sum(ConnReEstabAtt_count_SUM) ConnReEstabAtt,
        sum(ConnReEstabSucc_count_SUM) ConnReEstabSucc
      from apachedataingestion.lte.rrcconnectionreestablishment_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T15 on M0.D_DTM=T15.D_DTM and M0.NE_ID=T15.NE_ID  and M0.LOCATION=T15.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(RrcRrcConnectionSetupComplete_count_SUM) RrcRrcConnectionSetupComplete   
      from apachedataingestion.lte.rrcmessagepercell_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T16 on M0.D_DTM=T16.D_DTM and M0.NE_ID=T16.NE_ID  and M0.LOCATION=T16.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(IntraEnbAtt_count_SUM) IntraEnbAtt,
        sum(IntraEnbSucc_count_SUM) IntraEnbSucc,
        sum(IntraEnbPrepFail_CpCapaCacFail_count_SUM) IntraEnbPrepFail_CpCapaCacFail  
      from apachedataingestion.lte.intraenbhandover_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      and split(LOCATION,'/')[2]='RADIO_REASON'  
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T17 on M0.D_DTM=T17.D_DTM and M0.NE_ID=T17.NE_ID  and M0.LOCATION=T17.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(InterX2OutAtt_count_SUM) InterX2OutAtt,     
        sum(InterX2OutSucc_count_SUM) InterX2OutSucc  
      from apachedataingestion.lte.x2outhandover_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")    
      and split(LOCATION,'/')[2]='RADIO_REASON'  
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T18 on M0.D_DTM=T18.D_DTM and M0.NE_ID=T18.NE_ID  and M0.LOCATION=T18.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(InterS1OutAtt_count_SUM) InterS1OutAtt,     
        sum(InterS1OutSucc_count_SUM) InterS1OutSucc  
      from apachedataingestion.lte.s1outhandover_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")      
      and split(LOCATION,'/')[2]='RADIO_REASON'  
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T19 on M0.D_DTM=T19.D_DTM and M0.NE_ID=T19.NE_ID  and M0.LOCATION=T19.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(TtibActAtt_count_SUM) TtibActAtt,
        sum(TtibActSucc_count_SUM)   TtibActSucc,
        sum(TtibNoTot_count_SUM) TtibNoTot,
        sum(TtibNoCnt_count_SUM)   TtibNoCnt   
      from apachedataingestion.lte.ttibundling_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")    
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T20 on M0.D_DTM=T20.D_DTM and M0.NE_ID=T20.NE_ID  and M0.LOCATION=T20.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(DLTransmissionNackedRetrans_count_SUM) DLTransmissionNackedRetrans,      
        sum(DLTransmissionRetrans0_count_SUM) DLTransmissionRetrans0,
        sum(DLTransmissionRetrans1_count_SUM) DLTransmissionRetrans1
      from apachedataingestion.lte.downlinkharqtransmissionbler_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T21 on M0.D_DTM=T21.D_DTM and M0.NE_ID=T21.NE_ID  and M0.LOCATION=T21.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(DLTransmissionNackedRetrans_256QAM_count_SUM) DLTransmissionNackedRetrans_256QAM,      
        sum(DLTransmissionRetrans0_256QAM_count_SUM) DLTransmissionRetrans0_256QAM,
        sum(DLTransmissionRetrans1_256QAM_count_SUM) DLTransmissionRetrans1_256QAM
      from apachedataingestion.lte.downlinkharqtransmissionblerin256qamsupportedue_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T22 on M0.D_DTM=T22.D_DTM and M0.NE_ID=T22.NE_ID  and M0.LOCATION=T22.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(ULTransmissionNackedRetrans_count_SUM) ULTransmissionNackedRetrans,
        sum(ULTransmissionRetrans0_count_SUM) ULTransmissionRetrans0,
        sum(ULTransmissionRetrans1_count_SUM) ULTransmissionRetrans1
      from apachedataingestion.lte.uplinkharqtransmissionbler_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T23 on M0.D_DTM=T23.D_DTM and M0.NE_ID=T23.NE_ID  and M0.LOCATION=T23.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(ULTransmissionNackedRetrans_256QAM_count_SUM) ULTransmissionNackedRetrans_256QAM,
        sum(ULTransmissionRetrans0_256QAM_count_SUM) ULTransmissionRetrans0_256QAM,   
        sum(ULTransmissionRetrans1_256QAM_count_SUM) ULTransmissionRetrans1_256QAM
      from apachedataingestion.lte.uplinkharqtransmissionblerin256qamsupportedue_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T24 on M0.D_DTM=T24.D_DTM and M0.NE_ID=T24.NE_ID  and M0.LOCATION=T24.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(DLVoLTEHARQFail_count_SUM) DLVoLTEHARQFail,
        sum(DLVoLTEHARQSucc_count_SUM)   DLVoLTEHARQSucc,
        sum(ULVoLTEHARQFail_count_SUM) ULVoLTEHARQFail,
        sum(ULVoLTEHARQSucc_count_SUM)   ULVoLTEHARQSucc   
      from apachedataingestion.lte.harqperformanceforvoltebearer_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T25 on M0.D_DTM=T25.D_DTM and M0.NE_ID=T25.NE_ID  and M0.LOCATION=T25.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(RruCceAllocationFailDL_count_SUM) RruCceAllocationFailDL,
        sum(RruCceAllocationDL_count_SUM)   RruCceAllocationDL,
        sum(RruCceAllocationFailUL_count_SUM) RruCceAllocationFailUL,
        sum(RruCceAllocationUL_count_SUM)   RruCceAllocationUL   
      from apachedataingestion.lte.rrumeasurementinfo_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T26 on M0.D_DTM=T26.D_DTM and M0.NE_ID=T26.NE_ID  and M0.LOCATION=T26.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(TotPrbDLTot___SUM) TotPrbDLTot,     
        sum(TotPrbDLCnt_count_SUM) TotPrbDLCnt,
        sum(TotPrbULTot___SUM) TotPrbULTot,     
        sum(TotPrbULCnt_count_SUM) TotPrbULCnt,
        -- add DENG-2770
        sum(TotAvailPrbDLNum_count_SUM) TotAvailPrbDLNum,
        sum(TotAvailPrbULNum_count_SUM) TotAvailPrbULNum  
-- select *
      from apachedataingestion.lte.totalprbusage_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T27 on M0.D_DTM=T27.D_DTM and M0.NE_ID=T27.NE_ID  and M0.LOCATION=T27.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(UsageNbrTot_count_SUM) UsageNbrTot,
        sum(UsageNbrCnt_count_SUM)   UsageNbrCnt   
      from apachedataingestion.lte.activeerabnumber_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")      
      and  split(LOCATION,'/')[1]='QCI1'  
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T28 on M0.D_DTM=T28.D_DTM and M0.NE_ID=T28.NE_ID  and M0.LOCATION=T28.LOCATION
    full join  (
    SELECT D_DTM,NE_ID,LOCATION,SITE_ID,Timezone, 	
      sum(
      1*(DLTransmittedMcs1+DLTransmittedMcs1_256QAM)+
      2*(DLTransmittedMcs2+DLTransmittedMcs2_256QAM)+
      3*(DLTransmittedMcs3+DLTransmittedMcs3_256QAM)+
      4*(DLTransmittedMcs4+DLTransmittedMcs4_256QAM)+
      5*(DLTransmittedMcs5+DLTransmittedMcs5_256QAM)+
      6*(DLTransmittedMcs6+DLTransmittedMcs6_256QAM)+
      7*(DLTransmittedMcs7+DLTransmittedMcs7_256QAM)+
      8*(DLTransmittedMcs8+DLTransmittedMcs8_256QAM)+
      9*(DLTransmittedMcs9+DLTransmittedMcs9_256QAM)+
      10*(DLTransmittedMcs10+DLTransmittedMcs10_256QAM)+
      11*(DLTransmittedMcs11+DLTransmittedMcs11_256QAM)+
      12*(DLTransmittedMcs12+DLTransmittedMcs12_256QAM)+
      13*(DLTransmittedMcs13+DLTransmittedMcs13_256QAM)+
      14*(DLTransmittedMcs14+DLTransmittedMcs14_256QAM)+
      15*(DLTransmittedMcs15+DLTransmittedMcs15_256QAM)+
      16*(DLTransmittedMcs16+DLTransmittedMcs16_256QAM)+
      17*(DLTransmittedMcs17+DLTransmittedMcs17_256QAM)+
      18*(DLTransmittedMcs18+DLTransmittedMcs18_256QAM)+
      19*(DLTransmittedMcs19+DLTransmittedMcs19_256QAM)+
      20*(DLTransmittedMcs20+DLTransmittedMcs20_256QAM)+
      21*(DLTransmittedMcs21+DLTransmittedMcs21_256QAM)+
      22*(DLTransmittedMcs22+DLTransmittedMcs22_256QAM)+
      23*(DLTransmittedMcs23+DLTransmittedMcs23_256QAM)+
      24*(DLTransmittedMcs24+DLTransmittedMcs24_256QAM)+
      25*(DLTransmittedMcs25+DLTransmittedMcs25_256QAM)+
      26*(DLTransmittedMcs26+DLTransmittedMcs26_256QAM)+
      27*(DLTransmittedMcs27+DLTransmittedMcs27_256QAM)+
      28*(DLTransmittedMcs28+DLTransmittedMcs28_256QAM)+
      29*(DLTransmittedMcs29+DLTransmittedMcs29_256QAM)+
      30*(DLTransmittedMcs30+DLTransmittedMcs30_256QAM)+
      31*(DLTransmittedMcs31+DLTransmittedMcs31_256QAM)
      ) DL_MCS_NUM,						
      sum(
      (DLTransmittedMcs0+DLTransmittedMcs0_256QAM)+
      (DLTransmittedMcs1+DLTransmittedMcs1_256QAM)+
      (DLTransmittedMcs2+DLTransmittedMcs2_256QAM)+
      (DLTransmittedMcs3+DLTransmittedMcs3_256QAM)+
      (DLTransmittedMcs4+DLTransmittedMcs4_256QAM)+
      (DLTransmittedMcs5+DLTransmittedMcs5_256QAM)+
      (DLTransmittedMcs6+DLTransmittedMcs6_256QAM)+
      (DLTransmittedMcs7+DLTransmittedMcs7_256QAM)+
      (DLTransmittedMcs8+DLTransmittedMcs8_256QAM)+
      (DLTransmittedMcs9+DLTransmittedMcs9_256QAM)+
      (DLTransmittedMcs10+DLTransmittedMcs10_256QAM)+
      (DLTransmittedMcs11+DLTransmittedMcs11_256QAM)+
      (DLTransmittedMcs12+DLTransmittedMcs12_256QAM)+
      (DLTransmittedMcs13+DLTransmittedMcs13_256QAM)+
      (DLTransmittedMcs14+DLTransmittedMcs14_256QAM)+
      (DLTransmittedMcs15+DLTransmittedMcs15_256QAM)+
      (DLTransmittedMcs16+DLTransmittedMcs16_256QAM)+
      (DLTransmittedMcs17+DLTransmittedMcs17_256QAM)+
      (DLTransmittedMcs18+DLTransmittedMcs18_256QAM)+
      (DLTransmittedMcs19+DLTransmittedMcs19_256QAM)+
      (DLTransmittedMcs20+DLTransmittedMcs20_256QAM)+
      (DLTransmittedMcs21+DLTransmittedMcs21_256QAM)+
      (DLTransmittedMcs22+DLTransmittedMcs22_256QAM)+
      (DLTransmittedMcs23+DLTransmittedMcs23_256QAM)+
      (DLTransmittedMcs24+DLTransmittedMcs24_256QAM)+
      (DLTransmittedMcs25+DLTransmittedMcs25_256QAM)+
      (DLTransmittedMcs26+DLTransmittedMcs26_256QAM)+
      (DLTransmittedMcs27+DLTransmittedMcs27_256QAM)+
      (DLTransmittedMcs28+DLTransmittedMcs28_256QAM)+
      (DLTransmittedMcs29+DLTransmittedMcs29_256QAM)+
      (DLTransmittedMcs30+DLTransmittedMcs30_256QAM)+
      (DLTransmittedMcs31+DLTransmittedMcs31_256QAM)
      ) DL_MCS_DENOM
        from ( 
          SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,			                  
            sum(DLTransmittedMcs0_count_SUM) DLTransmittedMcs0,
            sum(DLTransmittedMcs1_count_SUM) DLTransmittedMcs1,
            sum(DLTransmittedMcs2_count_SUM) DLTransmittedMcs2,
            sum(DLTransmittedMcs3_count_SUM) DLTransmittedMcs3,
            sum(DLTransmittedMcs4_count_SUM) DLTransmittedMcs4,
            sum(DLTransmittedMcs5_count_SUM) DLTransmittedMcs5,
            sum(DLTransmittedMcs6_count_SUM) DLTransmittedMcs6,
            sum(DLTransmittedMcs7_count_SUM) DLTransmittedMcs7,
            sum(DLTransmittedMcs8_count_SUM) DLTransmittedMcs8,
            sum(DLTransmittedMcs9_count_SUM) DLTransmittedMcs9,
            sum(DLTransmittedMcs10_count_SUM) DLTransmittedMcs10,
            sum(DLTransmittedMcs11_count_SUM) DLTransmittedMcs11,
            sum(DLTransmittedMcs12_count_SUM) DLTransmittedMcs12,
            sum(DLTransmittedMcs13_count_SUM) DLTransmittedMcs13,
            sum(DLTransmittedMcs14_count_SUM) DLTransmittedMcs14,
            sum(DLTransmittedMcs15_count_SUM) DLTransmittedMcs15,
            sum(DLTransmittedMcs16_count_SUM) DLTransmittedMcs16,
            sum(DLTransmittedMcs17_count_SUM) DLTransmittedMcs17,
            sum(DLTransmittedMcs18_count_SUM) DLTransmittedMcs18,
            sum(DLTransmittedMcs19_count_SUM) DLTransmittedMcs19,
            sum(DLTransmittedMcs20_count_SUM) DLTransmittedMcs20,
            sum(DLTransmittedMcs21_count_SUM) DLTransmittedMcs21,
            sum(DLTransmittedMcs22_count_SUM) DLTransmittedMcs22,
            sum(DLTransmittedMcs23_count_SUM) DLTransmittedMcs23,
            sum(DLTransmittedMcs24_count_SUM) DLTransmittedMcs24,
            sum(DLTransmittedMcs25_count_SUM) DLTransmittedMcs25,
            sum(DLTransmittedMcs26_count_SUM) DLTransmittedMcs26,
            sum(DLTransmittedMcs27_count_SUM) DLTransmittedMcs27,
            sum(DLTransmittedMcs28_count_SUM) DLTransmittedMcs28,
            sum(DLTransmittedMcs29_count_SUM) DLTransmittedMcs29,
            sum(DLTransmittedMcs30_count_SUM) DLTransmittedMcs30,
            sum(DLTransmittedMcs31_count_SUM) DLTransmittedMcs31,
            0 DLTransmittedMcs0_256QAM,
            0 DLTransmittedMcs1_256QAM,
            0 DLTransmittedMcs2_256QAM,
            0 DLTransmittedMcs3_256QAM,
            0 DLTransmittedMcs4_256QAM,
            0 DLTransmittedMcs5_256QAM,
            0 DLTransmittedMcs6_256QAM,
            0 DLTransmittedMcs7_256QAM,
            0 DLTransmittedMcs8_256QAM,
            0 DLTransmittedMcs9_256QAM,
            0 DLTransmittedMcs10_256QAM,
            0 DLTransmittedMcs11_256QAM,
            0 DLTransmittedMcs12_256QAM,
            0 DLTransmittedMcs13_256QAM,
            0 DLTransmittedMcs14_256QAM,
            0 DLTransmittedMcs15_256QAM,
            0 DLTransmittedMcs16_256QAM,
            0 DLTransmittedMcs17_256QAM,
            0 DLTransmittedMcs18_256QAM,
            0 DLTransmittedMcs19_256QAM,
            0 DLTransmittedMcs20_256QAM,
            0 DLTransmittedMcs21_256QAM,
            0 DLTransmittedMcs22_256QAM,
            0 DLTransmittedMcs23_256QAM,
            0 DLTransmittedMcs24_256QAM,
            0 DLTransmittedMcs25_256QAM,
            0 DLTransmittedMcs26_256QAM,
            0 DLTransmittedMcs27_256QAM,
            0 DLTransmittedMcs28_256QAM,
            0 DLTransmittedMcs29_256QAM,
            0 DLTransmittedMcs30_256QAM,
            0 DLTransmittedMcs31_256QAM
          from apachedataingestion.lte.dlmcs_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
          GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 											                  
          union all
          SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            0 DLTransmittedMcs0,
            0 DLTransmittedMcs1,
            0 DLTransmittedMcs2,
            0 DLTransmittedMcs3,
            0 DLTransmittedMcs4,
            0 DLTransmittedMcs5,
            0 DLTransmittedMcs6,
            0 DLTransmittedMcs7,
            0 DLTransmittedMcs8,
            0 DLTransmittedMcs9,
            0 DLTransmittedMcs10,
            0 DLTransmittedMcs11,
            0 DLTransmittedMcs12,
            0 DLTransmittedMcs13,
            0 DLTransmittedMcs14,
            0 DLTransmittedMcs15,
            0 DLTransmittedMcs16,
            0 DLTransmittedMcs17,
            0 DLTransmittedMcs18,
            0 DLTransmittedMcs19,
            0 DLTransmittedMcs20,
            0 DLTransmittedMcs21,
            0 DLTransmittedMcs22,
            0 DLTransmittedMcs23,
            0 DLTransmittedMcs24,
            0 DLTransmittedMcs25,
            0 DLTransmittedMcs26,
            0 DLTransmittedMcs27,
            0 DLTransmittedMcs28,
            0 DLTransmittedMcs29,
            0 DLTransmittedMcs30,
            0 DLTransmittedMcs31,
            sum(DLTransmittedMcs0_256QAM_count_SUM) DLTransmittedMcs0_256QAM,
            sum(DLTransmittedMcs1_256QAM_count_SUM) DLTransmittedMcs1_256QAM,
            sum(DLTransmittedMcs2_256QAM_count_SUM) DLTransmittedMcs2_256QAM,
            sum(DLTransmittedMcs3_256QAM_count_SUM) DLTransmittedMcs3_256QAM,
            sum(DLTransmittedMcs4_256QAM_count_SUM) DLTransmittedMcs4_256QAM,
            sum(DLTransmittedMcs5_256QAM_count_SUM) DLTransmittedMcs5_256QAM,
            sum(DLTransmittedMcs6_256QAM_count_SUM) DLTransmittedMcs6_256QAM,
            sum(DLTransmittedMcs7_256QAM_count_SUM) DLTransmittedMcs7_256QAM,
            sum(DLTransmittedMcs8_256QAM_count_SUM) DLTransmittedMcs8_256QAM,
            sum(DLTransmittedMcs9_256QAM_count_SUM) DLTransmittedMcs9_256QAM,
            sum(DLTransmittedMcs10_256QAM_count_SUM) DLTransmittedMcs10_256QAM,
            sum(DLTransmittedMcs11_256QAM_count_SUM) DLTransmittedMcs11_256QAM,
            sum(DLTransmittedMcs12_256QAM_count_SUM) DLTransmittedMcs12_256QAM,
            sum(DLTransmittedMcs13_256QAM_count_SUM) DLTransmittedMcs13_256QAM,
            sum(DLTransmittedMcs14_256QAM_count_SUM) DLTransmittedMcs14_256QAM,
            sum(DLTransmittedMcs15_256QAM_count_SUM) DLTransmittedMcs15_256QAM,
            sum(DLTransmittedMcs16_256QAM_count_SUM) DLTransmittedMcs16_256QAM,
            sum(DLTransmittedMcs17_256QAM_count_SUM) DLTransmittedMcs17_256QAM,
            sum(DLTransmittedMcs18_256QAM_count_SUM) DLTransmittedMcs18_256QAM,
            sum(DLTransmittedMcs19_256QAM_count_SUM) DLTransmittedMcs19_256QAM,
            sum(DLTransmittedMcs20_256QAM_count_SUM) DLTransmittedMcs20_256QAM,
            sum(DLTransmittedMcs21_256QAM_count_SUM) DLTransmittedMcs21_256QAM,
            sum(DLTransmittedMcs22_256QAM_count_SUM) DLTransmittedMcs22_256QAM,
            sum(DLTransmittedMcs23_256QAM_count_SUM) DLTransmittedMcs23_256QAM,
            sum(DLTransmittedMcs24_256QAM_count_SUM) DLTransmittedMcs24_256QAM,
            sum(DLTransmittedMcs25_256QAM_count_SUM) DLTransmittedMcs25_256QAM,
            sum(DLTransmittedMcs26_256QAM_count_SUM) DLTransmittedMcs26_256QAM,
            sum(DLTransmittedMcs27_256QAM_count_SUM) DLTransmittedMcs27_256QAM,
            sum(DLTransmittedMcs28_256QAM_count_SUM) DLTransmittedMcs28_256QAM,
            sum(DLTransmittedMcs29_256QAM_count_SUM) DLTransmittedMcs29_256QAM,
            sum(DLTransmittedMcs30_256QAM_count_SUM) DLTransmittedMcs30_256QAM,
            sum(DLTransmittedMcs31_256QAM_count_SUM) DLTransmittedMcs31_256QAM
          from apachedataingestion.lte.dlmcsin256qam_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
          GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 		 
        ) y
      group by D_DTM,NE_ID,LOCATION,SITE_ID,Timezone
    ) T29 on M0.D_DTM=T29.D_DTM and M0.NE_ID=T29.NE_ID  and M0.LOCATION=T29.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(DLScheduledTTINum_count_SUM)/1000   MAC_DL_Cell_Throughput_DENOM ,
        sum(ULScheduledTTINum_count_SUM)/1000 MAC_UL_Cell_Throughput_DENOM,
        -- addd DENG-2770
        sum(DLTTINum_count_SUM) DLTTINum,
        sum(ULTTINum_count_SUM) ULTTINum
        -- select *
      from apachedataingestion.lte.scheduleduenumber_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T30 on M0.D_DTM=T30.D_DTM and M0.NE_ID=T30.NE_ID  and M0.LOCATION=T30.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(InterFreqOutWithHoCauseAtt_count_SUM - InterFreqOutWithHoCauseSucc_count_SUM) Inter_Freq_HOFR_NUM,
        sum(InterFreqOutWithHoCauseAtt_count_SUM)  Inter_Freq_HOFR_DENOM 
      from apachedataingestion.lte.outgoinginterfrequencyhandoverwithhandovercause_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")       
      and  split(LOCATION,'/')[1]='RADIO_REASON'  
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone  
    ) T31 on M0.D_DTM=T31.D_DTM and M0.NE_ID=T31.NE_ID  and M0.LOCATION=T31.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(
          PucchSinrDistBin0_count_SUM*power(10,-9/10) +
          PucchSinrDistBin1_count_SUM*power(10,-7/10) +
          PucchSinrDistBin2_count_SUM*power(10,-5/10) +
          PucchSinrDistBin3_count_SUM*power(10,-3/10) +
          PucchSinrDistBin4_count_SUM*power(10,-1/10) +
          PucchSinrDistBin5_count_SUM*power(10,1/10) +
          PucchSinrDistBin6_count_SUM*power(10,3/10) +
          PucchSinrDistBin7_count_SUM*power(10,5/10) +
          PucchSinrDistBin8_count_SUM*power(10,7/10) +
          PucchSinrDistBin9_count_SUM*power(10,9/10) +
          PucchSinrDistBin10_count_SUM*power(10,11/10) +
          PucchSinrDistBin11_count_SUM*power(10,13/10) +
          PucchSinrDistBin12_count_SUM*power(10,15/10) +
          PucchSinrDistBin13_count_SUM*power(10,17/10) +
          PucchSinrDistBin14_count_SUM*power(10,19/10) +
          PucchSinrDistBin15_count_SUM*power(10,21/10) +
          PucchSinrDistBin16_count_SUM*power(10,23/10) +
          PucchSinrDistBin17_count_SUM*power(10,25/10) +
          PucchSinrDistBin18_count_SUM*power(10,27/10) +
          PucchSinrDistBin19_count_SUM*power(10,29/10) 
        ) PUCCH_SINR_NUM
        ,
        sum(
          PucchSinrDistBin0_count_SUM+
          PucchSinrDistBin1_count_SUM+
          PucchSinrDistBin2_count_SUM+
          PucchSinrDistBin3_count_SUM+
          PucchSinrDistBin4_count_SUM+
          PucchSinrDistBin5_count_SUM+
          PucchSinrDistBin6_count_SUM+
          PucchSinrDistBin7_count_SUM+
          PucchSinrDistBin8_count_SUM+
          PucchSinrDistBin9_count_SUM+
          PucchSinrDistBin10_count_SUM+
          PucchSinrDistBin11_count_SUM+
          PucchSinrDistBin12_count_SUM+
          PucchSinrDistBin13_count_SUM+
          PucchSinrDistBin14_count_SUM+
          PucchSinrDistBin15_count_SUM+
          PucchSinrDistBin16_count_SUM+
          PucchSinrDistBin17_count_SUM+
          PucchSinrDistBin18_count_SUM+
          PucchSinrDistBin19_count_SUM
        ) PUCCH_SINR_DENOM
      from apachedataingestion.lte.pucchsinrdistributionperbin_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T32 on M0.D_DTM=T32.D_DTM and M0.NE_ID=T32.NE_ID  and M0.LOCATION=T32.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(
          SinrDistULWbPreComp_Bin0_count_SUM*power(10,-9/10) +
          SinrDistULWbPreComp_Bin1_count_SUM*power(10,-7/10) +
          SinrDistULWbPreComp_Bin2_count_SUM*power(10,-5/10) +
          SinrDistULWbPreComp_Bin3_count_SUM*power(10,-3/10) +
          SinrDistULWbPreComp_Bin4_count_SUM*power(10,-1/10) +
          SinrDistULWbPreComp_Bin5_count_SUM*power(10,1/10) +
          SinrDistULWbPreComp_Bin6_count_SUM*power(10,3/10) +
          SinrDistULWbPreComp_Bin7_count_SUM*power(10,5/10) +
          SinrDistULWbPreComp_Bin8_count_SUM*power(10,7/10) +
          SinrDistULWbPreComp_Bin9_count_SUM*power(10,9/10) +
          SinrDistULWbPreComp_Bin10_count_SUM*power(10,11/10) +
          SinrDistULWbPreComp_Bin11_count_SUM*power(10,13/10) +
          SinrDistULWbPreComp_Bin12_count_SUM*power(10,15/10) +
          SinrDistULWbPreComp_Bin13_count_SUM*power(10,17/10) +
          SinrDistULWbPreComp_Bin14_count_SUM*power(10,19/10) +
          SinrDistULWbPreComp_Bin15_count_SUM*power(10,21/10) +
          SinrDistULWbPreComp_Bin16_count_SUM*power(10,23/10) +
          SinrDistULWbPreComp_Bin17_count_SUM*power(10,25/10) +
          SinrDistULWbPreComp_Bin18_count_SUM*power(10,27/10) +
          SinrDistULWbPreComp_Bin19_count_SUM*power(10,29/10) 
        ) PUSCH_SINR_NUM 
        ,
        sum(
          SinrDistULWbPreComp_Bin0_count_SUM+
          SinrDistULWbPreComp_Bin1_count_SUM+
          SinrDistULWbPreComp_Bin2_count_SUM+
          SinrDistULWbPreComp_Bin3_count_SUM+
          SinrDistULWbPreComp_Bin4_count_SUM+
          SinrDistULWbPreComp_Bin5_count_SUM+
          SinrDistULWbPreComp_Bin6_count_SUM+
          SinrDistULWbPreComp_Bin7_count_SUM+
          SinrDistULWbPreComp_Bin8_count_SUM+
          SinrDistULWbPreComp_Bin9_count_SUM+
          SinrDistULWbPreComp_Bin10_count_SUM+
          SinrDistULWbPreComp_Bin11_count_SUM+
          SinrDistULWbPreComp_Bin12_count_SUM+
          SinrDistULWbPreComp_Bin13_count_SUM+
          SinrDistULWbPreComp_Bin14_count_SUM+
          SinrDistULWbPreComp_Bin15_count_SUM+
          SinrDistULWbPreComp_Bin16_count_SUM+
          SinrDistULWbPreComp_Bin17_count_SUM+
          SinrDistULWbPreComp_Bin18_count_SUM+
          SinrDistULWbPreComp_Bin19_count_SUM
        ) PUSCH_SINR_DENOM
      from apachedataingestion.lte.sinrdistribution_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T33 on M0.D_DTM=T33.D_DTM and M0.NE_ID=T33.NE_ID  and M0.LOCATION=T33.LOCATION
    full join  (
      SELECT D_DTM,NE_ID,LOCATION,SITE_ID,Timezone,
        sum(
        1*(ULSchedulerMcs1+ULReceivedMcs1_256QAM)+
        2*(ULSchedulerMcs2+ULReceivedMcs2_256QAM)+
        3*(ULSchedulerMcs3+ULReceivedMcs3_256QAM)+
        4*(ULSchedulerMcs4+ULReceivedMcs4_256QAM)+
        5*(ULSchedulerMcs5+ULReceivedMcs5_256QAM)+
        6*(ULSchedulerMcs6+ULReceivedMcs6_256QAM)+
        7*(ULSchedulerMcs7+ULReceivedMcs7_256QAM)+
        8*(ULSchedulerMcs8+ULReceivedMcs8_256QAM)+
        9*(ULSchedulerMcs9+ULReceivedMcs9_256QAM)+
        10*(ULSchedulerMcs10+ULReceivedMcs10_256QAM)+
        11*(ULSchedulerMcs11+ULReceivedMcs11_256QAM)+
        12*(ULSchedulerMcs12+ULReceivedMcs12_256QAM)+
        13*(ULSchedulerMcs13+ULReceivedMcs13_256QAM)+
        14*(ULSchedulerMcs14+ULReceivedMcs14_256QAM)+
        15*(ULSchedulerMcs15+ULReceivedMcs15_256QAM)+
        16*(ULSchedulerMcs16+ULReceivedMcs16_256QAM)+
        17*(ULSchedulerMcs17+ULReceivedMcs17_256QAM)+
        18*(ULSchedulerMcs18+ULReceivedMcs18_256QAM)+
        19*(ULSchedulerMcs19+ULReceivedMcs19_256QAM)+
        20*(ULSchedulerMcs20+ULReceivedMcs20_256QAM)+
        21*(ULSchedulerMcs21+ULReceivedMcs21_256QAM)+
        22*(ULSchedulerMcs22+ULReceivedMcs22_256QAM)+
        23*(ULSchedulerMcs23+ULReceivedMcs23_256QAM)+
        24*(ULSchedulerMcs24+ULReceivedMcs24_256QAM)+
        25*(ULSchedulerMcs25+ULReceivedMcs25_256QAM)+
        26*(ULSchedulerMcs26+ULReceivedMcs26_256QAM)+
        27*(ULSchedulerMcs27+ULReceivedMcs27_256QAM)+
        28*(ULSchedulerMcs28+ULReceivedMcs28_256QAM)+
        29*(ULSchedulerMcs29+ULReceivedMcs29_256QAM)+
        30*(ULSchedulerMcs30+ULReceivedMcs30_256QAM)+
        31*(ULSchedulerMcs31+ULReceivedMcs31_256QAM)
        ) UL_MCS_NUM,						
        sum(
        (ULSchedulerMcs0+ULReceivedMcs0_256QAM)+
        (ULSchedulerMcs1+ULReceivedMcs1_256QAM)+
        (ULSchedulerMcs2+ULReceivedMcs2_256QAM)+
        (ULSchedulerMcs3+ULReceivedMcs3_256QAM)+
        (ULSchedulerMcs4+ULReceivedMcs4_256QAM)+
        (ULSchedulerMcs5+ULReceivedMcs5_256QAM)+
        (ULSchedulerMcs6+ULReceivedMcs6_256QAM)+
        (ULSchedulerMcs7+ULReceivedMcs7_256QAM)+
        (ULSchedulerMcs8+ULReceivedMcs8_256QAM)+
        (ULSchedulerMcs9+ULReceivedMcs9_256QAM)+
        (ULSchedulerMcs10+ULReceivedMcs10_256QAM)+
        (ULSchedulerMcs11+ULReceivedMcs11_256QAM)+
        (ULSchedulerMcs12+ULReceivedMcs12_256QAM)+
        (ULSchedulerMcs13+ULReceivedMcs13_256QAM)+
        (ULSchedulerMcs14+ULReceivedMcs14_256QAM)+
        (ULSchedulerMcs15+ULReceivedMcs15_256QAM)+
        (ULSchedulerMcs16+ULReceivedMcs16_256QAM)+
        (ULSchedulerMcs17+ULReceivedMcs17_256QAM)+
        (ULSchedulerMcs18+ULReceivedMcs18_256QAM)+
        (ULSchedulerMcs19+ULReceivedMcs19_256QAM)+
        (ULSchedulerMcs20+ULReceivedMcs20_256QAM)+
        (ULSchedulerMcs21+ULReceivedMcs21_256QAM)+
        (ULSchedulerMcs22+ULReceivedMcs22_256QAM)+
        (ULSchedulerMcs23+ULReceivedMcs23_256QAM)+
        (ULSchedulerMcs24+ULReceivedMcs24_256QAM)+
        (ULSchedulerMcs25+ULReceivedMcs25_256QAM)+
        (ULSchedulerMcs26+ULReceivedMcs26_256QAM)+
        (ULSchedulerMcs27+ULReceivedMcs27_256QAM)+
        (ULSchedulerMcs28+ULReceivedMcs28_256QAM)+
        (ULSchedulerMcs29+ULReceivedMcs29_256QAM)+
        (ULSchedulerMcs30+ULReceivedMcs30_256QAM)+
        (ULSchedulerMcs31+ULReceivedMcs31_256QAM)
        ) UL_MCS_DENOM
        from ( 
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,		                  
              sum(ULSchedulerMcs0_count_SUM) ULSchedulerMcs0,
              sum(ULSchedulerMcs1_count_SUM) ULSchedulerMcs1,
              sum(ULSchedulerMcs2_count_SUM) ULSchedulerMcs2,
              sum(ULSchedulerMcs3_count_SUM) ULSchedulerMcs3,
              sum(ULSchedulerMcs4_count_SUM) ULSchedulerMcs4,
              sum(ULSchedulerMcs5_count_SUM) ULSchedulerMcs5,
              sum(ULSchedulerMcs6_count_SUM) ULSchedulerMcs6,
              sum(ULSchedulerMcs7_count_SUM) ULSchedulerMcs7,
              sum(ULSchedulerMcs8_count_SUM) ULSchedulerMcs8,
              sum(ULSchedulerMcs9_count_SUM) ULSchedulerMcs9,
              sum(ULSchedulerMcs10_count_SUM) ULSchedulerMcs10,
              sum(ULSchedulerMcs11_count_SUM) ULSchedulerMcs11,
              sum(ULSchedulerMcs12_count_SUM) ULSchedulerMcs12,
              sum(ULSchedulerMcs13_count_SUM) ULSchedulerMcs13,
              sum(ULSchedulerMcs14_count_SUM) ULSchedulerMcs14,
              sum(ULSchedulerMcs15_count_SUM) ULSchedulerMcs15,
              sum(ULSchedulerMcs16_count_SUM) ULSchedulerMcs16,
              sum(ULSchedulerMcs17_count_SUM) ULSchedulerMcs17,
              sum(ULSchedulerMcs18_count_SUM) ULSchedulerMcs18,
              sum(ULSchedulerMcs19_count_SUM) ULSchedulerMcs19,
              sum(ULSchedulerMcs20_count_SUM) ULSchedulerMcs20,
              sum(ULSchedulerMcs21_count_SUM) ULSchedulerMcs21,
              sum(ULSchedulerMcs22_count_SUM) ULSchedulerMcs22,
              sum(ULSchedulerMcs23_count_SUM) ULSchedulerMcs23,
              sum(ULSchedulerMcs24_count_SUM) ULSchedulerMcs24,
              sum(ULSchedulerMcs25_count_SUM) ULSchedulerMcs25,
              sum(ULSchedulerMcs26_count_SUM) ULSchedulerMcs26,
              sum(ULSchedulerMcs27_count_SUM) ULSchedulerMcs27,
              sum(ULSchedulerMcs28_count_SUM) ULSchedulerMcs28,
              sum(ULSchedulerMcs29_count_SUM) ULSchedulerMcs29,
              sum(ULSchedulerMcs30_count_SUM) ULSchedulerMcs30,
              sum(ULSchedulerMcs31_count_SUM) ULSchedulerMcs31,
              0 ULReceivedMcs0_256QAM,
              0 ULReceivedMcs1_256QAM,
              0 ULReceivedMcs2_256QAM,
              0 ULReceivedMcs3_256QAM,
              0 ULReceivedMcs4_256QAM,
              0 ULReceivedMcs5_256QAM,
              0 ULReceivedMcs6_256QAM,
              0 ULReceivedMcs7_256QAM,
              0 ULReceivedMcs8_256QAM,
              0 ULReceivedMcs9_256QAM,
              0 ULReceivedMcs10_256QAM,
              0 ULReceivedMcs11_256QAM,
              0 ULReceivedMcs12_256QAM,
              0 ULReceivedMcs13_256QAM,
              0 ULReceivedMcs14_256QAM,
              0 ULReceivedMcs15_256QAM,
              0 ULReceivedMcs16_256QAM,
              0 ULReceivedMcs17_256QAM,
              0 ULReceivedMcs18_256QAM,
              0 ULReceivedMcs19_256QAM,
              0 ULReceivedMcs20_256QAM,
              0 ULReceivedMcs21_256QAM,
              0 ULReceivedMcs22_256QAM,
              0 ULReceivedMcs23_256QAM,
              0 ULReceivedMcs24_256QAM,
              0 ULReceivedMcs25_256QAM,
              0 ULReceivedMcs26_256QAM,
              0 ULReceivedMcs27_256QAM,
              0 ULReceivedMcs28_256QAM,
              0 ULReceivedMcs29_256QAM,
              0 ULReceivedMcs30_256QAM,
              0 ULReceivedMcs31_256QAM
            from apachedataingestion.lte.mcs_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 										                  
            union all
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
              0 ULSchedulerMcs0,
              0 ULSchedulerMcs1,
              0 ULSchedulerMcs2,
              0 ULSchedulerMcs3,
              0 ULSchedulerMcs4,
              0 ULSchedulerMcs5,
              0 ULSchedulerMcs6,
              0 ULSchedulerMcs7,
              0 ULSchedulerMcs8,
              0 ULSchedulerMcs9,
              0 ULSchedulerMcs10,
              0 ULSchedulerMcs11,
              0 ULSchedulerMcs12,
              0 ULSchedulerMcs13,
              0 ULSchedulerMcs14,
              0 ULSchedulerMcs15,
              0 ULSchedulerMcs16,
              0 ULSchedulerMcs17,
              0 ULSchedulerMcs18,
              0 ULSchedulerMcs19,
              0 ULSchedulerMcs20,
              0 ULSchedulerMcs21,
              0 ULSchedulerMcs22,
              0 ULSchedulerMcs23,
              0 ULSchedulerMcs24,
              0 ULSchedulerMcs25,
              0 ULSchedulerMcs26,
              0 ULSchedulerMcs27,
              0 ULSchedulerMcs28,
              0 ULSchedulerMcs29,
              0 ULSchedulerMcs30,
              0 ULSchedulerMcs31,
              sum(ULReceivedMcs0_256QAM_count_SUM) ULReceivedMcs0_256QAM,
              sum(ULReceivedMcs1_256QAM_count_SUM) ULReceivedMcs1_256QAM,
              sum(ULReceivedMcs2_256QAM_count_SUM) ULReceivedMcs2_256QAM,
              sum(ULReceivedMcs3_256QAM_count_SUM) ULReceivedMcs3_256QAM,
              sum(ULReceivedMcs4_256QAM_count_SUM) ULReceivedMcs4_256QAM,
              sum(ULReceivedMcs5_256QAM_count_SUM) ULReceivedMcs5_256QAM,
              sum(ULReceivedMcs6_256QAM_count_SUM) ULReceivedMcs6_256QAM,
              sum(ULReceivedMcs7_256QAM_count_SUM) ULReceivedMcs7_256QAM,
              sum(ULReceivedMcs8_256QAM_count_SUM) ULReceivedMcs8_256QAM,
              sum(ULReceivedMcs9_256QAM_count_SUM) ULReceivedMcs9_256QAM,
              sum(ULReceivedMcs10_256QAM_count_SUM) ULReceivedMcs10_256QAM,
              sum(ULReceivedMcs11_256QAM_count_SUM) ULReceivedMcs11_256QAM,
              sum(ULReceivedMcs12_256QAM_count_SUM) ULReceivedMcs12_256QAM,
              sum(ULReceivedMcs13_256QAM_count_SUM) ULReceivedMcs13_256QAM,
              sum(ULReceivedMcs14_256QAM_count_SUM) ULReceivedMcs14_256QAM,
              sum(ULReceivedMcs15_256QAM_count_SUM) ULReceivedMcs15_256QAM,
              sum(ULReceivedMcs16_256QAM_count_SUM) ULReceivedMcs16_256QAM,
              sum(ULReceivedMcs17_256QAM_count_SUM) ULReceivedMcs17_256QAM,
              sum(ULReceivedMcs18_256QAM_count_SUM) ULReceivedMcs18_256QAM,
              sum(ULReceivedMcs19_256QAM_count_SUM) ULReceivedMcs19_256QAM,
              sum(ULReceivedMcs20_256QAM_count_SUM) ULReceivedMcs20_256QAM,
              sum(ULReceivedMcs21_256QAM_count_SUM) ULReceivedMcs21_256QAM,
              sum(ULReceivedMcs22_256QAM_count_SUM) ULReceivedMcs22_256QAM,
              sum(ULReceivedMcs23_256QAM_count_SUM) ULReceivedMcs23_256QAM,
              sum(ULReceivedMcs24_256QAM_count_SUM) ULReceivedMcs24_256QAM,
              sum(ULReceivedMcs25_256QAM_count_SUM) ULReceivedMcs25_256QAM,
              sum(ULReceivedMcs26_256QAM_count_SUM) ULReceivedMcs26_256QAM,
              sum(ULReceivedMcs27_256QAM_count_SUM) ULReceivedMcs27_256QAM,
              sum(ULReceivedMcs28_256QAM_count_SUM) ULReceivedMcs28_256QAM,
              sum(ULReceivedMcs29_256QAM_count_SUM) ULReceivedMcs29_256QAM,
              sum(ULReceivedMcs30_256QAM_count_SUM) ULReceivedMcs30_256QAM,
              sum(ULReceivedMcs31_256QAM_count_SUM) ULReceivedMcs31_256QAM
            from apachedataingestion.lte.mcsin256qamsupportedue_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 										                  	 
        ) y
        group by D_DTM,NE_ID,LOCATION,SITE_ID,Timezone	
    ) T34 on M0.D_DTM=T34.D_DTM and M0.NE_ID=T34.NE_ID  and M0.LOCATION=T34.LOCATION
    full join  (
      SELECT  D_DTM,NE_ID,LOCATION,SITE_ID,Timezone,
          sum(
            0*(DLReceivedCQI0+DLReceivedCQI0_256QAM)
            +1*(DLReceivedCQI1+DLReceivedCQI1_256QAM)
            +2*(DLReceivedCQI2)
            +3*(DLReceivedCQI3+DLReceivedCQI2_256QAM)
            +4*DLReceivedCQI4
            +5*(DLReceivedCQI5+DLReceivedCQI3_256QAM)
            +6*DLReceivedCQI6
            +7*(DLReceivedCQI7+DLReceivedCQI4_256QAM)
            +8*(DLReceivedCQI8+DLReceivedCQI5_256QAM)
            +9*(DLReceivedCQI9+DLReceivedCQI6_256QAM)
            +10*(DLReceivedCQI10+DLReceivedCQI7_256QAM)
            +11*(DLReceivedCQI11+DLReceivedCQI8_256QAM)
            +12*(DLReceivedCQI12+DLReceivedCQI9_256QAM)
            +13*(DLReceivedCQI13+DLReceivedCQI10_256QAM)
            +14*(DLReceivedCQI14+DLReceivedCQI11_256QAM)
            +15*(DLReceivedCQI15+DLReceivedCQI12_256QAM+DLReceivedCQI13_256QAM+DLReceivedCQI14_256QAM+DLReceivedCQI15_256QAM)
          ) Avg_CQI_NUM,						
          sum(
            (DLReceivedCQI0+DLReceivedCQI0_256QAM)+
            (DLReceivedCQI1+DLReceivedCQI1_256QAM)+
            (DLReceivedCQI2+DLReceivedCQI2_256QAM)+
            (DLReceivedCQI3+DLReceivedCQI3_256QAM)+
            (DLReceivedCQI4+DLReceivedCQI4_256QAM)+
            (DLReceivedCQI5+DLReceivedCQI5_256QAM)+
            (DLReceivedCQI6+DLReceivedCQI6_256QAM)+
            (DLReceivedCQI7+DLReceivedCQI7_256QAM)+
            (DLReceivedCQI8+DLReceivedCQI8_256QAM)+
            (DLReceivedCQI9+DLReceivedCQI9_256QAM)+
            (DLReceivedCQI10+DLReceivedCQI10_256QAM)+
            (DLReceivedCQI11+DLReceivedCQI11_256QAM)+
            (DLReceivedCQI12+DLReceivedCQI12_256QAM)+
            (DLReceivedCQI13+DLReceivedCQI13_256QAM)+
            (DLReceivedCQI14+DLReceivedCQI14_256QAM)+
            (DLReceivedCQI15+DLReceivedCQI15_256QAM)
      ) Avg_CQI_DENOM
      from ( 
              SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,			                  
                sum(DLReceivedCQI0_count_SUM) DLReceivedCQI0,
                sum(DLReceivedCQI1_count_SUM) DLReceivedCQI1,
                sum(DLReceivedCQI2_count_SUM) DLReceivedCQI2,
                sum(DLReceivedCQI3_count_SUM) DLReceivedCQI3,
                sum(DLReceivedCQI4_count_SUM) DLReceivedCQI4,
                sum(DLReceivedCQI5_count_SUM) DLReceivedCQI5,
                sum(DLReceivedCQI6_count_SUM) DLReceivedCQI6,
                sum(DLReceivedCQI7_count_SUM) DLReceivedCQI7,
                sum(DLReceivedCQI8_count_SUM) DLReceivedCQI8,
                sum(DLReceivedCQI9_count_SUM) DLReceivedCQI9,
                sum(DLReceivedCQI10_count_SUM) DLReceivedCQI10,
                sum(DLReceivedCQI11_count_SUM) DLReceivedCQI11,
                sum(DLReceivedCQI12_count_SUM) DLReceivedCQI12,
                sum(DLReceivedCQI13_count_SUM) DLReceivedCQI13,
                sum(DLReceivedCQI14_count_SUM) DLReceivedCQI14,
                sum(DLReceivedCQI15_count_SUM) DLReceivedCQI15,
                0 DLReceivedCQI0_256QAM,
                0 DLReceivedCQI1_256QAM,
                0 DLReceivedCQI2_256QAM,
                0 DLReceivedCQI3_256QAM,
                0 DLReceivedCQI4_256QAM,
                0 DLReceivedCQI5_256QAM,
                0 DLReceivedCQI6_256QAM,
                0 DLReceivedCQI7_256QAM,
                0 DLReceivedCQI8_256QAM,
                0 DLReceivedCQI9_256QAM,
                0 DLReceivedCQI10_256QAM,
                0 DLReceivedCQI11_256QAM,
                0 DLReceivedCQI12_256QAM,
                0 DLReceivedCQI13_256QAM,
                0 DLReceivedCQI14_256QAM,
                0 DLReceivedCQI15_256QAM
              from apachedataingestion.lte.dlwidebandcqi_lte_hr_silver
              WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
              GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 											                  
              union all
              SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                0 DLReceivedCQI0,
                0 DLReceivedCQI1,
                0 DLReceivedCQI2,
                0 DLReceivedCQI3,
                0 DLReceivedCQI4,
                0 DLReceivedCQI5,
                0 DLReceivedCQI6,
                0 DLReceivedCQI7,
                0 DLReceivedCQI8,
                0 DLReceivedCQI9,
                0 DLReceivedCQI10,
                0 DLReceivedCQI11,
                0 DLReceivedCQI12,
                0 DLReceivedCQI13,
                0 DLReceivedCQI14,
                0 DLReceivedCQI15,
                sum(DLReceivedCQI0_256QAM_count_SUM) DLReceivedCQI0_256QAM,
                sum(DLReceivedCQI1_256QAM_count_SUM) DLReceivedCQI1_256QAM,
                sum(DLReceivedCQI2_256QAM_count_SUM) DLReceivedCQI2_256QAM,
                sum(DLReceivedCQI3_256QAM_count_SUM) DLReceivedCQI3_256QAM,
                sum(DLReceivedCQI4_256QAM_count_SUM) DLReceivedCQI4_256QAM,
                sum(DLReceivedCQI5_256QAM_count_SUM) DLReceivedCQI5_256QAM,
                sum(DLReceivedCQI6_256QAM_count_SUM) DLReceivedCQI6_256QAM,
                sum(DLReceivedCQI7_256QAM_count_SUM) DLReceivedCQI7_256QAM,
                sum(DLReceivedCQI8_256QAM_count_SUM) DLReceivedCQI8_256QAM,
                sum(DLReceivedCQI9_256QAM_count_SUM) DLReceivedCQI9_256QAM,
                sum(DLReceivedCQI10_256QAM_count_SUM) DLReceivedCQI10_256QAM,
                sum(DLReceivedCQI11_256QAM_count_SUM) DLReceivedCQI11_256QAM,
                sum(DLReceivedCQI12_256QAM_count_SUM) DLReceivedCQI12_256QAM,
                sum(DLReceivedCQI13_256QAM_count_SUM) DLReceivedCQI13_256QAM,
                sum(DLReceivedCQI14_256QAM_count_SUM) DLReceivedCQI14_256QAM,
                sum(DLReceivedCQI15_256QAM_count_SUM) DLReceivedCQI15_256QAM
              from apachedataingestion.lte.dlwidebandcqiin256qam_lte_hr_silver
              WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
              GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 		 
      ) y
      group by D_DTM,NE_ID,LOCATION,SITE_ID,Timezone
    ) T35 on M0.D_DTM=T35.D_DTM and M0.NE_ID=T35.NE_ID  and M0.LOCATION=T35.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,				               
        sum(
          SCellActivatedTime_2CC_ms_SUM+
          SCellActivatedTime_3CC_ms_SUM+
          SCellActivatedTime_4CC_ms_SUM+
          SCellActivatedTime_5CC_ms_SUM+
          SCellActivatedTime_6CC_ms_SUM+
          SCellActivatedTime_7CC_ms_SUM
        ) /1000 Avg_CA_Active_Length_per_activation_attempt_NUM,	           
        sum(		         
          SCellActivation_2CC_count_SUM+
          SCellActivation_3CC_count_SUM+
          SCellActivation_4CC_count_SUM+
          SCellActivation_5CC_count_SUM+
          SCellActivation_6CC_count_SUM+
          SCellActivation_7CC_count_SUM
        )  Avg_CA_Active_Length_per_activation_attempt_DENOM
      from apachedataingestion.lte.caactivationinformationpercctype_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T36 on M0.D_DTM=T36.D_DTM and M0.NE_ID=T36.NE_ID  and M0.LOCATION=T36.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        SUM(PdcpVolDLByteCa_Kbytes_SUM*8) DL_User_Perceived_Throughput_CA_NUM,
        SUM(PdcpTimeDLCa_ms_SUM)  DL_User_Perceived_Throughput_CA_DENOM
        -- select *
      from apachedataingestion.lte.carrieraggregationpdcppacket_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      and split(LOCATION,'/')[1] in ('QCI7','QCI8','QCI9','QCI132','QCI133') 
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T37 on M0.D_DTM=T37.D_DTM and M0.NE_ID=T37.NE_ID  and M0.LOCATION=T37.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(AirMacDLByte_Kbytes_SUM) DL_MAC_Layer_Data_Volume_pCell_NUM,
        sum(AirMacULByte_Kbytes_SUM) AirMacULByte_P
      from apachedataingestion.lte.airmacpacketpcell_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone   
    ) T38 on M0.D_DTM=T38.D_DTM and M0.NE_ID=T38.NE_ID  and M0.LOCATION=T38.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(AirMacDLByte_Kbytes_SUM) DL_MAC_Layer_Data_Volume_sCell_NUM,        
        sum(AirMacULByte_Kbytes_SUM) AirMacULByte_S
      from apachedataingestion.lte.airmacpacketscell_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")       
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T39 on M0.D_DTM=T39.D_DTM and M0.NE_ID=T39.NE_ID  and M0.LOCATION=T39.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(TimeAdvanceSection0_count_SUM+TimeAdvanceSection1_count_SUM*3+TimeAdvanceSection2_count_SUM*5+TimeAdvanceSection3_count_SUM*7+TimeAdvanceSection4_count_SUM*9+TimeAdvanceSection5_count_SUM*11+TimeAdvanceSection6_count_SUM*13+TimeAdvanceSection7_count_SUM*15+TimeAdvanceSection8_count_SUM*17+TimeAdvanceSection9_count_SUM*19+TimeAdvanceSection10_count_SUM*22.5+TimeAdvanceSection11_count_SUM*27.5+TimeAdvanceSection12_count_SUM*32.5+TimeAdvanceSection13_count_SUM*37.5+TimeAdvanceSection14_count_SUM*45+TimeAdvanceSection15_count_SUM*55+TimeAdvanceSection16_count_SUM*65+TimeAdvanceSection17_count_SUM*75+TimeAdvanceSection18_count_SUM*85+TimeAdvanceSection19_count_SUM*95+TimeAdvanceSection20_count_SUM*110+TimeAdvanceSection21_count_SUM*130+TimeAdvanceSection22_count_SUM*150+TimeAdvanceSection23_count_SUM*170+TimeAdvanceSection24_count_SUM*190+TimeAdvanceSection25_count_SUM*225+TimeAdvanceSection26_count_SUM*275+TimeAdvanceSection27_count_SUM*325+TimeAdvanceSection28_count_SUM*375+TimeAdvanceSection29_count_SUM*500+TimeAdvanceSection30_count_SUM*800) Average_Timing_Advance_NUM,
        sum(TimeAdvanceSection0_count_SUM+TimeAdvanceSection1_count_SUM+TimeAdvanceSection2_count_SUM+TimeAdvanceSection3_count_SUM+TimeAdvanceSection4_count_SUM+TimeAdvanceSection5_count_SUM+TimeAdvanceSection6_count_SUM+TimeAdvanceSection7_count_SUM+TimeAdvanceSection8_count_SUM+TimeAdvanceSection9_count_SUM+TimeAdvanceSection10_count_SUM+TimeAdvanceSection11_count_SUM+TimeAdvanceSection12_count_SUM+TimeAdvanceSection13_count_SUM+TimeAdvanceSection14_count_SUM+TimeAdvanceSection15_count_SUM+TimeAdvanceSection16_count_SUM+TimeAdvanceSection17_count_SUM+TimeAdvanceSection18_count_SUM+TimeAdvanceSection19_count_SUM+TimeAdvanceSection20_count_SUM+TimeAdvanceSection21_count_SUM+TimeAdvanceSection22_count_SUM+TimeAdvanceSection23_count_SUM+TimeAdvanceSection24_count_SUM+TimeAdvanceSection25_count_SUM+TimeAdvanceSection26_count_SUM+TimeAdvanceSection27_count_SUM+TimeAdvanceSection28_count_SUM+TimeAdvanceSection29_count_SUM+TimeAdvanceSection30_count_SUM) Average_Timing_Advance_DENOM
      from apachedataingestion.lte.timingalignment_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T40 on M0.D_DTM=T40.D_DTM and M0.NE_ID=T40.NE_ID  and M0.LOCATION=T40.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(SCellAddSucc_HoIn_count_SUM+SCellAddSucc_AddSbcHoIn_count_SUM
        +SCellAddSucc_AddSbcRrcResetup_count_SUM
        +SCellAddSucc_AddSbcRrcSig_count_SUM
        +SCellAddSucc_EventA4_count_SUM
        +SCellAddSucc_EventA6_count_SUM
        +SCellAddSucc_RrcResetup_count_SUM
        +SCellAddSucc_RrcSig_count_SUM
        +SCellAddSucc_AddSbcReducedRrcSig_count_SUM
        +SCellAddSucc_AddSbcReducedHoIn_count_SUM
        +SCellAddSucc_AddSbcReducedRrcResetup_count_SUM
        +SCellAddSucc_EnhSbcSelec_count_SUM) SEA_CAConfigurationRate_NUM,
        sum(SCellAddAtt_count_SUM) SCellAddAtt
      from apachedataingestion.lte.carrieraggregationmessagecountforadditionrelease_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T41 on M0.D_DTM=T41.D_DTM and M0.NE_ID=T41.NE_ID  and M0.LOCATION=T41.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,
        sum(NBIoT_ConnEstabAtt_count_SUM - NBIoT_ConnEstabSucc_count_SUM)	NB_IOT_RRC_Conn_Setup_Failure_Rate_NUM,
        sum(NBIoT_ConnEstabAtt_count_SUM)  NB_IOT_RRC_Conn_Setup_Failure_Rate_DENOM
        --select NBIoT_ConnEstabAtt_count_SUM,NBIoT_ConnEstabSucc_count_SUM
      --from apachedataingestion.lte.nbiotrrcconnectionestablishmentpercelevel_lte_hr_silver
      from apachedataingestion.lte.NBIoTRRCConnectionEstablishment_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T42 on M0.D_DTM=T42.D_DTM and M0.NE_ID=T42.NE_ID  and M0.LOCATION=T42.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,
        sum(NBIoT_ConnNoTot_count_SUM)	 NB_IOT_Avg_RRC_Connections_NUM,
        sum(NBIoT_ConnNoCnt_count_SUM) NB_IOT_Avg_RRC_Connections_DENOM,
        max(NBIoT_ConnNoCeLv0Max_count_MAX) NBIoT_MaxRRCconnUsers_CeLv0,
        max(NBIoT_ConnNoCeLv1Max_count_MAX) NBIoT_MaxRRCconnUsers_CeLv1,
        max(NBIoT_ConnNoCeLv2Max_count_MAX) NBIoT_MaxRRCconnUsers_CeLv2,
        max(NBIoT_ConnNoMax_count_MAX) NBIoT_ConnNoMax
      from apachedataingestion.lte.nbiotrrcconnectionnumber_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")       
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T43 on M0.D_DTM=T43.D_DTM and M0.NE_ID=T43.NE_ID  and M0.LOCATION=T43.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,
        8*sum(NBIoT_AirMacDLByte_Kbytes_SUM)	 NB_IOT_DL_MAC_PDU_Tput_Kbps_NUM,
        10.24*sum(NBIoT_AirMacDLByteCnt_count_SUM) NB_IOT_DL_MAC_PDU_Tput_Kbps_DENOM,
        8*sum(NBIoT_AirMacULByte_Kbytes_SUM)	NB_IOT_UL_MAC_PDU_Tput_Kbps_NUM,
        10.24*sum(NBIoT_AirMacULByteCnt_count_SUM) NB_IOT_UL_MAC_PDU_Tput_Kbps_DENOM
      from apachedataingestion.lte.nbiotairmaculanddlpackets_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T44 on M0.D_DTM=T44.D_DTM and M0.NE_ID=T44.NE_ID  and M0.LOCATION=T44.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,
        sum(NBIoT_AttPaging_count_SUM) NB_IOT_RRC_Paging_Request_Atts
      from apachedataingestion.lte.nbiotpaging_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T45 on M0.D_DTM=T45.D_DTM and M0.NE_ID=T45.NE_ID  and M0.LOCATION=T45.LOCATION
    full join  (
      SELECT  D_DTM,NE_ID,LOCATION,SITE_ID,Timezone,
      sum(NBIoT_Total_ERAB_Drop_Rate_pct) NBIoT_Total_ERAB_Drop_Rate_pct_NUM,
      sum(NBIoT_EstabInitSuccNbr_count_SUM+NBIoT_EstabAddSuccNbr_count_SUM) NBIoT_Total_ERAB_Drop_Rate_pct_DENOM,  
      sum(NBIoT_Total_ERAB_Drop_Rate_pct) NB_IOT_Context_Drop_Rate_Num,    
      sum(NB_IOT_Context_Drop_Rate_Denom) NB_IOT_Context_Drop_Rate_Denom
      from ( 
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,			                  
              sum(NBIoT_CallDrop_NccbArqMaxReTransmission_count_SUM+	     
                  NBIoT_CallDrop_NccbDspAuditMacCallRelease_count_SUM+
                  NBIoT_CallDrop_NccbDspAuditRlcCallRelease_count_SUM+
                  NBIoT_CallDrop_NccbDspAuditRlcMacCallRelease_count_SUM+
                  NBIoT_CallDrop_NccbRadioLinkFailure_count_SUM+         
                  NBIoT_CallDrop_NccbRcvCellReleaseIndFromNcmb_count_SUM+			          
                  NBIoT_CallDrop_NccbRcvResetRequestFromNcmb_count_SUM+
                  NBIoT_CallDrop_NccbS1SctpOutOfService_count_SUM+
                  NBIoT_CallDrop_NccbTmoutRrcConnectionReconfig_count_SUM+
                  NBIoT_CallDrop_NccbTmoutRrcConnectionReestablish_count_SUM) NBIoT_Total_ERAB_Drop_Rate_pct,
                  0 NBIoT_EstabInitSuccNbr_count_SUM,
                  0 NBIoT_EstabAddSuccNbr_count_SUM,
                  0 NB_IOT_Context_Drop_Rate_Denom
            from apachedataingestion.lte.nbiotcalldrop_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")             
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone  										                  
            union all
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,
                0 NBIoT_Total_ERAB_Drop_Rate_pct,
                sum(NBIoT_EstabInitSuccNbr_count_SUM) NBIoT_EstabInitSuccNbr_count_SUM,
                0 NBIoT_EstabAddSuccNbr_count_SUM,
                0 NB_IOT_Context_Drop_Rate_Denom
            from apachedataingestion.lte.nbioterabsetup_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")         
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone  
            union all
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,
                0 NBIoT_Total_ERAB_Drop_Rate_pct,
                0 NBIoT_EstabInitSuccNbr_count_SUM,
                sum(NBIoT_EstabAddSuccNbr_count_SUM) NBIoT_EstabAddSuccNbr_count_SUM,
                0 NB_IOT_Context_Drop_Rate_Denom
            from apachedataingestion.lte.nbioterabsetupadd_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")             
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
             union all
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,
                0 NBIoT_Total_ERAB_Drop_Rate_pct,
                0 NBIoT_EstabInitSuccNbr_count_SUM,
                0 NBIoT_EstabAddSuccNbr_count_SUM,
                sum(NBIoT_S1ConnEstabSucc)  NB_IOT_Context_Drop_Rate_Denom
                --select *
            from apachedataingestion.lte.nbiotueassociatedlogicals1connectionestablishment_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")             
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) y  
      group by D_DTM,NE_ID,LOCATION,SITE_ID,Timezone 
    ) T46 on M0.D_DTM=T46.D_DTM and M0.NE_ID=T46.NE_ID  and M0.LOCATION=T46.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,
        sum(NBIoT_DLTransmissionRetrans1_count_SUM) NBIoT_DL_Initial_BLER_pct_NUM,
        sum(NBIoT_DLTransmissionRetrans0_count_SUM) NBIoT_DL_Initial_BLER_pct_DENOM,            
        sum(NBIoT_DLTransmissionNackedRetrans_count_SUM) NBIoT_DL_Residual_BLER_pct_NUM,
        sum(NBIoT_DLTransmissionRetrans0_count_SUM) NBIoT_DL_Residual_BLER_pct_DENOM
      from apachedataingestion.lte.nbiotnpdschharqtransmissiontimes_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T47 on M0.D_DTM=T47.D_DTM and M0.NE_ID=T47.NE_ID  and M0.LOCATION=T47.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,
        sum(NBIoT_ULTransmissionRetrans1_count_SUM) NBIoT_UL_Initial_BLER_pct_NUM,
        sum(NBIoT_ULTransmissionRetrans0_count_SUM) NBIoT_UL_Initial_BLER_pct_DENOM,
        sum(NBIoT_ULTransmissionNackedRetrans_count_SUM) NBIoT_UL_Residual_BLER_pct_NUM,
        sum(NBIoT_ULTransmissionRetrans0_count_SUM) NBIoT_UL_Residual_BLER_pct_DENOM
      from apachedataingestion.lte.nbiotnpuschharqtransmissiontimes_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T48 on M0.D_DTM=T48.D_DTM and M0.NE_ID=T48.NE_ID  and M0.LOCATION=T48.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,regexp_replace(split(LOCATION,'/')[0],'NBIoT_','') LOCATION,SITE_ID,Timezone,       
        sum(NBIoT_AirRlcDLByte_Kbytes_SUM) NBIoT_Air_RLC_Volume,
        sum(NBIoT_AirRlcULByte_Kbytes_SUM) NBIoT_Air_RLC_Volume_UL
      from apachedataingestion.lte.nbiotairrlculanddlpackets_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T49 on M0.D_DTM=T49.D_DTM and M0.NE_ID=T49.NE_ID  and M0.LOCATION=T49.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(eMTC_PdcpVolDlByte_Kbytes_SUM) eMTC_Downlink_PDCP_Volume_NUM,        
        sum(eMTC_PdcpVolDlByte_Kbytes_SUM*8)	eMTC_Downlink_PDCP_Throughput_NUM,
        sum(eMTC_PdcpTimeDl_ms_SUM/1000) eMTC_Downlink_PDCP_Throughput_DENOM
      from apachedataingestion.lte.emtcpdcppacket_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T50 on M0.D_DTM=T50.D_DTM and M0.NE_ID=T50.NE_ID  and M0.LOCATION=T50.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(eMTC_ConnNoTot_count_SUM)	 eMTC_Avg_RRC_Connected_Users_NUM,
        sum(eMTC_ConnNoCnt_count_SUM)  eMTC_Avg_RRC_Connected_Users_DENOM,
        max(eMTC_ConnNoMax_count_MAX)  eMTC_Max_RRC_Connected_Users
      from apachedataingestion.lte.emtcrrcconnectionnumber_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T51 on M0.D_DTM=T51.D_DTM and M0.NE_ID=T51.NE_ID  and M0.LOCATION=T51.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(eMTC_AttPaging_count_SUM) eMTC_Paging_Attempt
      from apachedataingestion.lte.emtcpagingattempts_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T52 on M0.D_DTM=T52.D_DTM and M0.NE_ID=T52.NE_ID  and M0.LOCATION=T52.LOCATION
    full join  (
      SELECT D_DTM,NE_ID,LOCATION,SITE_ID,Timezone,
          sum(eMTC_EstabInitAttNbr + eMTC_EstabAddAttNbr - eMTC_ErabAddFailNbr_CpCcInteraction -  eMTC_EstabInitSuccNbr -  eMTC_EstabAddSuccNbr)	eMTC_Total_ERAB_Setup_Failure_Rate_NUM,
          sum(eMTC_EstabInitAttNbr + eMTC_EstabAddAttNbr - eMTC_ErabAddFailNbr_CpCcInteraction) eMTC_Total_ERAB_Setup_Failure_Rate_DENOM,
          sum(eMTC_EstabInitSuccNbr) eMTC_EstabInitSuccNbr,
          sum(eMTC_EstabAddSuccNbr)  eMTC_EstabAddSuccNbr
      from ( 
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
              sum(eMTC_EstabInitAttNbr_count_SUM)   eMTC_EstabInitAttNbr,
              sum(eMTC_EstabInitSuccNbr_count_SUM)  eMTC_EstabInitSuccNbr,
              0 eMTC_EstabAddAttNbr,
              0 eMTC_ErabAddFailNbr_CpCcInteraction,
              0 eMTC_EstabAddSuccNbr
            from apachedataingestion.lte.emtcerabsetup_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 											                  
            union all
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
              0 eMTC_EstabInitAttNbr,
              0 eMTC_EstabInitSuccNbr,
              sum(eMTC_EstabAddAttNbr_count_SUM) eMTC_EstabAddAttNbr,
              sum(eMTC_ErabAddFailNbr_CpCcInteraction_count_SUM) eMTC_ErabAddFailNbr_CpCcInteraction,
              sum(eMTC_EstabAddSuccNbr_count_SUM)  eMTC_EstabAddSuccNbr
            from apachedataingestion.lte.emtcerabsetupadd_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) y
      group by D_DTM,NE_ID,LOCATION,SITE_ID,Timezone
    ) T53 on M0.D_DTM=T53.D_DTM and M0.NE_ID=T53.NE_ID  and M0.LOCATION=T53.LOCATION
    full join  (
      SELECT D_DTM,NE_ID,LOCATION,SITE_ID,Timezone,
        sum(eMTC_RlfDetection_RadioLinkFailure+eMTC_RlfDetection_ArqMaxReTransmission)	eMTC_RRC_Connection_Drop_Rate_NUM,
        sum(eMTC_CalDrpDefQci_EccbArqMaxReTransmission+eMTC_S1ConnEstabAtt)	eMTC_RRC_Connection_Drop_Rate_DENOM,		
        sum(eMTC_S1ConnEstabAtt)  eMTC_S1ConnEstabAtt,	
        sum(eMTC_S1ConnEstabSucc)  eMTC_S1ConnEstabSucc
      from ( 
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,			                  
              sum(eMTC_S1ConnEstabAtt_count_SUM)  eMTC_S1ConnEstabAtt,	
              sum(eMTC_S1ConnEstabSucc_count_SUM)  eMTC_S1ConnEstabSucc,
              0 eMTC_CalDrpDefQci_EccbArqMaxReTransmission,
              0 eMTC_RlfDetection_RadioLinkFailure,
              0 eMTC_RlfDetection_ArqMaxReTransmission
            from apachedataingestion.lte.emtcueassociatedlogicals1connectionestablishment_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
            union all
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
              0 eMTC_S1ConnEstabAtt,
              0 eMTC_S1ConnEstabSucc,
              sum(eMTC_CallDropQci_EccbArqMaxReTransmission_count_MAX) eMTC_CalDrpDefQci_EccbArqMaxReTransmission,
              0 eMTC_RlfDetection_RadioLinkFailure,
              0 eMTC_RlfDetection_ArqMaxReTransmission
            from apachedataingestion.lte.emtccalldropperqci_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
            union all
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
              0 eMTC_S1ConnEstabAtt,
              0 eMTC_S1ConnEstabSucc,
              0 eMTC_CalDrpDefQci_EccbArqMaxReTransmission,
              sum(eMTC_RlfDetection_RadioLinkFailure_count_SUM) eMTC_RlfDetection_RadioLinkFailure,
              sum(eMTC_RlfDetection_ArqMaxReTransmission_count_MAX) eMTC_RlfDetection_ArqMaxReTransmission
            from apachedataingestion.lte.emtcrlfdetectionnumberpercell_lte_hr_silver
            WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) y
      group by D_DTM,NE_ID,LOCATION,SITE_ID,Timezone
    ) T54 on M0.D_DTM=T54.D_DTM and M0.NE_ID=T54.NE_ID  and M0.LOCATION=T54.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(eMTC_CallDrop_EccbDspAuditRlcMacCallRelease_count_SUM
        +eMTC_CallDrop_EccbRcvResetRequestFromEcmb_count_SUM
        +eMTC_CallDrop_EccbRcvCellReleaseIndFromEcmb_count_SUM
        +eMTC_CallDrop_EccbRadioLinkFailure_count_SUM
        +eMTC_CallDrop_EccbDspAuditMacCallRelease_count_SUM
        +eMTC_CallDrop_EccbArqMaxReTransmission_count_SUM
        +eMTC_CallDrop_EccbDspAuditRlcCallRelease_count_SUM
        +eMTC_CallDrop_EccbTmoutRrcConnectionReconfig_count_SUM
        +eMTC_CallDrop_EccbTmoutRrcConnectionReestablish_count_SUM
        +eMTC_CallDrop_EccbS1SctpOutOfService_count_SUM) eMTC_Total_ERAB_Drop_Rate_NUM
      from apachedataingestion.lte.emtccalldrop_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T55 on M0.D_DTM=T55.D_DTM and M0.NE_ID=T55.NE_ID  and M0.LOCATION=T55.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(eMTC_IntraFreqOutAtt_count_SUM-eMTC_IntraFreqOutSucc_count_SUM) eMTC_Intra_Freq_HO_Fail_Rate_NUM,
        sum(eMTC_IntraFreqOutAtt_count_SUM) eMTC_Intra_Freq_HO_Fail_Rate_DENOM
      from apachedataingestion.lte.emtcintrafrequencyhandoverout_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T56 on M0.D_DTM=T56.D_DTM and M0.NE_ID=T56.NE_ID  and M0.LOCATION=T56.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(CEModeA_DLTransmissionRetrans1_count_SUM+CEModeB_DLTransmissionRetrans1_count_SUM)   eMTC_DL_Initial_BLER_pct_NUM,
        sum(CEModeA_DLTransmissionRetrans0_count_SUM+CEModeB_DLTransmissionRetrans0_count_SUM)   eMTC_DL_Initial_BLER_pct_DENOM,
        sum(CEModeA_DLTransmissionNackedRetrans_count_SUM+CEModeB_DLTransmissionNackedRetrans_count_SUM) eMTC_DL_Residual_BLER_pct_NUM,
        sum(CEModeA_DLTransmissionRetrans0_count_SUM+CEModeB_DLTransmissionRetrans0_count_SUM)   eMTC_DL_Residual_BLER_pct_DENOM
      from apachedataingestion.lte.emtcdownlinkharqtransmissionbler_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone
    ) T57 on M0.D_DTM=T57.D_DTM and M0.NE_ID=T57.NE_ID  and M0.LOCATION=T57.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(CEModeA_ULTransmissionRetrans1_count_SUM+CEModeB_ULTransmissionRetrans1_count_SUM)   eMTC_UL_Initial_BLER_pct_NUM,
        sum(CEModeA_ULTransmissionRetrans0_count_SUM+CEModeB_ULTransmissionRetrans0_count_SUM)   eMTC_UL_Initial_BLER_pct_DENOM,
        sum(CEModeA_ULTransmissionNackedRetrans_count_SUM+CEModeB_ULTransmissionNackedRetrans_count_SUM) eMTC_UL_Residual_BLER_pct_NUM,
        sum(CEModeA_ULTransmissionRetrans0_count_SUM+CEModeB_ULTransmissionRetrans0_count_SUM)   eMTC_UL_Residual_BLER_pct_DENOM
      from apachedataingestion.lte.emtcuplinkharqtransmissionbler_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T58 on M0.D_DTM=T58.D_DTM and M0.NE_ID=T58.NE_ID  and M0.LOCATION=T58.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
        sum(eMTC_ConnEstabAtt_count_SUM-eMTC_ConnEstabSucc_count_SUM) eMTC_RRC_Setup_Failure_pct_NUM,
        sum(eMTC_ConnEstabAtt_count_SUM)  eMTC_RRC_Setup_Failure_pct_DENOM
      from apachedataingestion.lte.emtcrrcconnectionestablishment_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T59 on M0.D_DTM=T59.D_DTM and M0.NE_ID=T59.NE_ID  and M0.LOCATION=T59.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,       
        sum(EnDcConnNoTot_count_SUM)  ENDC_Cap_UE_Avg_RRC_Connected_Users_NUM,
        sum(EnDcConnNoCnt_count_SUM) ENDC_Cap_UE_Avg_RRC_Connected_Users_DENOM,
        sum(EnDcAdditionUeTot_count_SUM) EnDcAdditionUeTot,
        sum(EnDcAdditionUeCnt_count_SUM) EnDcAdditionUeCnt,
        sum(Release5GCallHoldingTimeTot_s_SUM) Release5GCallHoldingTimeTot,
        sum(Release5GCallHoldingTimeCnt_count_SUM) Release5GCallHoldingTimeCnt,
        -- add  DENG-2770  
        sum(EnDcConnNoTot_count_SUM) EnDcConnNoTot,
        sum(EnDcConnNoCnt_count_SUM) EnDcConnNoCnt,        
        max(EnDcConnNoMax_count_MAX) EnDcConnNoMax,
        max(EnDcAdditionUeMax_count_MAX) EnDcAdditionUeMax
        -- select *
      from apachedataingestion.lte.5gcapablerrcconnectionnumber_lte_hr_silver 
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone  
    ) T60 on M0.D_DTM=T60.D_DTM and M0.NE_ID=T60.NE_ID  and M0.LOCATION=T60.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            sum( EnDc_ConnRelease_CpBhCacFail_count_SUM
                +EnDc_ConnRelease_CpCapaCacFail_count_SUM
                +EnDc_ConnRelease_CpCcFail_count_SUM
                +EnDc_ConnRelease_CpCcTo_count_SUM
                +EnDc_ConnRelease_CpPreemption_count_SUM
                +EnDc_ConnRelease_CpQosCacFail_count_SUM
                +EnDc_ConnRelease_MmeOverload_count_SUM
                +EnDc_ConnRelease_RrcHcTo_count_SUM
                +EnDc_ConnRelease_RrcSigFail_count_SUM
                +EnDc_ConnRelease_RrcSigTo_count_SUM
                +EnDc_ConnRelease_S1apCuFail_count_SUM
                +EnDc_ConnRelease_S1apLinkFail_count_SUM
                +EnDc_ConnRelease_S1apRoTo_count_SUM
                +EnDc_ConnRelease_S1apSigFail_count_SUM
                +EnDc_ConnRelease_S1apSigTo_count_SUM
                +EnDc_ConnRelease_UpGtpFail_count_SUM
                +EnDc_ConnRelease_UpMacFail_count_SUM
                +EnDc_ConnRelease_UpPdcpFail_count_SUM
                +EnDc_ConnRelease_UpRlcFail_count_SUM
                +EnDc_ConnRelease_X2apCuFail_count_SUM
                +EnDc_ConnRelease_X2apLinkFail_count_SUM
                +EnDc_ConnRelease_X2apRoTo_count_SUM
                +EnDc_ConnRelease_X2apSigFail_count_SUM) ENDC_Cap_UE_RRC_Drop_pct_NUM,
            sum( EnDc_ConnRelease_CpBhCacFail_count_SUM
                +EnDc_ConnRelease_CpCapaCacFail_count_SUM
                +EnDc_ConnRelease_CpCcFail_count_SUM
                +EnDc_ConnRelease_CpCcTo_count_SUM
                +EnDc_ConnRelease_CpPreemption_count_SUM
                +EnDc_ConnRelease_CpQosCacFail_count_SUM
                +EnDc_ConnRelease_MmeOverload_count_SUM
                +EnDc_ConnRelease_RrcHcTo_count_SUM
                +EnDc_ConnRelease_RrcSigFail_count_SUM
                +EnDc_ConnRelease_RrcSigTo_count_SUM
                +EnDc_ConnRelease_S1apCuFail_count_SUM
                +EnDc_ConnRelease_S1apLinkFail_count_SUM
                +EnDc_ConnRelease_S1apRoTo_count_SUM
                +EnDc_ConnRelease_S1apSigFail_count_SUM
                +EnDc_ConnRelease_S1apSigTo_count_SUM
                +EnDc_ConnRelease_UpGtpFail_count_SUM
                +EnDc_ConnRelease_UpMacFail_count_SUM
                +EnDc_ConnRelease_UpPdcpFail_count_SUM
                +EnDc_ConnRelease_UpRlcFail_count_SUM
                +EnDc_ConnRelease_X2apCuFail_count_SUM
                +EnDc_ConnRelease_X2apLinkFail_count_SUM
                +EnDc_ConnRelease_X2apRoTo_count_SUM
                +EnDc_ConnRelease_X2apSigFail_count_SUM
                +EnDc_ConnRelease_UpMacUEInact_count_SUM
                +EnDc_ConnRelease_CpCcNormal_count_SUM) ENDC_Cap_UE_RRC_Drop_pct_DENOM
      from apachedataingestion.lte.rrcconnectionreleaseofendccapableue_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")     
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T61 on M0.D_DTM=T61.D_DTM and M0.NE_ID=T61.NE_ID  and M0.LOCATION=T61.LOCATION
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            sum((EnDc_AddAtt_NrNcr_count_SUM-EnDc_AddRrcReconfigSucc_NrNcr_count_SUM)) SEA_ENDC_SgNBAdd_Fail_PCT_NCR_NUM,
            sum(EnDc_AddAtt_NrNcr_count_SUM) SEA_ENDC_SgNBAdd_Fail_PCT_NCR_DENOM,
            sum(EnDc_AddFailSyncReconfigFail_NrNcr_count_SUM) EnDc_AddFailSyncReconfigFail_NrNcr,
            sum(EnDc_AddRrcReconfigSucc_NrNcr_count_SUM) EnDc_AddRrcReconfigSucc_NrNcr,
            sum(EnDc_AddPrepFail_NrNcr_count_SUM) EnDc_AddPrepFail_NrNcr,
            sum(EnDc_AddAtt_NrNcr_count_SUM) EnDc_AddAtt_NrNcr
      from apachedataingestion.lte.endcadditioninformationpernrncr_lte_hr_silver
     WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")      
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T62 on M0.D_DTM=T62.D_DTM and M0.NE_ID=T62.NE_ID  and M0.LOCATION=T62.LOCATION    
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            sum(CellAvailPmPeriodTime_s_SUM - ReadCellUnavailableTime_s_SUM+ReadCellUnavailableTimeLock_s_SUM+ReadCellUnavailableTimeES_s_SUM) Cell_Availability_PCT_NUM,
            sum(CellAvailPmPeriodTime_s_SUM) Cell_Availability_PCT_DENOM,
            sum(ReadCellUnavailableTimeLock_s_SUM) ReadCellUnavailableTimeLock_s_SUM,
            sum(ReadCellUnavailableTimeES_s_SUM) ReadCellUnavailableTimeES_s_SUM,
            sum(CellAvailPmPeriodTime_s_SUM) CellAvailPmPeriodTime_s_SUM
      from apachedataingestion.lte.availability_lte_hr_silver
     WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")        
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
    ) T63 on M0.D_DTM=T63.D_DTM and M0.NE_ID=T63.NE_ID  and M0.LOCATION=T63.LOCATION    
      full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  (1.41*sum(PdcpTimeDl)/3600000) as ASEU
            from  apachedataingestion.lte.pdcppacket_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
          and split(LOCATION,'/')[1] in ('QCI1','QCI7','QCI8','QCI9','QCI132')  
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
          ) T64 on M0.D_DTM=T64.D_DTM and M0.NE_ID=T64.NE_ID  and M0.LOCATION=T64.LOCATION   
      full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  sum(CellCallContextCntTot) CellCallContextCntAvg_Num,
                  sum(CellCallContextCntCnt) CellCallContextCntAvg_Denom,
                  max(CellCallContextCntMax) CellCallContextCntMax
                  -- select datetime_utc
            from  apachedataingestion.lte.ltecallcontextpercell_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) T65 on M0.D_DTM=T65.D_DTM and M0.NE_ID=T65.NE_ID  and M0.LOCATION=T65.LOCATION      
  ) K            
) X
left join bh_conf B on cast(decode(len(SYSTEM_ID2),5,left(SYSTEM_ID2,2),10,left(SYSTEM_ID2,2),mod(left(SYSTEM_ID2,3),300) ) as int)=B.SUB_MARKET_ID and hour(D_DTM)=B.Busy_Hr
left join (
    select replace(ENODEB_ID,'VZ_','eNB_') NE_ID,concat('cNum',ECELL_ID) LOCATION,
    max(MARKET_NAME) MARKET_NAME,max(SUB_MARKET_ID) SUB_MARKET_ID,max(SUB_MARKET_NAME) SUB_MARKET_NAME,max(BAND_NAME) BAND_NAME 
    from hive_metastore.default.vz_conf_ecell_silver group by ENODEB_ID,ECELL_ID
) Y on X.NE_ID=Y.NE_ID and X.LOCATION=Y.LOCATION
Group by X.NE_ID,X.SYSTEM_ID2,X.LOCATION,D_DTM,Timezone,SITE_ID,bhcode,Y.MARKET_NAME,Y.SUB_MARKET_ID,Y.SUB_MARKET_NAME,Y.BAND_NAME;



delete from hive_metastore.default.widetable_4g_site_gold2_tmp2
;

insert into hive_metastore.default.widetable_4g_site_gold2_tmp2
with bh_conf as (
  select SUB_MARKET_ID,REGION_SILVER_TB, Busy_Hr,bhcode from hive_metastore.default.vz_conf_market A
  inner join (
    select DAILY,REGION,Busy_Hr, sum(bhcode) bhcode from (            
        select DAILY,REGION,LTE_Busy_Hour 	Busy_Hr,1 bhcode from snap.enb_iris.4g_busy_hour_kpi_gold where Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")
        union all
        select DAILY,REGION,VoLTE_Busy_Hour Busy_Hr,2 bhcode from snap.enb_iris.4g_busy_hour_kpi_gold where Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")        
    ) x group by DAILY,REGION,Busy_Hr
  ) B on A.REGION_SILVER_TB=B.REGION
)
SELECT 
convert_timezone('UTC','America/Chicago',current_timestamp())  Created_DTM,
decode(bhcode,null,0,bhcode) Busy_Hr,
/*X.NE_ID,*/
case when left(X.NE_ID,5)=='UADPF' and length(Y.SUB_MARKET_ID) = 3 then  replace(concat(left(X.NE_ID,9),right(X.NE_ID,3)),'UADPF','eNB') when left(X.NE_ID,5)=='UADPF' and length(Y.SUB_MARKET_ID) = 2 then replace(concat(left(X.NE_ID,8),right(X.NE_ID,3)),'UADPF','eNB')  else X.NE_ID  end NE_ID,            
X.SYSTEM_ID2 SYSTEM_ID,
X.LOCATION,D_DTM,Timezone,replace(SITE_ID,'N/A',null) SITE_ID,
/*case when length(ne_ID)=5 then substring(ne_id, 1, 2)
     when length(ne_ID)=6 then substring(ne_id, 1, 3)
     when length(ne_ID)=9 then substring(ne_id, 5, 2)
     when length(ne_ID)=10 then substring(ne_id,5, 3)
     when length(ne_ID)=16 then substring(ne_id, 7, 2)
     when length(ne_ID)=17 then substring(ne_id, 7, 3) end SITE_ID,*/ 
Y.MARKET_NAME,
Y.SUB_MARKET_ID,
Y.SUB_MARKET_NAME,
Y.BAND_NAME,     
max(	SEA_ENDC_ERAB_Estab_Fail_pct_Num	)	SEA_ENDC_ERAB_Estab_Fail_pct_Num	,
max(	SEA_ENDC_ERAB_Estab_Fail_pct_Denom	)	SEA_ENDC_ERAB_Estab_Fail_pct_Denom	,
max(	SEA_ENDC_IMLB_UE_Count	)	SEA_ENDC_IMLB_UE_Count	,
max(	SEA_ENDC_ERAB_Modif_Fail_pct_Num	)	SEA_ENDC_ERAB_Modif_Fail_pct_Num	,
max(	SEA_ENDC_ERAB_Modif_Fail_pct_Denom	)	SEA_ENDC_ERAB_Modif_Fail_pct_Denom	,
max(	SEA_ENDC_IntraMenbSgnbNoChgHO_ModifPrep_Fail_pct_Num	)	SEA_ENDC_IntraMenbSgnbNoChgHO_ModifPrep_Fail_pct_Num	,
max(	SEA_ENDC_IntraMenbSgnbNoChgHO_ModifPrep_Fail_pct_Denom	)	SEA_ENDC_IntraMenbSgnbNoChgHO_ModifPrep_Fail_pct_Denom	,
max(	SEA_ENDC_IntraMenbSgnbNoChgHO_Modif_Fail_pct_Num	)	SEA_ENDC_IntraMenbSgnbNoChgHO_Modif_Fail_pct_Num	,
max(	SEA_ENDC_IntraMenbSgnbNoChgHO_Modif_Fail_pct_Denom	)	SEA_ENDC_IntraMenbSgnbNoChgHO_Modif_Fail_pct_Denom	,
max(	SEA_ENDC_InterMenbSgnbNoChgHO_AddPrep_Fail_pct_Num	)	SEA_ENDC_InterMenbSgnbNoChgHO_AddPrep_Fail_pct_Num	,
max(	SEA_ENDC_InterMenbSgnbNoChgHO_AddPrep_Fail_pct_Denom	)	SEA_ENDC_InterMenbSgnbNoChgHO_AddPrep_Fail_pct_Denom	,
max(	SEA_ENDC_InterMenbSgnbNoChgHO_Add_Fail_pct_Num	)	SEA_ENDC_InterMenbSgnbNoChgHO_Add_Fail_pct_Num	,
max(	SEA_ENDC_InterMenbSgnbNoChgHO_Add_Fail_pct_Denom	)	SEA_ENDC_InterMenbSgnbNoChgHO_Add_Fail_pct_Denom	,
max(	SEA_X2HOInterruptionTime_ms_Num	)	SEA_X2HOInterruptionTime_ms_Num	,
max(	SEA_X2HOInterruptionTime_ms_Denom	)	SEA_X2HOInterruptionTime_ms_Denom	,
max(	SEA_PdcpIpLatencyDL_ms_QCI1_Num	)	SEA_PdcpIpLatencyDL_ms_QCI1_Num	,
max(	SEA_PdcpIpLatencyDL_ms_QCI1_Denom	)	SEA_PdcpIpLatencyDL_ms_QCI1_Denom	,
max(	CoverageHoleVoice	)	CoverageHoleVoice	,
max(	CoverageHoleRsrpVoice	)	CoverageHoleRsrpVoice	,
max(	HoTriggeringScellRsrpEventA3Tot	)	HoTriggeringScellRsrpEventA3Tot	,
max(	HoTriggeringScellRsrpEventA3Cnt	)	HoTriggeringScellRsrpEventA3Cnt	,
max(	DL_Active_UE_Num	)	DL_Active_UE_Num	,
max(	DL_Active_UE_Denom	)	DL_Active_UE_Denom	,
max(	UL_Active_UE_Num	)	UL_Active_UE_Num	,
max(	UL_Active_UE_Denom	)	UL_Active_UE_Denom	,
max(	UL_RSSI_Num	)	UL_RSSI_Num	,
max(	UL_RSSI_Denom	)	UL_RSSI_Denom	,
max(	DL_RI_Num	)	DL_RI_Num	,
max(	DL_RI_Denom	)	DL_RI_Denom	,
max(	Intra_Freq_HOFR_Denom	)	Intra_Freq_HOFR_Denom,
max(	Intra_Freq_HOFR_Num	)	Intra_Freq_HOFR_Num,
-- add DENG-2770
max(	5G_Fraction_of_RRC_Call_Hold_Time_pct_Num	)	5G_Fraction_of_RRC_Call_Hold_Time_pct_Num,
max(	SEA_ENDC_Addition_Duration_Sec_Num	)	SEA_ENDC_Addition_Duration_Sec_Num,
max(	SEA_ENDC_Addition_Duration_Sec_Denom          	)	SEA_ENDC_Addition_Duration_Sec_Denom,          
max(	SEA_ENDC_DSS_LTE_DL_TTI_UTIL_pct_Num	)	SEA_ENDC_DSS_LTE_DL_TTI_UTIL_pct_Num,
max(	SEA_ENDC_DSS_LTE_DL_TTI_UTIL_pct_Denom          	)	SEA_ENDC_DSS_LTE_DL_TTI_UTIL_pct_Denom,          
max(	SEA_ENDC_DSS_LTE_DL_PRB_UTIL_pct_Num	)	SEA_ENDC_DSS_LTE_DL_PRB_UTIL_pct_Num,
max(	SEA_ENDC_DSS_LTE_DL_PRB_UTIL_pct_Denom          	)	SEA_ENDC_DSS_LTE_DL_PRB_UTIL_pct_Denom,          
max(	SEA_ENDC_DSS_LTE_UL_TTI_UTIL_pct_Num	)	SEA_ENDC_DSS_LTE_UL_TTI_UTIL_pct_Num,
max(	SEA_ENDC_DSS_LTE_UL_TTI_UTIL_pct_Denom	)	SEA_ENDC_DSS_LTE_UL_TTI_UTIL_pct_Denom,
max(	SEA_ENDC_DSS_LTE_UL_PRB_UTIL_pct_Num	)	SEA_ENDC_DSS_LTE_UL_PRB_UTIL_pct_Num,
max(	SEA_ENDC_DSS_LTE_UL_PRB_UTIL_pct_Denom	)	SEA_ENDC_DSS_LTE_UL_PRB_UTIL_pct_Denom,
max(	SEA_ENDC_DSS_LTE_DL_PRB_Ratio_pct_Num	)	SEA_ENDC_DSS_LTE_DL_PRB_Ratio_pct_Num,
max(	SEA_ENDC_DSS_NR_DL_PRB_Ratio_pct_Num	)	SEA_ENDC_DSS_NR_DL_PRB_Ratio_pct_Num,
max(	SEA_ENDC_DSS_LTE_UL_PRB_Ratio_pct_Num	)	SEA_ENDC_DSS_LTE_UL_PRB_Ratio_pct_Num,
max(	SEA_ENDC_DSS_NR_UL_PRB_Ratio_pct_Num	)	SEA_ENDC_DSS_NR_UL_PRB_Ratio_pct_Num,
max(	SEA_ENDC_DSS_LTE_DL_TTI_Ratio_pct_Num	)	SEA_ENDC_DSS_LTE_DL_TTI_Ratio_pct_Num,
max(	SEA_ENDC_DSS_NR_DL_TTI_Ratio_pct_Num	)	SEA_ENDC_DSS_NR_DL_TTI_Ratio_pct_Num,
max(	SEA_ENDC_DSS_LTE_UL_TTI_Ratio_pct_Num	)	SEA_ENDC_DSS_LTE_UL_TTI_Ratio_pct_Num,
max(	SEA_ENDC_DSS_NR_UL_TTI_Ratio_pct_Num	)	SEA_ENDC_DSS_NR_UL_TTI_Ratio_pct_Num,
max(	SEA_ENDC_UL_VOLUME_FR1_MB	)	SEA_ENDC_UL_VOLUME_FR1_MB,
max(	SEA_ENDC_UL_THROUGHPUT_FR1_Mbps_Num	)	SEA_ENDC_UL_THROUGHPUT_FR1_Mbps_Num,
max(	SEA_ENDC_UL_THROUGHPUT_FR1_Mbps_Denom	)	SEA_ENDC_UL_THROUGHPUT_FR1_Mbps_Denom,
max(	SEA_ENDC_UL_VOLUME_FR2_MB	)	SEA_ENDC_UL_VOLUME_FR2_MB,
max(	SEA_ENDC_UL_THROUGHPUT_FR2_Mbps_Num	)	SEA_ENDC_UL_THROUGHPUT_FR2_Mbps_Num,
max(	SEA_ENDC_UL_THROUGHPUT_FR2_Mbps_Denom	)	SEA_ENDC_UL_THROUGHPUT_FR2_Mbps_Denom,
max(	SEA_NON_ENDC_UL_VOLUME__MB	)	SEA_NON_ENDC_UL_VOLUME__MB,
max(	SEA_NON_ENDC_UL_THROUGHPUT_Mbps_Num	)	SEA_NON_ENDC_UL_THROUGHPUT_Mbps_Num,
max(	SEA_NON_ENDC_UL_THROUGHPUT_Mbps_Denom	)	SEA_NON_ENDC_UL_THROUGHPUT_Mbps_Denom,
max(	SEA_ENDC_DL_VOLUME_FR1TDD_MB	)	SEA_ENDC_DL_VOLUME_FR1TDD_MB,
max(	SEA_ENDC_DL_THROUGHPUT_FR1TDD_Mbps_Num	)	SEA_ENDC_DL_THROUGHPUT_FR1TDD_Mbps_Num,
max(	SEA_ENDC_DL_THROUGHPUT_FR1TDD_Mbps_Denom	)	SEA_ENDC_DL_THROUGHPUT_FR1TDD_Mbps_Denom,
max(	SEA_ENDC_DL_VOLUME_FR1FDD_MB	)	SEA_ENDC_DL_VOLUME_FR1FDD_MB,
max(	SEA_ENDC_DL_THROUGHPUT_FR1FDD_Mbps_Num	)	SEA_ENDC_DL_THROUGHPUT_FR1FDD_Mbps_Num,
max(	SEA_ENDC_DL_THROUGHPUT_FR1FDD_Mbps_Denom	)	SEA_ENDC_DL_THROUGHPUT_FR1FDD_Mbps_Denom,
max(	SEA_ENDC_DL_VOLUME_FR1_MB	)	SEA_ENDC_DL_VOLUME_FR1_MB,
max(	SEA_ENDC_DL_THROUGHPUT_FR1_Mbps_Num	)	SEA_ENDC_DL_THROUGHPUT_FR1_Mbps_Num,
max(	SEA_ENDC_DL_THROUGHPUT_FR1_Mbps_Denom	)	SEA_ENDC_DL_THROUGHPUT_FR1_Mbps_Denom,
max(	SEA_ENDC_DL_VOLUME_FR2_MB	)	SEA_ENDC_DL_VOLUME_FR2_MB,
max(	SEA_ENDC_DL_THROUGHPUT_FR2_Mbps_Num	)	SEA_ENDC_DL_THROUGHPUT_FR2_Mbps_Num,
max(	SEA_ENDC_DL_THROUGHPUT_FR2_Mbps_Denom	)	SEA_ENDC_DL_THROUGHPUT_FR2_Mbps_Denom,
max(	SEA_NON_ENDC_DL_VOLUME_MB	)	SEA_NON_ENDC_DL_VOLUME_MB,
max(	SEA_NON_ENDC_DL_THROUGHPUT_Mbps_Num	)	SEA_NON_ENDC_DL_THROUGHPUT_Mbps_Num,
max(	SEA_NON_ENDC_DL_THROUGHPUT_Mbps_Denom	)	SEA_NON_ENDC_DL_THROUGHPUT_Mbps_Denom,
max(	UE_Power_Headroom_Num	)	UE_Power_Headroom_Num,
max(	UE_Power_Headroom_Denom	)	UE_Power_Headroom_Denom,
max(	CBRA_RACH_SR_Num	)	CBRA_RACH_SR_Num,
max(	CBRA_RACH_SR_Denom	)	CBRA_RACH_SR_Denom,
max(CellUnavailableTimeLock) CellUnavailableTimeLock,
max(CellUnavailableTimeDown) CellUnavailableTimeDown,                
max(CellUnavailableTimeES) CellUnavailableTimeES,
max(RRC_Redirection_CAC) RRC_Redirection_CAC,
max(RRC_Release_Preemption) RRC_Release_Preemption
from ( 	 
SELECT  cast(replace(replace(K.NE_ID,'eNB_',''),'UADPF_','') as int) SYSTEM_ID2,* from (
        SELECT 
          COALESCE(M0.NE_ID,T1.NE_ID,T2.NE_ID,T3.NE_ID,T4.NE_ID,T5.NE_ID,T6.NE_ID,T7.NE_ID,T8.NE_ID,T9.NE_ID,T10.NE_ID,T11.NE_ID,T12.NE_ID,T13.NE_ID,T14.NE_ID,T15.NE_ID,T16.NE_ID,T17.NE_ID,T18.NE_ID,T19.NE_ID,T20.NE_ID,T21.NE_ID) NE_ID,          
          COALESCE(M0.LOCATION,T1.LOCATION,T2.LOCATION,T3.LOCATION,T4.LOCATION,T5.LOCATION,T6.LOCATION,T7.LOCATION,T8.LOCATION,T9.LOCATION,T10.LOCATION,T11.LOCATION,T12.LOCATION,T13.LOCATION,T14.LOCATION,T15.LOCATION,T16.LOCATION,T17.LOCATION,T18.LOCATION,T19.LOCATION,T20.LOCATION,T21.LOCATION) LOCATION,
          COALESCE(M0.D_DTM,T1.D_DTM,T2.D_DTM,T3.D_DTM,T4.D_DTM,T5.D_DTM,T6.D_DTM,T7.D_DTM,T8.D_DTM,T9.D_DTM,T10.D_DTM,T11.D_DTM,T12.D_DTM,T13.D_DTM,T14.D_DTM,T15.D_DTM,T16.D_DTM,T17.D_DTM,T18.D_DTM,T19.D_DTM,T20.D_DTM,T21.D_DTM) D_DTM,
          COALESCE(M0.Timezone,T1.Timezone,T2.Timezone,T3.Timezone,T4.Timezone,T5.Timezone,T6.Timezone,T7.Timezone,T8.Timezone,T9.Timezone,T10.Timezone,T11.Timezone,T12.Timezone,T13.Timezone,T14.Timezone,T15.Timezone,T16.Timezone,T17.Timezone,T18.Timezone,T19.Timezone,T20.Timezone,T21.Timezone) Timezone,
          COALESCE(M0.SITE_ID,T1.SITE_ID,T2.SITE_ID,T3.SITE_ID,T4.SITE_ID,T5.SITE_ID,T6.SITE_ID,T7.SITE_ID,T8.SITE_ID,T9.SITE_ID,T10.SITE_ID,T11.SITE_ID,T12.SITE_ID,T13.SITE_ID,T14.SITE_ID,T15.SITE_ID,T16.SITE_ID,T17.SITE_ID,T18.SITE_ID,T19.SITE_ID,T20.SITE_ID,T21.SITE_ID) SITE_ID,
          SEA_ENDC_ERAB_Estab_Fail_pct_Num	,
          SEA_ENDC_ERAB_Estab_Fail_pct_Denom	,
          SEA_ENDC_IMLB_UE_Count	,
          SEA_ENDC_ERAB_Modif_Fail_pct_Num	,
          SEA_ENDC_ERAB_Modif_Fail_pct_Denom	,
          SEA_ENDC_IntraMenbSgnbNoChgHO_ModifPrep_Fail_pct_Num	,
          SEA_ENDC_IntraMenbSgnbNoChgHO_ModifPrep_Fail_pct_Denom	,
          SEA_ENDC_IntraMenbSgnbNoChgHO_Modif_Fail_pct_Num	,
          SEA_ENDC_IntraMenbSgnbNoChgHO_Modif_Fail_pct_Denom	,
          SEA_ENDC_InterMenbSgnbNoChgHO_AddPrep_Fail_pct_Num	,
          SEA_ENDC_InterMenbSgnbNoChgHO_AddPrep_Fail_pct_Denom	,
          SEA_ENDC_InterMenbSgnbNoChgHO_Add_Fail_pct_Num	,
          SEA_ENDC_InterMenbSgnbNoChgHO_Add_Fail_pct_Denom	,
          SEA_X2HOInterruptionTime_ms_Num	,
          SEA_X2HOInterruptionTime_ms_Denom	,
          SEA_PdcpIpLatencyDL_ms_QCI1_Num	,
          SEA_PdcpIpLatencyDL_ms_QCI1_Denom	,
          CoverageHoleVoice	,
          CoverageHoleRsrpVoice	,
          HoTriggeringScellRsrpEventA3Tot	,
          HoTriggeringScellRsrpEventA3Cnt	,
          DL_Active_UE_Num	,
          DL_Active_UE_Denom	,
          UL_Active_UE_Num	,
          UL_Active_UE_Denom	,
          UL_RSSI_Num	,
          UL_RSSI_Denom	,
          DL_RI_Num	,
          DL_RI_Denom	,
          Intra_Freq_HOFR_Denom,
          Intra_Freq_HOFR_Num,    
-- add DENG-2770          
EnDc_AdditionTimeTot	5G_Fraction_of_RRC_Call_Hold_Time_pct_Num,
EnDc_AdditionTimeTot	SEA_ENDC_Addition_Duration_Sec_Num,	
EnDc_AdditionTimeCnt	SEA_ENDC_Addition_Duration_Sec_Denom,          
DssLteDLTtiUsed	SEA_ENDC_DSS_LTE_DL_TTI_UTIL_pct_Num,	
DssLteDLTtiAvailable	SEA_ENDC_DSS_LTE_DL_TTI_UTIL_pct_Denom,          
DssLteDLPrbUsed	SEA_ENDC_DSS_LTE_DL_PRB_UTIL_pct_Num,	
DssLteDLPrbAvailable	SEA_ENDC_DSS_LTE_DL_PRB_UTIL_pct_Denom,          
DssLteULTtiUsed	SEA_ENDC_DSS_LTE_UL_TTI_UTIL_pct_Num,	
DssLteULTtiAvailable	SEA_ENDC_DSS_LTE_UL_TTI_UTIL_pct_Denom,
DssLteULPrbUsed	SEA_ENDC_DSS_LTE_UL_PRB_UTIL_pct_Num,	
DssLteULPrbAvailable	SEA_ENDC_DSS_LTE_UL_PRB_UTIL_pct_Denom,
DssLteDLPrbAvailable	SEA_ENDC_DSS_LTE_DL_PRB_Ratio_pct_Num,	
DssNrDLPrbReserved	SEA_ENDC_DSS_NR_DL_PRB_Ratio_pct_Num,	
DssLteULPrbAvailable	SEA_ENDC_DSS_LTE_UL_PRB_Ratio_pct_Num,	
DssNrULPrbReserved	SEA_ENDC_DSS_NR_UL_PRB_Ratio_pct_Num,	
DssLteDLTtiAvailable	SEA_ENDC_DSS_LTE_DL_TTI_Ratio_pct_Num,	
DssNrDLTTiReserved	SEA_ENDC_DSS_NR_DL_TTI_Ratio_pct_Num,	
DssLteULTtiAvailable	SEA_ENDC_DSS_LTE_UL_TTI_Ratio_pct_Num,	
DssNrULTTiReserved	SEA_ENDC_DSS_NR_UL_TTI_Ratio_pct_Num,	
ULDataVolumeFr1/1000	SEA_ENDC_UL_VOLUME_FR1_MB,		
(ULDataVolumeFr1/1000)	SEA_ENDC_UL_THROUGHPUT_FR1_Mbps_Num,
ULTimePeriodFr1	SEA_ENDC_UL_THROUGHPUT_FR1_Mbps_Denom,
ULDataVolumeFr2/1000	SEA_ENDC_UL_VOLUME_FR2_MB,		
(ULDataVolumeFr2/1000)	SEA_ENDC_UL_THROUGHPUT_FR2_Mbps_Num,	
ULTimePeriodFr2	SEA_ENDC_UL_THROUGHPUT_FR2_Mbps_Denom,
ULDataVolumeNoEndc/1000	SEA_NON_ENDC_UL_VOLUME__MB,		
(ULDataVolumeNoEndc/1000)	SEA_NON_ENDC_UL_THROUGHPUT_Mbps_Num,	
ULTimePeriodNoEndc	SEA_NON_ENDC_UL_THROUGHPUT_Mbps_Denom,
(AirRlcPduDLByte_FR1_TDD_ENDC/1000)	SEA_ENDC_DL_VOLUME_FR1TDD_MB,		
(8*IpThruVoDLByte_FR1_TDD_ENDC/1000)	SEA_ENDC_DL_THROUGHPUT_FR1TDD_Mbps_Num,	
(IpThruDLTime_FR1_TDD_ENDC/1000)	SEA_ENDC_DL_THROUGHPUT_FR1TDD_Mbps_Denom,
(AirRlcPduDLByte_FR1_FDD_ENDC/1000)	SEA_ENDC_DL_VOLUME_FR1FDD_MB,		
(8*IpThruVoDLByte_FR1_FDD_ENDC/1000)	SEA_ENDC_DL_THROUGHPUT_FR1FDD_Mbps_Num,	
(IpThruDLTime_FR1_FDD_ENDC/1000)	SEA_ENDC_DL_THROUGHPUT_FR1FDD_Mbps_Denom,
((AirRlcPduDLByte_FR1_FDD_ENDC+AirRlcPduDLByte_FR1_TDD_ENDC)/1000)	SEA_ENDC_DL_VOLUME_FR1_MB,		
(8*(IpThruVoDLByte_FR1_FDD_ENDC+IpThruVoDLByte_FR1_TDD_ENDC)/1000)	SEA_ENDC_DL_THROUGHPUT_FR1_Mbps_Num,	
((IpThruDLTime_FR1_FDD_ENDC+IpThruDLTime_FR1_TDD_ENDC)/1000)	SEA_ENDC_DL_THROUGHPUT_FR1_Mbps_Denom,
(AirRlcPduDLByte_FR2_ENDC/1000)	SEA_ENDC_DL_VOLUME_FR2_MB,		
(8*IpThruVoDLByte_FR2_ENDC/1000)	SEA_ENDC_DL_THROUGHPUT_FR2_Mbps_Num,	
(IpThruDLTime_FR2_ENDC/1000)	SEA_ENDC_DL_THROUGHPUT_FR2_Mbps_Denom,
(AirRlcPduDLByte_NON_ENDC/1000)	SEA_NON_ENDC_DL_VOLUME_MB,		
(8*IpThruVoDLByte_NON_ENDC/1000)	SEA_NON_ENDC_DL_THROUGHPUT_Mbps_Num,	
(IpThruDLTime_NON_ENDC/1000)	SEA_NON_ENDC_DL_THROUGHPUT_Mbps_Denom,
UE_Power_Headroom_Num,
UE_Power_Headroom_Denom,
(NumMsg4GroupA + NumMsg4GroupB) CBRA_RACH_SR_Num,
(RandomlySelectedPreamblesLow + RandomlySelectedPreambleshigh)	CBRA_RACH_SR_Denom,
CellUnavailableTimeLock,
CellUnavailableTimeDown,                
CellUnavailableTimeES,
RedirLbCac RRC_Redirection_CAC,
ConnRelease_CpPreemption RRC_Release_Preemption
-- select *          
  from (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  sum(EnDc_ErabEstabAtt - EnDc_ErabEstabSucc)	SEA_ENDC_ERAB_Estab_Fail_pct_Num,
                  sum(EnDc_ErabEstabAtt) SEA_ENDC_ERAB_Estab_Fail_pct_Denom
                  -- select *
            from  apachedataingestion.lte.ENDCERABEstablishInformation_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) M0			        
    full join  (    
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  sum(EnDc_UECount) SEA_ENDC_IMLB_UE_Count
                  -- select *
            from  apachedataingestion.lte.ENDCIdleLoadBalancing_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) T1 on M0.D_DTM=T1.D_DTM and M0.NE_ID=T1.NE_ID  and M0.LOCATION=T1.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  sum(EnDc_ErabModAtt - EnDc_ErabModSucc)	SEA_ENDC_ERAB_Modif_Fail_pct_Num,
                  sum(EnDc_ErabModAtt)  SEA_ENDC_ERAB_Modif_Fail_pct_Denom

                  -- select *
            from  apachedataingestion.lte.ENDCERABModificationInformation_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
     ) T2 on M0.D_DTM=T2.D_DTM and M0.NE_ID=T2.NE_ID  and M0.LOCATION=T2.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            sum(IntraMenbToMenbNoSgnbChg_ModAtt - IntraMenbToMenbNoSgnbChg_ModPrepSucc)	SEA_ENDC_IntraMenbSgnbNoChgHO_ModifPrep_Fail_pct_Num,
            sum(IntraMenbToMenbNoSgnbChg_ModAtt) SEA_ENDC_IntraMenbSgnbNoChgHO_ModifPrep_Fail_pct_Denom,
            sum(IntraMenbToMenbNoSgnbChg_ModAtt - IntraMenbToMenbNoSgnbChg_ModSucc)	 SEA_ENDC_IntraMenbSgnbNoChgHO_Modif_Fail_pct_Num,
            sum(IntraMenbToMenbNoSgnbChg_ModAtt) SEA_ENDC_IntraMenbSgnbNoChgHO_Modif_Fail_pct_Denom                  
                  -- select *
            from  apachedataingestion.lte.SgNBModificationduetoMCGMobility_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) T3 on M0.D_DTM=T3.D_DTM and M0.NE_ID=T3.NE_ID  and M0.LOCATION=T3.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            sum(InterMenbToMenbNoSgnbChg_AddAtt - InterMenbToMenbNoSgnbChg_AddPrepSucc )	 SEA_ENDC_InterMenbSgnbNoChgHO_AddPrep_Fail_pct_Num,
            sum(InterMenbToMenbNoSgnbChg_AddAtt)  SEA_ENDC_InterMenbSgnbNoChgHO_AddPrep_Fail_pct_Denom,
            sum(InterMenbToMenbNoSgnbChg_AddAtt - InterMenbToMenbNoSgnbChg_AddSucc )	 SEA_ENDC_InterMenbSgnbNoChgHO_Add_Fail_pct_Num,
            sum(InterMenbToMenbNoSgnbChg_AddAtt) SEA_ENDC_InterMenbSgnbNoChgHO_Add_Fail_pct_Denom                  
                  -- select *
            from  apachedataingestion.lte.SgNBAdditionduetoMCGMobility_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) T4 on M0.D_DTM=T4.D_DTM and M0.NE_ID=T4.NE_ID  and M0.LOCATION=T4.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  sum(X2HOTimeTot)	SEA_X2HOInterruptionTime_ms_Num,
                  sum(X2HOTimeCnt) SEA_X2HOInterruptionTime_ms_Denom
                  -- select *
            from  apachedataingestion.lte.HandoverTime_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) T5 on M0.D_DTM=T5.D_DTM and M0.NE_ID=T5.NE_ID  and M0.LOCATION=T5.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  sum(IpLateDLTot)	 SEA_PdcpIpLatencyDL_ms_QCI1_Num,
                  sum(IpLateDLCnt)  SEA_PdcpIpLatencyDL_ms_QCI1_Denom
                  -- select *
            from  apachedataingestion.lte.IPLatency_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
          and split(LOCATION,'/')[1] ='QCI1'
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) T6 on M0.D_DTM=T6.D_DTM and M0.NE_ID=T6.NE_ID  and M0.LOCATION=T6.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  sum(CoverageHoleVoice)	CoverageHoleVoice,
                  sum(CoverageHoleRsrpVoice) CoverageHoleRsrpVoice
                  -- select *
            from  apachedataingestion.lte.MRORLFVOICEClassification_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
     ) T7 on M0.D_DTM=T7.D_DTM and M0.NE_ID=T7.NE_ID  and M0.LOCATION=T7.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  sum(HoTriggeringScellRsrpEventA3Tot)	HoTriggeringScellRsrpEventA3Tot,
                  sum(HoTriggeringScellRsrpEventA3Cnt) HoTriggeringScellRsrpEventA3Cnt
                  -- select *
            from  apachedataingestion.lte.MeasurementResultsofSourceCell_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) T8 on M0.D_DTM=T8.D_DTM and M0.NE_ID=T8.NE_ID  and M0.LOCATION=T8.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  sum(UEActiveDLTot)	DL_Active_UE_Num,
                  sum(UEActiveDLCnt) DL_Active_UE_Denom,
                  sum(UEActiveULTot)	 UL_Active_UE_Num,
                  sum(UEActiveULCnt)  UL_Active_UE_Denom
                  -- select *
            from  apachedataingestion.lte.ActiveUENumber_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) T9 on M0.D_DTM=T9.D_DTM and M0.NE_ID=T9.NE_ID  and M0.LOCATION=T9.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                  sum(RssiPathTot)	UL_RSSI_Num,
                  sum(RssiPathCnt) UL_RSSI_Denom
                  -- select *
            from  apachedataingestion.lte.RSSIofEachPathMeasuredbyMAC_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
      ) T10 on M0.D_DTM=T10.D_DTM and M0.NE_ID=T10.NE_ID  and M0.LOCATION=T10.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,                  
                sum(DLReceivedRI0*0 + DLReceivedRI1*1 + DLReceivedRI2*2 + DLReceivedRI3*3 + DLReceivedRI4*4) DL_RI_Num,
                sum(DLReceivedRI0 + DLReceivedRI1 + DLReceivedRI2 + DLReceivedRI3 + DLReceivedRI4) DL_RI_Denom
                  -- select *
            from  apachedataingestion.lte.DLRI_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
     ) T11 on M0.D_DTM=T11.D_DTM and M0.NE_ID=T11.NE_ID  and M0.LOCATION=T11.LOCATION
    full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
                    sum(IntraFreqOutWithHoCauseAtt)	 Intra_Freq_HOFR_Denom,
                    sum(IntraFreqOutWithHoCauseAtt - IntraFreqOutWithHoCauseSucc)	Intra_Freq_HOFR_Num
                  -- select *
            from  apachedataingestion.lte.OutgoingIntraFrequencyHandoverWithHandoverCause_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
          and split(LOCATION,'/')[1]='RADIO_REASON'  
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
       ) T12 on M0.D_DTM=T12.D_DTM and M0.NE_ID=T12.NE_ID  and M0.LOCATION=T12.LOCATION     
      
      --  add DENG-2770
       full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            -- select 
                    sum(DssLteDLTtiUsed) DssLteDLTtiUsed,
                    sum(DssLteDLPrbUsed) DssLteDLPrbUsed,
                    sum(DssLteULTtiUsed) DssLteULTtiUsed,	
                    sum(DssLteULPrbUsed) DssLteULPrbUsed,	
                    sum(DssLteDLPrbAvailable) DssLteDLPrbAvailable,	
                    sum(DssNrDLPrbReserved) DssNrDLPrbReserved,	
                    sum(DssLteULPrbAvailable) DssLteULPrbAvailable,	
                    sum(DssNrULPrbReserved) DssNrULPrbReserved,	
                    sum(DssLteDLTtiAvailable) DssLteDLTtiAvailable,	
                    sum(DssNrDLTTiReserved) DssNrDLTTiReserved,	
                    sum(DssLteULTtiAvailable) DssLteULTtiAvailable,	
                    sum(DssNrULTTiReserved)  DssNrULTTiReserved
                  -- select *
            from  apachedataingestion.lte.DSSRelatedInformation_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
       ) T13 on M0.D_DTM=T13.D_DTM and M0.NE_ID=T13.NE_ID  and M0.LOCATION=T13.LOCATION 
full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            -- select 
                    SUM(case when split(LOCATION,'/')[3] IN ('FR2_ENDC') then AirRlcPduDLByte else 0 end) AirRlcPduDLByte_FR2_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('FR2_ENDC') then IpThruVoDLByte else 0 end) IpThruVoDLByte_FR2_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('FR2_ENDC') then IpThruDLTime else 0 end) IpThruDLTime_FR2_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('FR1_FDD_ENDC') then AirRlcPduDLByte else 0 end) AirRlcPduDLByte_FR1_FDD_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('FR1_FDD_ENDC') then IpThruVoDLByte else 0 end) IpThruVoDLByte_FR1_FDD_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('FR1_FDD_ENDC') then IpThruDLTime else 0 end) IpThruDLTime_FR1_FDD_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('FR1_TDD_ENDC') then AirRlcPduDLByte else 0 end) AirRlcPduDLByte_FR1_TDD_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('FR1_TDD_ENDC') then IpThruVoDLByte else 0 end) IpThruVoDLByte_FR1_TDD_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('FR1_TDD_ENDC') then IpThruDLTime else 0 end) IpThruDLTime_FR1_TDD_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('NON_ENDC') then AirRlcPduDLByte else 0 end) AirRlcPduDLByte_NON_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('NON_ENDC') then IpThruVoDLByte else 0 end) IpThruVoDLByte_NON_ENDC,
                    SUM(case when split(LOCATION,'/')[3] IN ('NON_ENDC') then IpThruDLTime else 0 end) IpThruDLTime_NON_ENDC  
                  -- select *
            from  apachedataingestion.lte.AirRLCPacketperPLMNperENDCStatus_lte_hr_silver 
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
       ) T14 on M0.D_DTM=T14.D_DTM and M0.NE_ID=T14.NE_ID  and M0.LOCATION=T14.LOCATION   
full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            -- select                     
                sum(ULDataVolumeFr1) ULDataVolumeFr1,
                sum(ULTimePeriodFr1) ULTimePeriodFr1,
                sum(ULDataVolumeFr2) ULDataVolumeFr2,
                sum(ULTimePeriodFr2) ULTimePeriodFr2,
                sum(ULDataVolumeNoEndc) ULDataVolumeNoEndc,
                sum(ULTimePeriodNoEndc) ULTimePeriodNoEndc
                  -- select *
            from  apachedataingestion.lte.ULIPThroughputperENDCNRCelltype_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
       ) T15 on M0.D_DTM=T15.D_DTM and M0.NE_ID=T15.NE_ID  and M0.LOCATION=T15.LOCATION 
full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            -- select                                     
                sum(EnDc_AdditionTimeTot) EnDc_AdditionTimeTot,
                sum(EnDc_AdditionTimeCnt) EnDc_AdditionTimeCnt
                  -- select *
            from  apachedataingestion.lte.endcconnectioninformation_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
       ) T16 on M0.D_DTM=T16.D_DTM and M0.NE_ID=T16.NE_ID  and M0.LOCATION=T16.LOCATION   

full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            -- select                                     
                sum(NumMsg4GroupA) NumMsg4GroupA,
                sum(NumMsg4GroupB) NumMsg4GroupB,
                sum(RandomlySelectedPreamblesLow) RandomlySelectedPreamblesLow,
                sum(RandomlySelectedPreambleshigh) RandomlySelectedPreambleshigh
                  -- select *
            from  apachedataingestion.lte.randomaccesspreambles_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
       ) T17 on M0.D_DTM=T17.D_DTM and M0.NE_ID=T17.NE_ID  and M0.LOCATION=T17.LOCATION          

full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            -- select   
                sum(PhrIndex0_count_*Power(10,-2.3) + PhrIndex1_count_*Power(10,-2.2) + PhrIndex2_count_*Power(10,-2.1) + PhrIndex3_count_*Power(10,-2) + PhrIndex4_count_*Power(10,-1.9) + PhrIndex5_count_*Power(10,-1.8) + PhrIndex6_count_*Power(10,-1.7) + PhrIndex7_count_*Power(10,-1.6) + PhrIndex8_count_*Power(10,-1.5) + PhrIndex9_count_*Power(10,-1.4) + PhrIndex10_count_*Power(10,-1.3) + PhrIndex11_count_*Power(10,-1.2) + PhrIndex12_count_*Power(10,-1.1) + PhrIndex13_count_*Power(10,-1) + PhrIndex14_count_*Power(10,-0.9) + PhrIndex15_count_*Power(10,-0.8) + PhrIndex16_count_*Power(10,-0.7) + PhrIndex17_count_*Power(10,-0.6) + PhrIndex18_count_*Power(10,-0.5) + PhrIndex19_count_*Power(10,-0.4) + PhrIndex20_count_*Power(10,-0.3) + PhrIndex21_count_*Power(10,-0.2) + PhrIndex22_count_*Power(10,-0.1) + PhrIndex23_count_*Power(10,0) + PhrIndex24_count_*Power(10,0.1) + PhrIndex25_count_*Power(10,0.2) + PhrIndex26_count_*Power(10,0.3) + PhrIndex27_count_*Power(10,0.4) + PhrIndex28_count_*Power(10,0.5) + PhrIndex29_count_*Power(10,0.6) + PhrIndex30_count_*Power(10,0.7) + PhrIndex31_count_*Power(10,0.8) + PhrIndex32_count_*Power(10,0.9) + PhrIndex33_count_*Power(10,1) + PhrIndex34_count_*Power(10,1.1) + PhrIndex35_count_*Power(10,1.2) + PhrIndex36_count_*Power(10,1.3) + PhrIndex37_count_*Power(10,1.4) + PhrIndex38_count_*Power(10,1.5) + PhrIndex39_count_*Power(10,1.6) + PhrIndex40_count_*Power(10,1.7) + PhrIndex41_count_*Power(10,1.8) + PhrIndex42_count_*Power(10,1.9) + PhrIndex43_count_*Power(10,2) + PhrIndex44_count_*Power(10,2.1) + PhrIndex45_count_*Power(10,2.2) + PhrIndex46_count_*Power(10,2.3) + PhrIndex47_count_*Power(10,2.4) + PhrIndex48_count_*Power(10,2.5) + PhrIndex49_count_*Power(10,2.6) + PhrIndex50_count_*Power(10,2.7) + PhrIndex51_count_*Power(10,2.8) + PhrIndex52_count_*Power(10,2.9) + PhrIndex53_count_*Power(10,3) + PhrIndex54_count_*Power(10,3.1) + PhrIndex55_count_*Power(10,3.2) + PhrIndex56_count_*Power(10,3.3) + PhrIndex57_count_*Power(10,3.4) + PhrIndex58_count_*Power(10,3.5) + PhrIndex59_count_*Power(10,3.6) + PhrIndex60_count_*Power(10,3.7) + PhrIndex61_count_*Power(10,3.8) + PhrIndex62_count_*Power(10,3.9) + PhrIndex63_count_*Power(10,4)) UE_Power_Headroom_Num,
                sum(PhrIndex0_count_ + PhrIndex1_count_ + PhrIndex2_count_ + PhrIndex3_count_ + PhrIndex4_count_ + PhrIndex5_count_ + PhrIndex6_count_ + PhrIndex7_count_ + PhrIndex8_count_ + PhrIndex9_count_ + PhrIndex10_count_ + PhrIndex11_count_ + PhrIndex12_count_ + PhrIndex13_count_ + PhrIndex14_count_ + PhrIndex15_count_ + PhrIndex16_count_ + PhrIndex17_count_ + PhrIndex18_count_ + PhrIndex19_count_ + PhrIndex20_count_ + PhrIndex21_count_ + PhrIndex22_count_ + PhrIndex23_count_ + PhrIndex24_count_ + PhrIndex25_count_ + PhrIndex26_count_ + PhrIndex27_count_ + PhrIndex28_count_ + PhrIndex29_count_ + PhrIndex30_count_ + PhrIndex31_count_ + PhrIndex32_count_ + PhrIndex33_count_ + PhrIndex34_count_ + PhrIndex35_count_ + PhrIndex36_count_ + PhrIndex37_count_ + PhrIndex38_count_ + PhrIndex39_count_ + PhrIndex40_count_ + PhrIndex41_count_ + PhrIndex42_count_ + PhrIndex43_count_ + PhrIndex44_count_ + PhrIndex45_count_ + PhrIndex46_count_ + PhrIndex47_count_ + PhrIndex48_count_ + PhrIndex49_count_ + PhrIndex50_count_ + PhrIndex51_count_ + PhrIndex52_count_ + PhrIndex53_count_ + PhrIndex54_count_ + PhrIndex55_count_ + PhrIndex56_count_ + PhrIndex57_count_ + PhrIndex58_count_ + PhrIndex59_count_ + PhrIndex60_count_ + PhrIndex61_count_ + PhrIndex62_count_ + PhrIndex63_count_)  UE_Power_Headroom_Denom
                  -- select *
            from  apachedataingestion.lte.powerheadroomreport_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
       ) T18 on M0.D_DTM=T18.D_DTM and M0.NE_ID=T18.NE_ID  and M0.LOCATION=T18.LOCATION   
       full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            -- select                                     
                sum(CellUnavailableTimeLock) CellUnavailableTimeLock,
                sum(CellUnavailableTimeDown) CellUnavailableTimeDown,                
                sum(CellUnavailableTimeES) CellUnavailableTimeES
                  -- select *
            from  apachedataingestion.lte.cellunavailabletime_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
       ) T19 on M0.D_DTM=T19.D_DTM and M0.NE_ID=T19.NE_ID  and M0.LOCATION=T19.LOCATION  
       full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            -- select                                     
                sum(RedirLbCac) RedirLbCac
                  -- select *
            from  apachedataingestion.lte.redirectionbycac_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
       ) T20 on M0.D_DTM=T20.D_DTM and M0.NE_ID=T20.NE_ID  and M0.LOCATION=T20.LOCATION  
       full join  (
            SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,SITE_ID,Timezone,
            -- select                                                    
                sum(ConnRelease_CpPreemption) ConnRelease_CpPreemption
                  -- select *
            from  apachedataingestion.lte.rrcconnectionrelease_lte_hr_silver
          WHERE Daily =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd")   
            GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0],SITE_ID,Timezone 
       ) T21 on M0.D_DTM=T21.D_DTM and M0.NE_ID=T21.NE_ID  and M0.LOCATION=T21.LOCATION  

  ) K            
) X
left join bh_conf B on cast(decode(len(SYSTEM_ID2),5,left(SYSTEM_ID2,2),10,left(SYSTEM_ID2,2),mod(left(SYSTEM_ID2,3),300) ) as int)=B.SUB_MARKET_ID and hour(D_DTM)=B.Busy_Hr
left join (
    select replace(ENODEB_ID,'VZ_','eNB_') NE_ID,concat('cNum',ECELL_ID) LOCATION,
    max(MARKET_NAME) MARKET_NAME,max(SUB_MARKET_ID) SUB_MARKET_ID,max(SUB_MARKET_NAME) SUB_MARKET_NAME,max(BAND_NAME) BAND_NAME 
    from hive_metastore.default.vz_conf_ecell_silver group by ENODEB_ID,ECELL_ID
) Y on X.NE_ID=Y.NE_ID and X.LOCATION=Y.LOCATION
Group by X.NE_ID,X.SYSTEM_ID2,X.LOCATION,D_DTM,Timezone,SITE_ID,bhcode,Y.MARKET_NAME,Y.SUB_MARKET_ID,Y.SUB_MARKET_NAME,Y.BAND_NAME;


delete from apachedataingestion.lte.widetable_4g_site_gold2
where D_DTM >= date_format(date_add(now(), int(:days)),"yyyy-MM-dd") and D_DTM < date_format(date_add(now(), int((:days)+1)),"yyyy-MM-dd") 
;



insert into apachedataingestion.lte.widetable_4g_site_gold2
with conf_market (select cast(SUB_MARKET_ID AS INT) SUB_MARKET_ID,SUB_MARKET_NAME,REGION,Time_Offset from hive_metastore.default.vz_conf_market)
select 
  case when len(replace(location,'cNum',''))=1 then replace(location,'cNum','') when len(replace(location,'cNum',''))=2 then left(replace(location,'cNum',''),1) else left(replace(location,'cNum',''),2) end Sector_ID,
  case when len(replace(location,'cNum',''))=1 then 1 else right(replace(location,'cNum',''),1) end Carrier_ID,
  date(D_DTM) CalendarDate,hour(D_DTM) Hour24,
  case when B.Time_Offset=-5 then  convert_timezone('America/New_York','UTC',A.D_DTM) 
	 when B.Time_Offset=-6 then  convert_timezone('America/Chicago','UTC',A.D_DTM) 
	 when B.Time_Offset=-8 then  convert_timezone('America/Los_Angeles','UTC',A.D_DTM) 
 end D_DTM_UTC,
 case when LEN(A.SUB_MARKET_ID)=2 then concat('0',replace(A.ne_id,'eNB_','')) else replace(A.ne_id,'eNB_','') end ne_id2,
 A.*
 from (
            select 
            COALESCE(M0.Created_DTM,T1.Created_DTM) Created_DTM,          
            COALESCE(M0.Busy_Hour,T1.Busy_Hour) Busy_Hour,
            COALESCE(M0.NE_ID,T1.NE_ID) NE_ID,
            COALESCE(M0.SYSTEM_ID,T1.SYSTEM_ID) SYSTEM_ID,
            COALESCE(M0.LOCATION,T1.LOCATION) LOCATION,
            COALESCE(M0.D_DTM,T1.D_DTM) D_DTM,
            COALESCE(M0.Timezone,T1.Timezone) Timezone,
            COALESCE(M0.Site_ID,T1.Site_ID) Site_ID,
            COALESCE(M0.MARKET_NAME,T1.MARKET_NAME) MARKET_NAME,
            COALESCE(M0.SUB_MARKET_ID,T1.SUB_MARKET_ID) SUB_MARKET_ID,
            COALESCE(M0.SUB_MARKET_NAME,T1.SUB_MARKET_NAME) SUB_MARKET_NAME,
            COALESCE(M0.BAND_NAME,T1.BAND_NAME) BAND_NAME,
            RRC_connection_failure_rate_NUM,
            RRC_connection_failure_rate_DENOM,
            Total_ERAB_Drop_Rate_NUM,
            Total_ERAB_Drop_Rate_DENOM,
            VoLTE_Drop_Rate_NUM,
            VoLTE_Drop_Rate_DENOM,
            dl_user_throughput_num,
            dl_user_throughput_denom,
            ul_user_throughput_num,
            ul_user_throughput_denom,
            volte_dl_pdcp_packet_loss_num,
            volte_dl_pdcp_packet_loss_denom,
            avg_rrc_connected_users_num,
            avg_rrc_connected_users_denom,
            dl_rlc_data_volume,
            ul_rlc_data_volume,
            RRC_Connection_Attempts_num,
            endc_add_attempt_num,
            SEA_ENDC_SgNBAdd_Fail_num,
            SEA_ENDC_SgNBAdd_Fail_denom,
            SEA_ENDC_SCG_Failure_NUM,
            SEA_ENDC_SCG_Failure_DENOM,
            ERAB_Setup_Failure_Rate_NUM,
            ERAB_Setup_Failure_Rate_DENOM,
            Context_Setup_Failure_NUM,
            Context_Setup_Failure_DENOM,
            RRC_connection_Drop_Rate_NUM,
            RRC_connection_Drop_Rate_DENOM,
            Context_Drop_Rate_NUM,
            Context_Drop_Rate_DENOM,
            RRC_Reestablishment_Rate_NUM,
            RRC_Reestablishment_Rate_DENUM,
            RRC_Reest_Failure_Rate_NUM,
            RRC_Reest_Failure_Rate_DENOM,
            Intra_eNB_HOFR_NUM,
            Intra_eNB_HOFR_DENOM,
            X2_Total_HOFR_NUM,
            X2_Total_HOFR_DENOM,
            S1_HOFR_NUM,
            S1_HOFR_DENOM,
            Total_DL_RLC_ReTx_NUM,
            Total_DL_RLC_ReTx_DENOM,
            TTI_Bundling_Activation_FR_NUM,
            TTI_Bundling_Activation_FR_DENOM,
            DL_Residual_BLER_NUM,
            DL_Residual_BLER_DENOM,
            UL_Residual_BLER_NUM,
            UL_Residual_BLER_DENOM,
            VoLTE_DL_Residual_BLER_NUM,
            VoLTE_DL_Residual_BLER_DENOM,
            VoLTE_UL_Residual_BLER_NUM,
            VoLTE_UL_Residual_BLER_DENOM,
            PDCCH_DL_Blocking_NUM,
            PDCCH_DL_Blocking_DENOM,
            PDCCH_UL_Blocking_NUM,
            PDCCH_UL_Blocking_DENOM,
            Intra_eNB_Total_HO_Attempts,
            X2_Total_HO_Attempts,
            S1_HO_Attempt,
            DL_PRB_Utilization_NUM,
            DL_PRB_Utilization_DENOM,
            UL_PRB_Utilization_NUM,
            UL_PRB_Utilization_DENOM,
            Avg_TTI_Bundling_Users_NUM,
            Avg_TTI_Bundling_Users_DENOM,
            Avg_Volte_Users_NUM,
            Avg_Volte_Users_DENOM,
            Data_Drop_Rate_NUM,
            Data_Drop_Rate_DENOM,
            max_rrc_connected_users,
            RRC_Connection_HoldTime_s_NUM,
            RRC_Connection_HoldTime_s_DENOM,
            DL_MCS_NUM,
            DL_MCS_DENOM,
            MAC_DL_Cell_Throughput_NUM,
            MAC_DL_Cell_Throughput_DENOM,
            Inter_Freq_HOFR_NUM,
            Inter_Freq_HOFR_DENOM,
            PUCCH_SINR_NUM,
            PUCCH_SINR_DENOM,
            PUSCH_SINR_NUM,
            PUSCH_SINR_DENOM,
            UL_MCS_NUM,
            UL_MCS_DENOM,
            MAC_UL_Cell_Throughput_NUM,
            MAC_UL_Cell_Throughput_DENOM,
            Avg_CQI_NUM,
            Avg_CQI_DENOM,
            Avg_CA_Active_Length_per_activation_attempt_NUM,
            Avg_CA_Active_Length_per_activation_attempt_DENOM,
            CA_activation_rate_NUM,
            CA_activation_rate_DENOM,
            DL_Init_BLER_NUM,
            DL_Init_BLER_DENOM,
            DL_User_Perceived_Throughput_CA_NUM,
            DL_User_Perceived_Throughput_CA_DENOM,
            UL_Init_BLER_NUM,
            UL_Init_BLER_DENOM,
            DL_MAC_Layer_Data_Volume_pCell,
            DL_MAC_Layer_Data_Volume_sCell,
            UL_MAC_Volume_MB UL_MAC_Layer_Data_Volume_CA, -- changed column name            
            Average_Timing_Advance_NUM,
            Average_Timing_Advance_DENOM,
            SEA_CAConfigurationRate_NUM,
            SEA_CAConfigurationRate_DENOM,
            SEA_CAConfigurationFailure_pct_NUM,
            SEA_CAConfigurationFailure_pct_DENOM,
            NB_IOT_RRC_Conn_Setup_Failure_Rate_NUM,
            NB_IOT_RRC_Conn_Setup_Failure_Rate_DENOM,
            NB_IOT_Avg_RRC_Connections_NUM,
            NB_IOT_Avg_RRC_Connections_DENOM,
            NB_IOT_DL_MAC_PDU_Tput_Kbps_NUM,
            NB_IOT_DL_MAC_PDU_Tput_Kbps_DENOM,
            NB_IOT_UL_MAC_PDU_Tput_Kbps_NUM,
            NB_IOT_UL_MAC_PDU_Tput_Kbps_DENOM,
            NB_IOT_RRC_Paging_Request_Atts,
            NBIoT_Total_ERAB_Drop_Rate_pct_NUM,
            NBIoT_Total_ERAB_Drop_Rate_pct_DENOM,
            NBIoT_DL_Initial_BLER_pct_NUM,
            NBIoT_DL_Initial_BLER_pct_DENOM,
            NBIoT_DL_Residual_BLER_pct_NUM,
            NBIoT_DL_Residual_BLER_pct_DENOM,
            NBIoT_UL_Initial_BLER_pct_NUM,
            NBIoT_UL_Initial_BLER_pct_DENOM,
            NBIoT_UL_Residual_BLER_pct_NUM,
            NBIoT_UL_Residual_BLER_pct_DENOM,
            NBIoT_MaxRRCconnUsers_CeLv0,
            NBIoT_MaxRRCconnUsers_CeLv1,
            NBIoT_MaxRRCconnUsers_CeLv2,
            NBIoT_Air_RLC_Volume,
            eMTC_Downlink_PDCP_Volume,
            eMTC_Downlink_PDCP_Throughput_NUM,
            eMTC_Downlink_PDCP_Throughput_DENOM,
            eMTC_Avg_RRC_Connected_Users_NUM,
            eMTC_Avg_RRC_Connected_Users_DENOM,
            eMTC_Max_RRC_Connected_Users,
            eMTC_Paging_Attempt,
            eMTC_Total_ERAB_Setup_Failure_Rate_NUM,
            eMTC_Total_ERAB_Setup_Failure_Rate_DENOM,
            eMTC_RRC_Connection_Drop_Rate_NUM,
            eMTC_RRC_Connection_Drop_Rate_DENOM,
            eMTC_Total_ERAB_Drop_Rate_NUM,
            eMTC_Total_ERAB_Drop_Rate_DENOM,
            eMTC_Context_Setup_Failure_NUM,
            eMTC_Context_Setup_Failure_DENOM,
            eMTC_Context_Drop_Rate_NUM,
            eMTC_Context_Drop_Rate_DENOM,
            eMTC_Intra_Freq_HO_Fail_Rate_NUM,
            eMTC_Intra_Freq_HO_Fail_Rate_DENOM,
            eMTC_DL_Initial_BLER_pct_NUM,
            eMTC_DL_Initial_BLER_pct_DENOM,
            eMTC_DL_Residual_BLER_pct_NUM,
            eMTC_DL_Residual_BLER_pct_DENOM,
            eMTC_UL_Initial_BLER_pct_NUM,
            eMTC_UL_Initial_BLER_pct_DENOM,
            eMTC_UL_Residual_BLER_pct_NUM,
            eMTC_UL_Residual_BLER_pct_DENOM,
            eMTC_RRC_Setup_Failure_pct_NUM,
            eMTC_RRC_Setup_Failure_pct_DENOM,
            ENDC_Cap_UE_Avg_RRC_Connected_Users_NUM,
            ENDC_Cap_UE_Avg_RRC_Connected_Users_DENOM,
            ENDC_Cap_UE_RRC_Drop_pct_NUM,
            ENDC_Cap_UE_RRC_Drop_pct_DENOM,
            SEA_ENDC_SgNBAdd_Fail_PCT_NCR_NUM,
            SEA_ENDC_SgNBAdd_Fail_PCT_NCR_DENOM,
            DL_MAC_Layer_Data_Volume_CA,
            Cell_Availability_PCT_NUM,
            Cell_Availability_PCT_DENOM,
            NB_IOT_Max_Num_of_RRC_Conn_UE,
            DL_MAC_Volume_MB,
            SEA_ENDC_SgNBAddPrep_Fail_PCT_Num,
            SEA_ENDC_SgNBAddPrep_Fail_PCT_Denom,
            SEA_ENDC_CapUE_AvgEnDcUEAddition_Num,
            SEA_ENDC_CapUE_AvgEnDcUEAddition_Denom,
            SEA_ENDC_CapUE_AvgRRCConnectionHoldTime_s_Num,
            SEA_ENDC_CapUE_AvgRRCConnectionHoldTime_s_Denom,
            SEA_ENDC_NRsyncRecon_Fail_PCT_NCR_Num,
            SEA_ENDC_NRsyncRecon_Fail_PCT_NCR_Denom,
            SEA_ENDC_SgNBAddPrep_Fail_PCT_NCR_Num,
            SEA_ENDC_SgNBAddPrep_Fail_PCT_NCR_Denom,
            volte_ul_pdcp_packet_loss_num,
            volte_ul_pdcp_packet_loss_denom,
            DL_User_Throughput_QCI8_NUM,
            Dl_User_Throughput_QCI8_DENOM,
            Dl_Rlc_Data_volume_QCI8,
            Ul_Rlc_Data_volume_QCI8,
            ERAB_Setup_Failure_Rate_QCI5_NUM,
            ERAB_Setup_Failure_Rate_QCI5_DENOM,
            Total_ERAB_Drop_Rate_QCI5_NUM,
            Total_ERAB_Drop_Rate_QCI5_DENOM,
            ASEU,
            CellCallContextCntAvg_Num,
            CellCallContextCntAvg_Denom,
            CellCallContextCntMax,
            -- add
            NBIoT_Air_RLC_Volume_UL,
            NB_IOT_Context_Drop_Rate_Num,
            NB_IOT_Context_Drop_Rate_Denom,
            -- add
            SEA_ENDC_ERAB_Estab_Fail_pct_Num,
            SEA_ENDC_ERAB_Estab_Fail_pct_Denom,
            SEA_ENDC_IMLB_UE_Count,
            SEA_ENDC_ERAB_Modif_Fail_pct_Num,
            SEA_ENDC_ERAB_Modif_Fail_pct_Denom,
            SEA_ENDC_IntraMenbSgnbNoChgHO_ModifPrep_Fail_pct_Num,
            SEA_ENDC_IntraMenbSgnbNoChgHO_ModifPrep_Fail_pct_Denom,
            SEA_ENDC_IntraMenbSgnbNoChgHO_Modif_Fail_pct_Num,
            SEA_ENDC_IntraMenbSgnbNoChgHO_Modif_Fail_pct_Denom,
            SEA_ENDC_InterMenbSgnbNoChgHO_AddPrep_Fail_pct_Num,
            SEA_ENDC_InterMenbSgnbNoChgHO_AddPrep_Fail_pct_Denom,
            SEA_ENDC_InterMenbSgnbNoChgHO_Add_Fail_pct_Num,
            SEA_ENDC_InterMenbSgnbNoChgHO_Add_Fail_pct_Denom,
            SEA_X2HOInterruptionTime_ms_Num,
            SEA_X2HOInterruptionTime_ms_Denom,
            SEA_PdcpIpLatencyDL_ms_QCI1_Num,
            SEA_PdcpIpLatencyDL_ms_QCI1_Denom,
            CoverageHoleVoice,
            CoverageHoleRsrpVoice,
            HoTriggeringScellRsrpEventA3Tot,
            HoTriggeringScellRsrpEventA3Cnt,
            DL_Active_UE_Num,
            DL_Active_UE_Denom,
            UL_Active_UE_Num,
            UL_Active_UE_Denom,
            UL_RSSI_Num,
            UL_RSSI_Denom,
            DL_RI_Num,
            DL_RI_Denom,
            Intra_Freq_HOFR_Denom,
            Intra_Freq_HOFR_Num,
            MAC_UL_Cell_Throughput_NUM/8  UL_MAC_Volume_MB,
            UL_MAC_Layer_Data_Volume_pCell,
            UL_MAC_Layer_Data_Volume_sCell,
            -- add DENG-2770
             5G_Fraction_of_RRC_Call_Hold_Time_pct_Denom	,
             5G_Fraction_of_RRC_Call_Hold_Time_pct_Num	,
             CBRA_RACH_SR_Denom	,
             CBRA_RACH_SR_Num	,
             SEA_ENDC_Addition_Duration_Sec_Denom	,
             SEA_ENDC_Addition_Duration_Sec_Num	,
             SEA_ENDC_CapUE_AvgRRCconnectedUsers_Denom	,
             SEA_ENDC_CapUE_AvgRRCconnectedUsers_Num	,
             SEA_ENDC_CapUE_MaxEnDcUEAddition	,
             SEA_ENDC_CapUE_MaxRRCconnectedUsers	,
             SEA_ENDC_DL_THROUGHPUT_FR1_Mbps_Denom	,
             SEA_ENDC_DL_THROUGHPUT_FR1_Mbps_Num	,
             SEA_ENDC_DL_THROUGHPUT_FR1FDD_Mbps_Denom	,
             SEA_ENDC_DL_THROUGHPUT_FR1FDD_Mbps_Num	,
             SEA_ENDC_DL_THROUGHPUT_FR1TDD_Mbps_Denom	,
             SEA_ENDC_DL_THROUGHPUT_FR1TDD_Mbps_Num	,
             SEA_ENDC_DL_THROUGHPUT_FR2_Mbps_Denom	,
             SEA_ENDC_DL_THROUGHPUT_FR2_Mbps_Num	,
             SEA_ENDC_DL_VOLUME_FR1_MB	,
             SEA_ENDC_DL_VOLUME_FR1FDD_MB	,
             SEA_ENDC_DL_VOLUME_FR1TDD_MB	,
             SEA_ENDC_DL_VOLUME_FR2_MB	,
             SEA_ENDC_DSS_LTE_DL_PRB_Ratio_pct_Denom	,
             SEA_ENDC_DSS_LTE_DL_PRB_Ratio_pct_Num	,
             SEA_ENDC_DSS_LTE_DL_PRB_UTIL_pct_Denom	,
             SEA_ENDC_DSS_LTE_DL_PRB_UTIL_pct_Num	,
             SEA_ENDC_DSS_LTE_DL_TTI_Ratio_pct_Denom	,
             SEA_ENDC_DSS_LTE_DL_TTI_Ratio_pct_Num	,
             SEA_ENDC_DSS_LTE_DL_TTI_UTIL_pct_Denom	,
             SEA_ENDC_DSS_LTE_DL_TTI_UTIL_pct_Num	,
             SEA_ENDC_DSS_LTE_UL_PRB_Ratio_pct_Denom	,
             SEA_ENDC_DSS_LTE_UL_PRB_Ratio_pct_Num	,
             SEA_ENDC_DSS_LTE_UL_PRB_UTIL_pct_Denom	,
             SEA_ENDC_DSS_LTE_UL_PRB_UTIL_pct_Num	,
             SEA_ENDC_DSS_LTE_UL_TTI_Ratio_pct_Denom	,
             SEA_ENDC_DSS_LTE_UL_TTI_Ratio_pct_Num	,
             SEA_ENDC_DSS_LTE_UL_TTI_UTIL_pct_Denom	,
             SEA_ENDC_DSS_LTE_UL_TTI_UTIL_pct_Num	,
             SEA_ENDC_DSS_NR_DL_PRB_Ratio_pct_Denom	,
             SEA_ENDC_DSS_NR_DL_PRB_Ratio_pct_Num	,
             SEA_ENDC_DSS_NR_DL_TTI_Ratio_pct_Denom	,
             SEA_ENDC_DSS_NR_DL_TTI_Ratio_pct_Num	,
             SEA_ENDC_DSS_NR_UL_PRB_Ratio_pct_Denom	,
             SEA_ENDC_DSS_NR_UL_PRB_Ratio_pct_Num	,
             SEA_ENDC_DSS_NR_UL_TTI_Ratio_pct_Denom	,
             SEA_ENDC_DSS_NR_UL_TTI_Ratio_pct_Num	,
             SEA_ENDC_UL_THROUGHPUT_FR1_Mbps_Denom	,
             SEA_ENDC_UL_THROUGHPUT_FR1_Mbps_Num	,
             SEA_ENDC_UL_THROUGHPUT_FR2_Mbps_Denom	,
             SEA_ENDC_UL_THROUGHPUT_FR2_Mbps_Num	,
             SEA_ENDC_UL_VOLUME_FR1_MB	,
             SEA_ENDC_UL_VOLUME_FR2_MB	,
             SEA_NON_ENDC_DL_THROUGHPUT_Mbps_Denom	,
             SEA_NON_ENDC_DL_THROUGHPUT_Mbps_Num	,
             SEA_NON_ENDC_DL_VOLUME_MB	,
             SEA_NON_ENDC_UL_THROUGHPUT_Mbps_Denom	,
             SEA_NON_ENDC_UL_THROUGHPUT_Mbps_Num	,
             SEA_NON_ENDC_UL_VOLUME__MB	,
             UE_Power_Headroom_Denom	,
             UE_Power_Headroom_Num,
            ReadCellUnavailableTimeLock_s_SUM,
            ReadCellUnavailableTimeES_s_SUM,
            CellAvailPmPeriodTime_s_SUM,
            CellUnavailableTimeLock,
            CellUnavailableTimeDown,                
            CellUnavailableTimeES,
RRC_Rejection_CpCc,
RRC_Rejection_CapaCac,
HPA_RRC_Setup_Failure_Rate_Num,
HPA_RRC_Setup_Failure_Rate_Denom,
HPA_ERAB_Setup_Failure_Rate_Num,
HPA_ERAB_Setup_Failure_Rate_Denom,
HPA_ERAB_Drop_Rate_Num,
HPA_ERAB_Drop_Rate_Denom,
HPA_DL_User_Throughput_Num,
HPA_DL_User_Throughput_Denom,
HPA_DL_RLC_Vol,
HPA_UL_RLC_Vol,
RRC_Redirection_CAC,
RRC_Release_Preemption            
            from hive_metastore.default.widetable_4g_site_gold2_tmp M0
            full join hive_metastore.default.widetable_4g_site_gold2_tmp2  	T1 on M0.D_DTM=T1.D_DTM and  M0.NE_ID=T1.NE_ID and M0.LOCATION=T1.LOCATION 
            where M0.D_DTM >= date_format(date_add(now(), int(:days)),"yyyy-MM-dd") and M0.D_DTM < date_format(date_add(now(), int((:days)+1)),"yyyy-MM-dd")  
  ) A
 inner join conf_market B on A.SUB_MARKET_ID=B.SUB_MARKET_ID
 where A.SUB_MARKET_ID is not null and A.D_DTM >= date_format(date_add(now(), int(:days)),"yyyy-MM-dd") and A.D_DTM < date_format(date_add(now(), int((:days)+1)),"yyyy-MM-dd") 
;



/*



ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN RRC_Rejection_CpCc DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN RRC_Rejection_CapaCac DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN HPA_RRC_Setup_Failure_Rate_Num DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN HPA_RRC_Setup_Failure_Rate_Denom DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN HPA_ERAB_Setup_Failure_Rate_Num DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN HPA_ERAB_Setup_Failure_Rate_Denom DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN HPA_ERAB_Drop_Rate_Num DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN HPA_ERAB_Drop_Rate_Denom DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN HPA_DL_User_Throughput_Num DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN HPA_DL_User_Throughput_Denom DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN HPA_DL_RLC_Vol DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp ADD COLUMN HPA_UL_RLC_Vol DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp2 ADD COLUMN RRC_Redirection_CAC DOUBLE;
ALTER TABLE hive_metastore.default.widetable_4g_site_gold2_tmp2 ADD COLUMN RRC_Release_Preemption DOUBLE;

ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN RRC_Rejection_CpCc DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN RRC_Rejection_CapaCac DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN HPA_RRC_Setup_Failure_Rate_Num DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN HPA_RRC_Setup_Failure_Rate_Denom DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN HPA_ERAB_Setup_Failure_Rate_Num DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN HPA_ERAB_Setup_Failure_Rate_Denom DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN HPA_ERAB_Drop_Rate_Num DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN HPA_ERAB_Drop_Rate_Denom DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN HPA_DL_User_Throughput_Num DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN HPA_DL_User_Throughput_Denom DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN HPA_DL_RLC_Vol DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN HPA_UL_RLC_Vol DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN RRC_Redirection_CAC DOUBLE;
ALTER TABLE apachedataingestion.lte.widetable_4g_site_gold2 ADD COLUMN RRC_Release_Preemption DOUBLE;

*/

